In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
from conf import BasicConf as conf
from dataproc import proc

In [2]:
conf.MIMIC_3_DIR

'/home/xiexc/workspace/hierarchy_icd/mimicdata/mimic3'

In [3]:
Y = 'full' #use all available labels in the dataset for prediction
notes_file = '%s/NOTEEVENTS.csv' % conf.MIMIC_3_DIR # raw note events downloaded from MIMIC-III
vocab_size = 'full' #don't limit the vocab size to a specific number
vocab_min = 3 #discard tokens appearing in fewer than this many documents

In [4]:
dfproc = pd.read_csv('%s/PROCEDURES_ICD.csv' % conf.MIMIC_3_DIR,dtype={"ICD9_CODE": str,"HADM_ID":int})
dfdiag = pd.read_csv('%s/DIAGNOSES_ICD.csv' % conf.MIMIC_3_DIR,dtype={"ICD9_CODE": str,"HADM_ID":int})

In [5]:
dfdiag['absolute_code'] = dfdiag.apply(lambda row: str(proc.code_reformat(str(row[4]), True)), axis=1)
dfproc['absolute_code'] = dfproc.apply(lambda row: str(proc.code_reformat(str(row[4]), False)), axis=1)

In [6]:
dfcodes = pd.concat([dfdiag, dfproc])

In [7]:
dfcodes.to_csv('%s/ALL_CODES.csv' % conf.MIMIC_3_DIR, index=False,
               columns=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'absolute_code'],
               header=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'])

In [8]:
#In the full dataset (not just discharge summaries)
dfcodes = pd.read_csv('%s/ALL_CODES.csv' % conf.MIMIC_3_DIR, dtype={"ICD9_CODE": str,"HADM_ID":int})

In [9]:
notes = pd.read_csv(notes_file,dtyoe={'HADM_ID':int})

/home/xiexc/miniconda3/envs/torch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
notes = notes[notes['CATEGORY'] == 'Discharge summary']

In [11]:
train_hadm = pd.read_csv(f'{conf.MIMIC_3_DIR}/train_full_hadm_ids.csv', header=None)
train_hadm.columns = ['HADM_ID']
train_hadm.head()

,HADM_ID
0,169433
1,178124
2,139808
3,117638
4,190945


In [12]:
train_notes = notes[notes['HADM_ID'].isin(train_hadm.HADM_ID.values)]
index = int(np.random.random() * len(train_notes))
record = train_notes.iloc[index]
summary = record['TEXT']
codes = dfcodes[dfcodes['HADM_ID'] == record['HADM_ID']]['ICD9_CODE'].values
print(codes)
print(summary)

['414.01' '411.1' '997.1' '427.31' '401.9' '272.0' '555.9' 'E878.2'
 'V10.49' '36.12' '37.22' '88.56' '88.53' '36.15' '39.61']
Admission Date:  [**2197-4-26**]       Discharge Date:  [**2197-5-6**]

Date of Birth:   [**2130-3-5**]       Sex:  M

Service:  CARD-[**Last Name (un) **]

HISTORY OF PRESENT ILLNESS:   This is a 67 year old male who
presented to his primary care provider with [**Name Initial (PRE) **] chief complaint
of progressive dyspnea on exertion over the past 18 months.
The patient's wife reported that the patient has become
progressively more and more dyspneic upon walking up a flight
of stairs and has episodes every day that resolve with rest.
The patient denied ever experiencing any chest pain in
association with these episodes.  The patient subsequently
underwent a stress test in [**2197-3-28**], which was stopped
after six minutes and 34 seconds of the [**Doctor First Name **] protocol
secondary to shortness of breath.  The patient did not
experience any chest pain

In [13]:
import re
import regex
import json
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import RegexpTokenizer

In [14]:
notes = notes.sort_values(by=['SUBJECT_ID', 'HADM_ID'])
def parse_notes(notes):
#     json_results = json.loads(notes.to_json(orient='index'))
    parsed_notes = notes.copy()
    parsed_docs = []
    for index, row in tqdm(notes.iterrows(), total=notes.shape[0]):
        summary = row['TEXT'].strip()
        parsed_sections = parse_document(summary)
        merged_sections = merge_nearby_sections(parsed_sections)
        merged_sections = section_postprocess(parsed_sections)
        parsed_docs.append(merged_sections)
    parsed_notes['TEXT'] = parsed_docs
    return parsed_notes

def parse_document(document):
    pattern = regex.compile(r'^([a-zA-Z]+[\s]{0,2})+(\[(\*){1,3}.*(\*){1,3}\])?:')
    sentences = document.split('\n')
    sections = []
    current_section = []
    current_topic = None
    for sentence in sentences:
        ## find next section
        groups = pattern.match(sentence)
        if groups is not None:
            if len(current_section):
                sections.append({'topic':current_topic, 'content':current_section})
            current_topic = ''.join(groups.captures(1))
            current_section = [sentence]
        else:
            current_section.append(sentence)
    sections.append({'topic':current_topic, 'content':current_section})
    return sections

def merge_nearby_sections(sections):
    merged_sections = []
    merged_topic = sections[0]['topic']
    merged_content = sections[0]['content']
    for index, section in enumerate(sections[:-1]):
        topic = section['topic']
        content = section['content']
        ## no need to merge next section
        if content[-1] == '':
            merged_sections.append({'topic':merged_topic,'content':merged_content})
            merged_topic = sections[index+1]['topic']
            merged_content = sections[index+1]['content']
        else:
            merged_content = merged_content + sections[index+1]['content']
    merged_sections.append({'topic':merged_topic,'content':merged_content})
    return merged_sections

def section_postprocess(parsed_sections):
    ## rule 1
    for section in parsed_sections:
        topic = str(section['topic']).lower()
        content = section['content']
        word_count = 0
        for sentence in content:
            word_count += len(sentence)
        if topic == 'service' and word_count >=100:
            section['topic'] = 'ADDENDUM'
    return parsed_sections      


parsed_notes = parse_notes(notes)
parsed_notes.to_csv('{}/parsed_notes.csv'.format(conf.MIMIC_3_DIR),index=False,header=True)
# for section in merged_sections:
#     print(section['content'])

In [15]:
parsed_notes = pd.read_csv('{}/parsed_notes.csv'.format(conf.MIMIC_3_DIR))

In [16]:
import ast
parsed_notes['TEXT'] = parsed_notes['TEXT'].apply(lambda x:ast.literal_eval(x))

In [17]:
parsed_sections = parse_document(summary)
merged_doc = merge_nearby_sections(parsed_sections)
merged_doc = section_postprocess(merged_doc)
print(merged_doc)

[{'topic': 'Admission Date', 'content': ['Admission Date:  [**2197-4-26**]       Discharge Date:  [**2197-5-6**]', '']}, {'topic': 'Date of Birth', 'content': ['Date of Birth:   [**2130-3-5**]       Sex:  M', '']}, {'topic': 'Service', 'content': ['Service:  CARD-[**Last Name (un) **]', '']}, {'topic': 'HISTORY OF PRESENT ILLNESS', 'content': ['HISTORY OF PRESENT ILLNESS:   This is a 67 year old male who', 'presented to his primary care provider with [**Name Initial (PRE) **] chief complaint', 'of progressive dyspnea on exertion over the past 18 months.', "The patient's wife reported that the patient has become", 'progressively more and more dyspneic upon walking up a flight', 'of stairs and has episodes every day that resolve with rest.', 'The patient denied ever experiencing any chest pain in', 'association with these episodes.  The patient subsequently', 'underwent a stress test in [**2197-3-28**], which was stopped', 'after six minutes and 34 seconds of the [**Doctor First Name **]

In [18]:
topics = {}
for key,row in parsed_notes.iterrows():
    doc = row['TEXT']
    for section in doc:
        topic = str(section['topic']).lower()
        if topic in topics:
            topics[topic] += 1
        else:
            topics[topic] = 1
            
topics = sorted(topics.items(), key=lambda x:x[1],reverse=True)
print(len(topics))
print(topics)


merged_topics = {
    'discharge diagnosis':['discharge diagnoses','diagnosis','final diagnosis','admission diagnosis','admitting diagnosis'],
    'brief hospital course':['hospital course','summary of hospital course by systems','hospital course by systems','hospital course by system','summary of hospital course by system','history of hospital course by systems','review of hospital course by systems','brief summary of hospital course'],
    'major surgical or invasive procedure':['procedure','procedures','description of procedure','procedures performed','procedure performed','procedure in detail','major surgical or invasive procedures','major surgical procedures','surgical course'],
    'physical exam':['physical examination on presentation','physical exam on admission','physical examination on admission','physical examination','discharge physical exam','physical exam on discharge','physical therapy','physical examination on discharge','physical exam at discharge','discharge physical','discharge exam','discharge examination'],
    'history of present illness':['history of the present illness','present illness','illness','chief complaint and history of present illness'],
    'pertinent results':['pertinent studies'],
    
}
preserve_topics = [
                   'discharge diagnosis',
                   'brief hospital course',
                   'major surgical or invasive procedure',
                   'history of present illness',
                   'pertinent results',
                   'imaging',
                   'physical exam',
                   'past medical history',
                   'discharge medications',
                   'discharge disposition',
                   'medications on admission',
                   'discharge condition',
                   'discharge instructions', 
                   'social history',
                   'followup instructions',
                   'family history',
                   'facility',
                   'medications on discharge',
                   'addendum'
                ]

36357
[('disp', 103369), ('admission date', 59588), ('service', 58105), ('date of birth', 51967), ('history of present illness', 50809), ('allergies', 50404), ('past medical history', 50304), ('social history', 45347), ('discharge disposition', 44120), ('discharge medications', 44088), ('discharge diagnosis', 41959), ('medications on admission', 41902), ('family history', 41657), ('chief complaint', 41579), ('attending', 41045), ('discharge condition', 40488), ('physical exam', 39872), ('discharge instructions', 39680), ('brief hospital course', 38887), ('major surgical or invasive procedure', 38735), ('pertinent results', 37923), ('followup instructions', 37679), ('impression', 35596), ('heent', 30597), ('facility', 26674), ('completed by', 23174), ('neck', 22531), ('neuro', 21169), ('ext', 19724), ('t', 19609), ('d', 18621), ('dictated by', 18617), ('cv', 17927), ('general', 16724), ('abd', 16677), ('lungs', 16666), ('abdomen', 16217), ('gen', 16048), ('provider', 14586), ('mental st

In [19]:
topic_mapping_dict = dict()
for k,v in merged_topics.items():
    for item in v:
        topic_mapping_dict[item] = k

In [22]:
def mapping_topics(doc, topic_mapping_dict):
    mapped_doc = []
    for section in doc:
        topic = str(section['topic']).lower()
        if topic in topic_mapping_dict:
            mapped_doc.append({'topic':topic_mapping_dict[topic],'content':section['content']})
        else:
            mapped_doc.append(section)
    return mapped_doc

def doc_split_by_topics(doc, topics):
    merged_content = []
    current_topic = 'service'
    merged_doc = []
    for section in doc:
        topic = str(section['topic']).lower()
        content = section['content']
        if topic in topics:
            if len(merged_content):
                merged_doc.append({'topic':current_topic,'content':merged_content})
            current_topic = topic
            merged_content = content
        else:
            merged_content = merged_content + content
    merged_doc.append({'topic':current_topic,'content':merged_content})
    
    return merged_doc

def notes_topic_split(notes, topics):
    split_docs = []
    split_notes = notes.copy()
    for index, row in tqdm(notes.iterrows(),total=notes.shape[0]):
        doc = row['TEXT']
        split_doc = doc_split_by_topics(doc, topics)
        split_docs.append(split_doc)
    
    split_notes['TEXT'] = split_docs
        
    return split_notes

topic_split_notes = notes_topic_split(parsed_notes, preserve_topics)

# mapped_doc = mapping_topics(doc, topic_mapping_dict)

In [23]:
topic_split_notes.to_csv('{}/topic_split_notes.csv'.format(conf.MIMIC_3_DIR),index=False)

In [24]:
def tokenize_notes(notes):
    #retain only alphanumeric
    tokenizer = RegexpTokenizer(r'\w+')
    docs = []
    for index, row in tqdm(notes.iterrows(), total=notes.shape[0]):
        tokenize_text = []
        text = row['TEXT']
        for section in text:
            content = '\n'.join(section['content'])
            topic = section['topic']
            ## replace de-identified item and date
            content = re.sub(r'\[\*+[^\]]*\*+\]',' ',content)
            tokens = [t.lower() for t in tokenizer.tokenize(content) if not t.isnumeric()]
            tokens = ' '.join(tokens)
            tokenize_text.append({'topic':topic,'content':tokens})
        docs.append(tokenize_text)
    notes['TEXT'] = docs
    return notes

full_notes = tokenize_notes(topic_split_notes.copy())

In [25]:
full_notes.head(1000)['TEXT'][60]

[{'topic': 'service',
  'content': 'unit no admission date discharge date date of birth sex f service neonatology'},
 {'topic': 'history of present illness',
  'content': 'history of present illness baby girl was the kg product of a and week gestation born to a year old gravida ii para now i blood type a positive antibody negative hepatitis surface antigen negative rubella immune rpr nonreactive gbs positive mother followed at clinic for diabetes diagnosed in has a history of mild diabetic retinopathy mother admitted to due to preterm rupture of membranes for clear fluid given gestational age mother s labor was induced with pitocin labor progressed well one hour prior to delivery fetal tachycardia was noted vaginal delivery required low forceps assistance report of mild shoulder dystocia initially infant was floppy dusky no immediate spontaneous cry and then onset of crying within seconds on warmer infant stimulated continued crying and received blow by oxygen left arm noted to have de

In [26]:
full_notes.to_csv('{}/disch_full.csv'.format(conf.MIMIC_3_DIR))

In [27]:
full_notes = full_notes.sort_values(by=['SUBJECT_ID', 'HADM_ID'])
dfcodes = dfcodes.sort_values(by=['SUBJECT_ID', 'HADM_ID'])
print(len(dfcodes['HADM_ID'].unique()), len(full_notes['HADM_ID'].unique()))

58976 52726


In [28]:
import math
full_notes['HADM_ID'] = full_notes['HADM_ID'].astype('int64')
full_hadm_ids = full_notes['HADM_ID'].unique()
filter_codes = dfcodes[dfcodes['HADM_ID'].isin(full_hadm_ids)]

In [29]:
filter_codes = filter_codes.sort_values(by=['SUBJECT_ID', 'HADM_ID'])
filter_codes.to_csv('{}/ALL_CODES_filtered.csv'.format(conf.MIMIC_3_DIR), index=False)

In [30]:
print(len(filter_codes['ICD9_CODE'].unique()))
print(len(dfproc[dfproc['HADM_ID'].isin(full_hadm_ids)]['ICD9_CODE'].unique()))
print(len(dfdiag[dfdiag['HADM_ID'].isin(full_hadm_ids)]['ICD9_CODE'].unique()))

all_codes = filter_codes['ICD9_CODE'].unique()
with open('{}/FULL_CODES.csv'.format(conf.MIMIC_3_DIR),'w') as of:
    for code in all_codes:
        of.write(str(code)+'\n')

8908
1989
6919


In [31]:
heads = full_notes.head(30)
heads

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,44005,3,145834,2101-10-31,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
1,4788,4,185777,2191-03-23,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
2,20825,6,107064,2175-06-15,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
3,20070,9,150750,2149-11-13,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
4,57115,9,150750,2149-11-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"[{'topic': 'service', 'content': 'name unit no..."
5,17390,10,184167,2103-07-06,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
6,30120,11,194540,2178-05-11,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
7,50972,12,112213,2104-08-20,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
8,20168,13,143045,2167-01-15,NaN,NaN,Discharge summary,Report,NaN,NaN,"[{'topic': 'service', 'content': 'admission da..."
9,57099,13,143045,2167-01-15,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"[{'topic': 'service', 'content': 'name c unit ..."


In [32]:
## concat notes file and codes file
def concat_data(notes_df, codes_df):
    concate_data = []
    notes_gen = next_notes(notes_df)
    labels_gen = next_labels(codes_df)
    for index, (hadm_id, subj_id, texts) in enumerate(notes_gen):
        cur_hadm, cur_subj, cur_labels = next(labels_gen)
        if index % 1000 == 0:
            print('{} finished'.format(index))
            
        if cur_hadm == hadm_id:
            merged_texts = merge_same_hadm_note(texts)
            merged_labels = merge_codes(cur_labels)
            concate_data.append({'SUBJECT_ID':cur_subj, 'HADM_ID':cur_hadm, 'TEXT':merged_texts, 'LABELS':merged_labels})
        else:
            print("couldn't find matching hadm_id. data is probably not sorted correctly")
            break
    print(index)
    return concate_data

def merge_same_hadm_note(notes):
    
    topic_to_index = dict()
    merged_sections = notes[0]
    for section in notes[1:]:
        merged_sections = merged_sections + section
        
    merged_note = []
    topic_count = 0
    for index, section in enumerate(merged_sections):
        topic = str(section['topic']).lower()
        content = section['content']
        if topic in topic_to_index:
            index = topic_to_index[topic]
            merged_note[index]['content'] = merged_note[index]['content'] + ' ' + content
        else:
            topic_to_index[topic] = topic_count
            topic_count += 1
            merged_note.append(section)  
    return merged_note
    
def merge_codes(codes):
    return ';'.join(codes)

def next_notes(notes_df):
    cur_hadm = None
    cur_subj = None
    cur_text = []
    for index, row in notes_df.iterrows():
        hadm_id = row['HADM_ID']
        subj_id = row['SUBJECT_ID']
        text = row['TEXT']
        if cur_hadm is not None:
            if hadm_id != cur_hadm or subj_id != cur_subj:
                yield cur_hadm, cur_subj, cur_text
                cur_hadm = hadm_id
                cur_subj = subj_id
                cur_text = [text]
            else:
                cur_text.append(text)
        else:
            cur_hadm = hadm_id
            cur_subj = subj_id
            cur_text = [text]
    yield cur_hadm, cur_subj, cur_text
                
def next_labels(codes_df):
    cur_hadm = None
    cur_subj = None
    cur_label = None
    for index, row in codes_df.iterrows():
        hadm_id = row['HADM_ID']
        subj_id = row['SUBJECT_ID']
        label = str(row['ICD9_CODE'])
        if cur_hadm is not None:
            if hadm_id != cur_hadm or subj_id != cur_subj:
                yield cur_hadm, cur_subj, cur_label
                cur_hadm = hadm_id
                cur_subj = subj_id
                cur_label = [label]
            else:
                cur_label.append(label)
        else:
            cur_hadm = hadm_id
            cur_subj = subj_id
            cur_label = [label]
    yield cur_hadm, cur_subj, cur_label 
    
labeled_notes = concat_data(full_notes, filter_codes)  
print(len(labeled_notes))

0 finished
1000 finished
2000 finished
3000 finished
4000 finished
5000 finished
6000 finished
7000 finished
8000 finished
9000 finished
10000 finished
11000 finished
12000 finished
13000 finished
14000 finished
15000 finished
16000 finished
17000 finished
18000 finished
19000 finished
20000 finished
21000 finished
22000 finished
23000 finished
24000 finished
25000 finished
26000 finished
27000 finished
28000 finished
29000 finished
30000 finished
31000 finished
32000 finished
33000 finished
34000 finished
35000 finished
36000 finished
37000 finished
38000 finished
39000 finished
40000 finished
41000 finished
42000 finished
43000 finished
44000 finished
45000 finished
46000 finished
47000 finished
48000 finished
49000 finished
50000 finished
51000 finished
52000 finished
52725
52726


In [33]:
labeled_notes_df = pd.DataFrame(labeled_notes)

In [34]:
def word_count(row):
    text = row['TEXT']
    word_num = 0
    for section in text:
        content = section['content']
        word_num += len(content.split())
    return word_num

In [35]:
labeled_notes_df['LENGTH'] = labeled_notes_df.apply(word_count, axis=1)
labeled_notes_df.to_csv('{}/notes_labeled.csv'.format(conf.MIMIC_3_DIR),index=None)

In [ ]:
#read in train, dev, test splits
base_name = "%s/disch" % conf.MIMIC_3_DIR #for output
hadm_ids = {}
for splt in ['train', 'dev', 'test']:
    hadm_ids[splt] = set()
    with open('%s/%s_full_hadm_ids.csv' % (conf.MIMIC_3_DIR, splt), 'r') as f:
        for line in f:
            hadm_ids[splt].add(line.rstrip())

train_disch_notes = labeled_notes_df[labeled_notes_df['HADM_ID'].isin(hadm_ids['train'])]
train_disch_notes.to_csv('%s_train_split.csv' % (base_name), index=False)

dev_disch_notes = labeled_notes_df[labeled_notes_df['HADM_ID'].isin(hadm_ids['dev'])]
dev_disch_notes.to_csv('%s_dev_split.csv' % (base_name), index=False)

test_disch_notes = labeled_notes_df[labeled_notes_df['HADM_ID'].isin(hadm_ids['test'])]
test_disch_notes.to_csv('%s_test_split.csv' % (base_name), index=False)

In [ ]:
## build vocab
vocab_min = 3
vname = '{}/vocab.csv'.format(conf.MIMIC_3_DIR)

In [ ]:
from scipy.sparse import csr_matrix
import operator
def build_vocab(vocab_min, notes_df, vocab_filename):
    """
        INPUTS:
            vocab_min: how many documents a word must appear in to be kept
            notes_df: (training) data file to build vocabulary from
            vocab_filename: name for the file to output
    """
    note_numwords = []
    #indices where notes start
    note_inds = [0]
    #indices of discovered words
    indices = []
    #holds a bunch of ones
    data = []
    #keep track of discovered words
    vocab = {}
    #build lookup table for terms
    num2term = {}
    #preallocate array to hold number of notes each term appears in
    note_occur = np.zeros(400000, dtype=int)
    
    for index, row in notes_df.iterrows():
        sections = row['TEXT']
        text = sections[0]['content']
        for section in sections[1:]:
            text = text + ' ' + section['content']
        
        numwords = 0
        for term in text.split():
            #put term in vocab if it's not there. else, get the index
            index = vocab.setdefault(term, len(vocab))
            indices.append(index)
            num2term[index] = term
            data.append(1)
            numwords += 1
        #record where the next note starts
        note_inds.append(len(indices))
        indset = set(indices[note_inds[-2]:note_inds[-1]])
        #go through all the word indices you just added, and add to the note occurrence count for each of them
        for ind in indset:
            note_occur[ind] += 1
        note_numwords.append(numwords)
            
    #clip trailing zeros
    note_occur = note_occur[note_occur>0]

    #turn vocab into a list so indexing doesn't get fd up when we drop rows
    vocab_list = np.array([word for word,ind in sorted(vocab.items(), key=operator.itemgetter(1))])

    #1. create sparse document matrix
    C = csr_matrix((data, indices, note_inds), dtype=int).transpose()
    #also need the numwords array to be a sparse matrix
    note_numwords = csr_matrix(1. / np.array(note_numwords))

    #2. remove rows with less than 3 total occurrences
    print("removing rare terms")
    #inds holds indices of rows corresponding to terms that occur in < 3 documents
    inds = np.nonzero(note_occur >= vocab_min)[0]
    print(str(len(inds)) + " terms qualify out of " + str(C.shape[0]) + " total")
    #drop those rows
    C = C[inds,:]
    note_occur = note_occur[inds]
    vocab_list = vocab_list[inds]

    print("writing output")
    with open(vocab_filename, 'w') as vocab_file:
        for word in vocab_list:
            vocab_file.write(word + "\n")


In [ ]:
build_vocab(vocab_min, train_disch_notes, vname)

In [ ]:
## train word embedding
import gensim.models.word2vec as w2v
import os

class ProcessedIter(object):

    def __init__(self, Y, notes_df):
        self.notes_df = notes_df
        
    def __iter__(self):
        for index, row in self.notes_df.iterrows():
            sections = row['TEXT']
            text = sections[0]['content']
            for section in sections[1:]:
                text = text + ' ' + section['content']
            yield text.split()

def word_embeddings(Y, notes_df, embedding_size, min_count, n_iter, out_dir):
    modelname = "processed_%s.w2v" % (Y)
    sentences = ProcessedIter(Y, notes_df)

    model = w2v.Word2Vec(size=embedding_size, min_count=min_count, workers=4, iter=n_iter)
    print("building word2vec vocab")
    
    model.build_vocab(sentences)
    print("training...")
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    out_file = os.path.join(out_dir, modelname)
    print("writing embeddings to %s" % (out_file))
    model.save(out_file)
    return out_file

In [ ]:
out_dir = conf.MIMIC_3_DIR
w2v_file = word_embeddings('full', labeled_notes_df, 100, 0, 5, out_dir)

In [ ]:
## transfer gensim w2v to embed
import gensim
PAD_CHAR = "**PAD**"

def gensim_to_embeddings(wv_file, vocab_file, Y, outfile=None):
    model = gensim.models.Word2Vec.load(wv_file)
    wv = model.wv
    #free up memory
    del model
    vocab = set()
    with open(vocab_file, 'r') as vocabfile:
        for i,line in enumerate(vocabfile):
            line = line.strip()
            if line != '':
                vocab.add(line)
    ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}

    W, words = build_matrix(ind2w, wv)

    if outfile is None:
        outfile = wv_file.replace('.w2v', '.embed')
    #smash that save button
    save_embeddings(W, words, outfile)

    
def build_matrix(ind2w, wv):
    """
        Go through vocab in order. Find vocab word in wv.index2word, then call wv.word_vec(wv.index2word[i]).
        Put results into one big matrix.
        Note: ind2w starts at 1 (saving 0 for the pad character), but gensim word vectors starts at 0
    """
    W = np.zeros((len(ind2w)+1, len(wv.word_vec(wv.index2word[0])) ))
    words = [PAD_CHAR]
    W[0][:] = np.zeros(len(wv.word_vec(wv.index2word[0])))
    for idx, word in tqdm(ind2w.items()):
        if idx >= W.shape[0]:
            break    
        W[idx][:] = wv.word_vec(word)
        words.append(word)
    return W, words

def load_embeddings(embed_file):
    #also normalizes the embeddings
    W = []
    with open(embed_file) as ef:
        for line in ef:
            line = line.rstrip().split()
            vec = np.array(line[1:]).astype(np.float)
            vec = vec / float(np.linalg.norm(vec) + 1e-6)
            W.append(vec)
        #UNK embedding, gaussian randomly initialized 
        print("adding unk embedding")
        vec = np.random.randn(len(W[-1]))
        vec = vec / float(np.linalg.norm(vec) + 1e-6)
        W.append(vec)
    W = np.array(W)
    return W
    
def save_embeddings(W, words, outfile):
    with open(outfile, 'w') as o:
        #pad token already included
        for i in range(len(words)):
            line = [words[i]]
            line.extend([str(d) for d in W[i]])
            o.write(" ".join(line) + "\n")

In [ ]:
gensim_to_embeddings('%s/processed_full.w2v' % conf.MIMIC_3_DIR, '%s/vocab.csv' % conf.MIMIC_3_DIR, Y)

In [44]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import os
import csv
from collections import defaultdict
DATA_DIR = '/home/xiexc/workspace/hierarchy_icd/mimicdata/'

def load_code_descriptions(version='mimic3'):
    #load description lookup from the appropriate data files
    desc_dict = defaultdict(str)
    ## read diagnosis code
    diagnose_df = pd.read_csv(os.path.join(DATA_DIR,'D_ICD_DIAGNOSES.csv'),dtype={"ICD9_CODE": str,"HADM_ID":int})
    for index, row in diagnose_df.iterrows():
        code = str(row['ICD9_CODE'])
        desc = row['LONG_TITLE']
        desc_dict[proc.code_reformat(code, True)] = desc
    ## read procedure code
    procedure_df = pd.read_csv(os.path.join(DATA_DIR,'D_ICD_PROCEDURES.csv'),dtype={"ICD9_CODE": str,"HADM_ID":int})
    for index, row in procedure_df.iterrows():
        code = str(row['ICD9_CODE'])
        desc = row['LONG_TITLE']
        desc_dict[proc.code_reformat(code, False)] = desc
        
    with open('%s/ICD9_descriptions' % DATA_DIR, 'r') as labelfile:
        for i,row in enumerate(labelfile):
            row = row.rstrip().split()
            code = row[0]
            if code not in desc_dict.keys():
                desc_dict[code] = ' '.join(row[1:])
    return desc_dict
    
    
def vocab_index_descriptions(vocab_file, vectors_file):
    #load lookups
    vocab = set()
    with open(vocab_file, 'r') as vocabfile:
        for i,line in enumerate(vocabfile):
            line = line.strip()
            if line != '':
                vocab.add(line)
    ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}
    w2ind = {w:i for i,w in ind2w.items()}
    desc_dict = load_code_descriptions()
        
    tokenizer = RegexpTokenizer(r'\w+')

    with open(vectors_file, 'w') as of:
        w = csv.writer(of, delimiter=' ')
        w.writerow(["CODE", "VECTOR"])
        for code, desc in tqdm(desc_dict.items()):
            #same preprocessing steps as in get_discharge_summaries
            tokens = [t.lower() for t in tokenizer.tokenize(desc) if not t.isnumeric()]
            inds = [w2ind[t] if t in w2ind.keys() else len(w2ind)+1 for t in tokens]
            w.writerow([code] + [str(i) for i in inds])


In [45]:
vocab_index_descriptions('%s/vocab.csv' % conf.MIMIC_3_DIR,'%s/description_vectors.vocab' % conf.MIMIC_3_DIR)

In [52]:
labeled_notes_df.head()

,HADM_ID,LABELS,SUBJECT_ID,TEXT,LENGTH
0,145834,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...,3,"[{'topic': 'service', 'content': 'admission da...",2130
1,185777,042;136.3;799.4;276.3;790.7;571.5;041.11;V09.0...,4,"[{'topic': 'service', 'content': 'admission da...",1398
2,107064,403.91;444.0;997.2;276.6;276.7;285.9;275.3;V15...,6,"[{'topic': 'service', 'content': 'admission da...",997
3,150750,431;507.0;428.0;584.9;276.5;401.9;96.72;96.04,9,"[{'topic': 'service', 'content': 'admission da...",1340
4,184167,V30.00;774.2;765.25;765.15;V29.0;99.83;99.15;96.6,10,"[{'topic': 'service', 'content': 'admission da...",482


In [53]:
from collections import Counter, defaultdict
Y = 50
#first calculate the top k
counts = Counter()
for index, row in labeled_notes_df.iterrows():
    for label in str(row['LABELS']).split(';'):
        counts[label] += 1

In [55]:
import operator
codes_sort_by_frequency = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
codes_50 = [code[0] for code in codes_sort_by_frequency[:Y]]

## write result to file
with open('{}/TOP_50_CODES.csv'.format(conf.MIMIC_3_DIR),'w') as of:
    for code in codes_50:
        of.write(str(code)+'\n')
print(codes_50)

['401.9', '38.93', '428.0', '427.31', '414.01', '96.04', '96.6', '584.9', '250.00', '96.71', '272.4', '518.81', '99.04', '39.61', '599.0', '530.81', '96.72', '272.0', '285.9', '88.56', '244.9', '486', '38.91', '285.1', '36.15', '276.2', '496', '99.15', '995.92', 'V58.61', '507.0', '038.9', '88.72', '585.9', '403.90', '311', '305.1', '37.22', '412', '33.24', '39.95', '287.5', '410.71', '276.1', 'V45.81', '424.0', '45.13', 'V15.82', '511.9', '93.90']


In [ ]:
# another_counter = Counter({'401.9': 20053, '38.93': 14444, '428.0': 12842, '427.31': 12594, '414.01': 12179, '96.04': 9932, '96.6': 9161, '584.9': 8907, '250.00': 8784, '96.71': 8619, '272.4': 8504, '518.81': 7249, '99.04': 7147, '39.61': 6809, '599.0': 6442, '530.81': 6156, '96.72': 5926, '272.0': 5766, '285.9': 5296, '88.56': 5240, '244.9': 4788, '486': 4733, '38.91': 4575, '285.1': 4499, '36.15': 4390, '276.2': 4358, '496': 4296, '99.15': 4172, '995.92': 3792, 'V58.61': 3698, '507.0': 3592, '038.9': 3580, '88.72': 3500, '585.9': 3367, '403.90': 3350, '311': 3347, '305.1': 3272, '37.22': 3248, '412': 3203, '33.24': 3188, '39.95': 3178, '287.5': 3002, '410.71': 3001, '276.1': 2985, 'V45.81': 2943, '424.0': 2878, '45.13': 2849, 'V15.82': 2741, '511.9': 2693, '37.23': 2659, 'V45.82': 2651, '403.91': 2566, 'V29.0': 2529, '424.1': 2517, '785.52': 2501, 'V58.67': 2497, '427.89': 2396, '327.23': 2328, '997.1': 2313, '99.55': 2304, '93.9': 2233, '584.5': 2223, '276.0': 2221, '99.83': 2185, '774.2': 2183, '54.91': 2178, '518.0': 2137, '33.1': 2129, 'V05.3': 2119, '416.8': 2118, '276.7': 2113, '493.90': 2113, '458.29': 2094, '88.53': 2074, '274.9': 2033, '99.07': 1995, '458.9': 1992, '733.00': 1910, '36.12': 1897, '780.39': 1879, '585.6': 1868, '34.91': 1809, '38.95': 1791, '518.5': 1778, '427.1': 1767, '31.1': 1756, '34.04': 1689, '425.4': 1666, '38.97': 1657, '411.1': 1649, '40.0': 1599, 'V12.51': 1568, '300.00': 1552, '357.2': 1552, '43.11': 1523, '998.11': 1508, '36.13': 1498, '278.00': 1483, '88.41': 1477, '790.7': 1472, 'E879.8': 1466, '600.00': 1464, '293.0': 1434, '414.00': 1429, '008.45': 1428, '35.21': 1416, '276.8': 1400, 'V30.01': 1391, '285.21': 1367, '443.9': 1364, 'V45.01': 1360, '578.9': 1355, '276.51': 1349, '276.52': 1347, 'E878.8': 1317, '769': 1299, '66.0': 1295, '431': 1294, '276.5': 1294, '427.5': 1281, 'V49.86': 1275, '799.02': 1267, '707.03': 1260, '995.91': 1251, '571.5': 1250, '263.9': 1240, 'V10.3': 1228, '428.32': 1221, '33.22': 1218, '36.06': 1215, '99.05': 1213, '428.33': 1210, '998.12': 1199, '427.32': 1191, 'E878.2': 1189, '070.54': 1187, '99.2': 1184, 'V10.46': 1180, '491.21': 1177, '998.59': 1175, '571.2': 1163, '280.9': 1157, '278.01': 1150, '36.07': 1147, '33.23': 1144, '428.23': 1127, '250.60': 1114, '785.51': 1108, '294.8': 1094, '428.22': 1081, '413.9': 1078, '285.29': 1078, '300.4': 1076, '89.64': 1060, '36.01': 1058, '560.1': 1056, '779.3': 1047, 'V12.54': 1043, '787.91': 1037, '570': 1034, '770.81': 1033, 'E849.7': 1024, '788.20': 1012, '37.61': 1003, '280.0': 991, '286.9': 985, '572.3': 984, '790.92': 971, 'V66.7': 968, '276.3': 967, 'E888.9': 965, '041.4': 958, 'V30.00': 949, '36.11': 949, '789.59': 946, '996.62': 943, '577.0': 943, '345.90': 933, '250.40': 933, '348.5': 923, '998.2': 900, '305.00': 884, 'V58.65': 881, '45.0': 877, '715.90': 873, '765.27': 869, 'V31.01': 853, '362.01': 849, '414.8': 848, '415.19': 845, '288.60': 844, '88.55': 834, '64.0': 830, '562.10': 830, '198.5': 813, '564.00': 810, '99.62': 798, 'V43.65': 795, '291.81': 788, '482.41': 786, '428.30': 785, '512.1': 780, '17.0': 778, 'V43.64': 777, '433.10': 773, '45.23': 773, '397.0': 764, '765.18': 762, '197.7': 762, '997.4': 754, '38.7': 754, 'V10.05': 748, '518.82': 743, '198.3': 738, '88.42': 736, '96.05': 733, '493.20': 732, '197.0': 729, '682.6': 728, 'V50.2': 724, '585.3': 723, 'V45.02': 711, '88.47': 710, '365.9': 701, '434.91': 700, '303.91': 699, '041.11': 693, '86.22': 692, 'E878.1': 691, 'V17.3': 687, '54.59': 686, '338.29': 685, '518.89': 679, 'E934.2': 678, 'E885.9': 678, 'V15.3': 675, '578.1': 674, '86.59': 674, '518.84': 672, '593.9': 663, '348.30': 659, '44.43': 656, 'V09.0': 655, 'V43.3': 655, '070.70': 655, '572.2': 654, '348.4': 654, '14.0': 652, '745.5': 651, '441.4': 650, '576.1': 649, '553.3': 643, '714.0': 640, '112.0': 638, '349.82': 636, '434.11': 630, 'V15.81': 626, '492.8': 621, '15.9': 619, '430': 617, '041.04': 609, '88.48': 607, '39.5': 606, '779.81': 601, '453.8': 599, '45.16': 596, '99.1': 594, '294.10': 593, '785.0': 588, '583.81': 578, '296.80': 578, '303.90': 577, '287.4': 576, 'V45.11': 573, '276.6': 568, '780.57': 563, '996.74': 562, 'V44.0': 560, '37.21': 558, '410.41': 557, '780.09': 555, 'E933.1': 554, '99.6': 554, '331.0': 553, '88.52': 552, '427.41': 551, '331.4': 547, '997.31': 545, '96.07': 543, 'E932.0': 543, '693.0': 539, '410.11': 532, '515': 529, '276.4': 523, '042': 522, '38.45': 521, '789.5': 518, '519.19': 518, '99.25': 517, '765.28': 515, '426.0': 514, '346.90': 514, '038.11': 511, 'V70.7': 510, '35.22': 506, '785.59': 505, 'E880.9': 505, 'V10.11': 502, '332.0': 501, 'V10.83': 501, '996.81': 500, '536.3': 499, '459.81': 499, '599.7': 497, 'V44.1': 497, '99.20': 496, '253.6': 494, '770.6': 494, '99.29': 490, '13.1': 489, '51.87': 489, '765.19': 488, '765.26': 487, '428.21': 484, '787.20': 482, '441.2': 479, 'V45.89': 477, '557.0': 475, '724.2': 475, '30.9': 474, '784.3': 472, '041.3': 470, '33.21': 470, '342.90': 465, '414.02': 464, '038.42': 464, 'V42.2': 463, '996.72': 460, '765.17': 456, '286.7': 456, '284.1': 454, '46.39': 454, '438.89': 452, '250.02': 452, '275.3': 451, 'V46.2': 451, '401.1': 451, 'E879.0': 450, '37.78': 445, '770.89': 444, '997.5': 444, '51.22': 443, '51.85': 442, '37.83': 440, '37.72': 439, '250.50': 435, '286.6': 432, '423.9': 430, '250.80': 430, '93.90': 430, '535.50': 430, '790.4': 430, '398.91': 429, '428.31': 427, '438.20': 426, '707.15': 426, '356.9': 426, '198.89': 425, '45.62': 425, '440.20': 425, '36.14': 424, '577.1': 424, '482.1': 423, '707.22': 423, '348.1': 422, '46.0': 420, 'V10.52': 420, '37.34': 419, 'V10.82': 419, '780.2': 419, 'E878.0': 418, '37.26': 416, '578.0': 416, '733.90': 415, '997.02': 412, '250.61': 412, '780.6': 410, '456.21': 409, '428.20': 408, '35.12': 407, '776.6': 406, '39.0': 405, '37.0': 405, '250.13': 401, '568.0': 400, 'V87.41': 400, '38.92': 399, '591': 398, '041.7': 398, '41.31': 397, '416.0': 395, '861.21': 394, '81.62': 393, '682.2': 392, '39.79': 392, '038.49': 391, '93.59': 391, '997.3': 388, '786.3': 386, '427.81': 385, '747.0': 382, '305.01': 382, '305.60': 381, '396.2': 380, '421.0': 380, '432.1': 380, '728.88': 378, '707.14': 377, '39.9': 375, '39.29': 373, 'V15.88': 372, '50.11': 371, '31.42': 370, '81.63': 369, '275.41': 368, '86.28': 368, '162.8': 367, '038.0': 366, '99.61': 365, '852.21': 363, '39.72': 363, 'E849.8': 362, '42.33': 362, '41.0': 360, '292.81': 359, '860.0': 355, '81.91': 355, '765.16': 353, '86.04': 353, '576.2': 353, '276.69': 353, '285.22': 351, '437.3': 350, '729.1': 350, '724.5': 350, '54.12': 349, '410.91': 346, '35.23': 345, '303.01': 345, 'E849.0': 344, '574.20': 343, '79.35': 342, '456.20': 341, '396.3': 336, '779.89': 336, '774.6': 334, '112.2': 333, 'V10.51': 333, '572.4': 333, '873.0': 332, '790.29': 332, '537.89': 331, 'V42.0': 330, '585.4': 329, '453.41': 329, '22.0': 329, '564.09': 328, '86.11': 328, '562.12': 328, '202.80': 326, '276.50': 326, '54.11': 325, '34.09': 325, '79.36': 324, '155.0': 324, '518.83': 323, '428.43': 323, '440.24': 322, 'E947.8': 322, '162.3': 321, '42.23': 319, 'V08': 319, '710.0': 317, '284.8': 316, '40.3': 315, '94.62': 315, '211.3': 314, '600.01': 313, '250.41': 312, '746.4': 311, '77.79': 309, '250.01': 309, '45.25': 308, '97.23': 308, '89.68': 305, '197.6': 305, '707.05': 304, '805.4': 304, '780.52': 303, '303.93': 302, 'V12.59': 299, '88.57': 299, 'E878.6': 299, '518.4': 299, '041.85': 299, '765.15': 298, '805.2': 298, '291.0': 298, '84.51': 297, '39.64': 297, '340': 297, '571.1': 297, '530.85': 295, '788.30': 295, '784.7': 295, '493.22': 295, '733.13': 295, '185': 294, '041.19': 293, '790.01': 293, '780.60': 292, '707.07': 292, '13.0': 291, '50.59': 290, '414.2': 290, '787.01': 289, '45.73': 288, '93.0': 288, '255.4': 287, '458.0': 285, '696.1': 283, '35.3': 283, 'V12.72': 283, '775.6': 282, '81.05': 281, '765.25': 280, '43.19': 280, 'E928.9': 280, '21.2': 279, '999.31': 279, '309.81': 279, '576.8': 278, '348.39': 278, '788.5': 278, '440.0': 277, 'V58.66': 276, '716.90': 276, '304.01': 275, '426.3': 274, 'V49.75': 274, '250.51': 274, '37.27': 273, '070.32': 273, '348.31': 272, '458.8': 271, '571.8': 271, '782.3': 271, '89.6': 269, '532.40': 269, '83.45': 269, '555.9': 268, '410.72': 267, '560.9': 266, '275.2': 263, '585': 263, '707.0': 262, '997.2': 261, '531.40': 261, '784.0': 261, '725': 261, '51.88': 261, '37.33': 259, '250.63': 259, '39.49': 259, '440.23': 259, '482.83': 258, '873.42': 257, '97.44': 256, '433.30': 256, 'V60.0': 255, '92.29': 254, '452': 251, '440.21': 250, '808.2': 250, '135': 249, 'E812.0': 249, '786.59': 248, '196.2': 247, '275.42': 247, '799.4': 245, '86.69': 245, '266.2': 245, '38.44': 244, '196.1': 244, '345.3': 243, '295.90': 243, '782.1': 241, '87.54': 241, '369.4': 241, '288.00': 241, '35.33': 240, '344.1': 240, '556.9': 239, '852.20': 239, '333.94': 239, '37.94': 237, '041.12': 237, '438.11': 236, 'V34.01': 235, '40.11': 235, 'E814.7': 233, '780.97': 233, '567.22': 233, 'E930.8': 232, '80.51': 231, '440.1': 231, '998.32': 231, '562.11': 231, '802.4': 228, '564.1': 227, '519.1': 226, '560.81': 226, '482.9': 226, '423.3': 226, 'V45.86': 225, '569.83': 224, '441.01': 224, '771.81': 223, 'V13.01': 223, '51.98': 222, 'V49.87': 222, '389.9': 221, '362.50': 221, 'E935.2': 221, '482.0': 221, '802.0': 221, '36.05': 220, '35.24': 220, '45.24': 219, 'V45.4': 219, '729.89': 219, '96.56': 218, '189.0': 217, '596.54': 216, 'V44.3': 215, '682.3': 214, 'V45.79': 214, '225.2': 213, '599.70': 213, '255.41': 213, '765.14': 212, '41.5': 212, 'E816.0': 212, '567.2': 211, '792.1': 211, '996.64': 211, '51.1': 210, '996.59': 209, '802.8': 209, '788.29': 209, '197.2': 208, '530.19': 208, 'V44.2': 207, '785.6': 207, '041.6': 207, '455.0': 207, '238.71': 207, '458.2': 205, '998.89': 205, 'V46.11': 205, '965.4': 204, '038.8': 203, '34.03': 203, '261': 203, '295.70': 203, '451.82': 202, '37.12': 202, '86.05': 201, '041.86': 201, '577.2': 201, '250.12': 201, '592.0': 200, 'E879.2': 199, '39.31': 199, '996.73': 199, '933.1': 199, '79.66': 198, '83.82': 198, 'E915': 198, '250.62': 198, '411.89': 198, '415.11': 197, '84.52': 197, 'E879.6': 197, '568.81': 197, '244.0': 197, '444.22': 196, '721.0': 196, '530.10': 195, '238.75': 194, '99.06': 194, '437.0': 194, 'E950.3': 193, '765.24': 193, '296.7': 193, '781.94': 193, '780.61': 193, '572.8': 192, '481': 191, '867.0': 191, '999.9': 191, 'V45.73': 191, '250.11': 190, '510.9': 189, '041.49': 189, '530.7': 188, '724.00': 188, 'V12.04': 188, '707.09': 187, '808.0': 187, '205.00': 186, '35.71': 185, '770.7': 185, '807.01': 185, '567.23': 185, '251.8': 184, '558.9': 184, '494.0': 183, '456.8': 183, 'V09.80': 182, '560.89': 182, '682.7': 182, 'E912': 182, '250.43': 181, '305.90': 181, '493.92': 181, '715.36': 181, '785.50': 181, '770.8': 180, '99.14': 180, '790.5': 180, '425.1': 179, '204.10': 179, '530.3': 179, '920': 179, '203.00': 178, '599.71': 178, '81.02': 178, '83.14': 178, 'V18.0': 178, 'E870.0': 178, '79.32': 177, '999.2': 177, 'V49.83': 177, '81.03': 177, '553.1': 177, '424.2': 177, '428.42': 177, '99.71': 176, '86.74': 175, '305.50': 175, '88.51': 175, 'E950.0': 175, '511.89': 175, '783.21': 174, 'V45.1': 174, '724.02': 174, '61.0': 173, '289.81': 173, 'E879.1': 173, '996.82': 173, '575.0': 173, 'V64.1': 173, '362.21': 172, '319': 172, 'V10.87': 172, '518.51': 172, '783.7': 171, '569.81': 171, '112.5': 170, '781.2': 170, '070.44': 170, '32.01': 170, '338.3': 170, 'V58.69': 169, '588.81': 169, 'E849.5': 169, '13.9': 168, '403.01': 167, '292.0': 167, '401.0': 166, '59.8': 166, '162.9': 166, '070.30': 166, '88.49': 166, '77.61': 165, '23.9': 165, '511.8': 165, '294.20': 164, '426.4': 163, '162.5': 163, '574.10': 162, '969.4': 162, '737.30': 162, '250.53': 162, '57.32': 162, '998.0': 161, '786.09': 161, '787.2': 160, '199.1': 160, '574.51': 160, 'V10.42': 159, '998.31': 158, '11.0': 158, '782.4': 158, 'V12.71': 158, '573.8': 158, '482.42': 158, 'V10.06': 158, '83.39': 157, '427.0': 157, '459.2': 156, '296.20': 156, '780.79': 156, '44.32': 156, '366.9': 156, '281.9': 156, '45.76': 156, '54.4': 155, 'V02.54': 155, '805.6': 155, '531.90': 154, '55.03': 154, '290.40': 154, '51.01': 154, '96.34': 153, 'V88.01': 153, '807.09': 153, '79.39': 153, 'E939.4': 153, '31.79': 151, '790.6': 150, '996.1': 150, '810.00': 150, '535.60': 149, '514': 149, '63.0': 148, '99.21': 148, '770.82': 148, '721.3': 148, '268.9': 148, '038.19': 148, '692.9': 148, 'V45.72': 148, '535.51': 147, '307.9': 147, '473.9': 147, '45.43': 147, '780.62': 147, '805.02': 146, '55.69': 145, 'V29.3': 145, '443.0': 145, '533.90': 145, '420.90': 145, '47.0': 144, 'V64.41': 144, '996.61': 144, '519.02': 144, 'V13.02': 144, '573.3': 143, '54.19': 143, '348.8': 142, '996.85': 142, '55.23': 142, '719.41': 142, '478.6': 142, '707.23': 142, '338.18': 142, '553.21': 141, '707.8': 141, '789.00': 140, '34.51': 140, '512.8': 140, '241.0': 140, '97.05': 139, '765.03': 139, '39.1': 139, '802.6': 139, '041.09': 139, 'E950.4': 139, '820.21': 139, '288.50': 139, 'V12.55': 139, 'V62.84': 139, 'V45.3': 138, 'E884.4': 138, 'V53.31': 138, '728.89': 138, '442.3': 138, 'E935.9': 138, '410.01': 137, '39.63': 137, '45.93': 137, '429.9': 137, '557.9': 137, '284.19': 137, '288.0': 136, '273.8': 136, 'V42.7': 136, 'E849.9': 136, '453.42': 136, '337.1': 136, '54.98': 135, '707.19': 135, '46.32': 135, '277.39': 135, '89.45': 134, '88.45': 134, '860.4': 134, '780.01': 134, '934.1': 134, '729.5': 134, '23.19': 133, '314.01': 133, '794.31': 133, '263.0': 133, '45.91': 133, '766.1': 132, '008.8': 132, '426.11': 132, '197.8': 132, '287.49': 132, '22.': 131, '324.1': 131, '34.92': 131, '410.31': 131, '998.6': 131, '80.99': 131, '997.39': 130, '293.9': 130, '995.1': 130, '50.12': 130, '585.2': 130, '852.01': 130, '250.42': 130, '590.10': 130, '39.51': 129, '426.13': 129, '868.03': 129, '112.3': 129, '569.69': 129, '198.7': 129, '511.81': 129, '784.51': 128, '447.1': 128, '715.96': 128, '117.9': 128, 'V45.2': 128, '34.79': 127, 'V31.00': 127, '157.0': 126, '355.8': 126, '579.3': 126, '45.42': 126, '567.29': 126, '39.50': 125, '93.96': 125, '15.1': 124, '12.4': 124, 'V72.1': 124, '958.4': 124, '288.03': 124, '775.0': 124, '537.83': 124, '48.23': 124, '112.84': 124, '433.11': 124, '81.04': 123, '807.03': 123, 'E944.4': 123, '550.90': 123, '787.22': 123, '23.4': 123, '557.1': 122, '807.02': 122, '403.10': 122, 'V55.0': 122, '038.43': 122, '33.48': 122, '274.0': 121, '787.02': 121, 'E815.0': 121, '466.0': 121, '745.4': 121, '344.00': 121, '46.73': 121, 'V85.0': 121, '83.09': 120, '250.70': 120, '456.1': 120, '251.2': 120, '333.1': 120, '45.75': 119, '453.81': 119, '785.4': 118, '37.36': 118, '707.12': 118, '32.29': 117, '458.21': 117, '348.89': 117, '238.7': 116, '707.24': 116, '775.5': 116, '707.21': 116, 'V16.0': 116, '038.12': 116, '34.21': 116, '519.09': 116, 'V10.79': 115, '501': 115, '747.3': 115, '33.27': 115, '46.11': 115, '575.11': 115, '300.01': 115, '410.21': 114, '054.9': 114, '864.05': 114, '289.59': 114, '305.03': 114, '535.40': 114, 'V45.71': 114, '288.3': 114, 'E930.0': 114, '765.02': 113, '11.8': 113, '730.28': 113, '44.39': 113, '117.3': 113, '707.20': 113, '707.25': 113, '396.8': 112, '451.84': 112, '38.18': 112, '46.21': 112, '97.49': 112, '786.50': 112, '459.0': 112, 'V11.3': 112, '81.08': 112, '997.49': 112, '997.09': 111, '250.82': 111, '995.94': 111, '57.94': 111, '336.3': 111, '238.4': 111, '772.11': 110, '112.89': 110, '13.': 110, '298.9': 110, 'E942.0': 110, '78.17': 109, '272.1': 109, '428.40': 109, '96.08': 109, '97.02': 109, '765.23': 108, '242.90': 108, '038.3': 108, 'V49.76': 108, '770.2': 108, 'E930.5': 107, 'V12.01': 107, 'E884.9': 107, '88.44': 107, '730.08': 106, '38.08': 106, '730.07': 106, '516.8': 106, '728.87': 106, '31.99': 105, '355.9': 105, '55.51': 105, '998.81': 105, '54.25': 105, '593.2': 105, '54.62': 105, '54.21': 105, '249.00': 105, '38.12': 104, '592.1': 104, '873.43': 104, '785.2': 104, 'E942.9': 104, '535.41': 104, '482.82': 103, '758.0': 103, '39.32': 102, '304.00': 102, 'V10.41': 102, '39.98': 102, 'V49.72': 102, '577.8': 102, '801.26': 102, 'E929.0': 102, '81.52': 102, 'V02.59': 102, '88.43': 101, '54.0': 101, '112.1': 101, '86.07': 101, '691.0': 101, 'E947.9': 101, 'E878.3': 101, '444.89': 100, '574.00': 100, '84.11': 100, '805.07': 100, '909.2': 100, '39.56': 100, '440.4': 100, '707.13': 100, '807.2': 99, '455.2': 99, '84.15': 99, '274.01': 99, '574.50': 99, '278.03': 99, 'E881.0': 98, '287.31': 98, '775.7': 98, '852.00': 98, '87.51': 98, 'V10.43': 98, '53.61': 97, '99.77': 97, '88.1': 97, '521.00': 97, '276.9': 97, '289.84': 97, '569.3': 97, '151.0': 97, '77.49': 97, '23.09': 97, '429.89': 97, '197.4': 97, '252.00': 96, 'V12.09': 96, '89.19': 96, '572.0': 95, 'E942.6': 95, '569.85': 95, '99.10': 95, '331.82': 95, '860.2': 95, 'V53.32': 95, '807.04': 95, '465.9': 95, '781.0': 95, 'E812.2': 94, '585.5': 94, 'V10.09': 94, '780.4': 94, '83.21': 94, '453.40': 94, 'V10.03': 94, '372.30': 94, '579.0': 93, '435.9': 93, '196.0': 93, '736.79': 93, '77.69': 93, 'V85.4': 93, '998.83': 93, '482.2': 92, '807.08': 92, 'V12.53': 92, '93.57': 92, '81.06': 92, 'V10.21': 92, '852.26': 92, '600.0': 91, 'E879.9': 91, '99.69': 91, '078.5': 91, '517.8': 91, '65.61': 91, '44.0': 90, '070.51': 90, '35.96': 90, '811.00': 90, '865.00': 90, '553.20': 90, '368.46': 90, '722.10': 89, '746.02': 89, '305.20': 89, '44.44': 89, 'V17.49': 89, 'V15.51': 89, 'V64.2': 88, '83.65': 88, '567.9': 88, '322.9': 88, '157.8': 88, '51.23': 88, '42.0': 87, '441.03': 87, '39.59': 87, '440.22': 87, '198.4': 87, '241.1': 87, 'V15.84': 87, '429.5': 87, '39.73': 87, '801.21': 87, '39.90': 86, '729.92': 86, '724.3': 86, '423.0': 86, '781.3': 86, '34.0': 86, 'V58.83': 86, '250.81': 86, '532.90': 86, '794.8': 86, '345.10': 86, '136.3': 85, '423.8': 85, '038.2': 85, '567.21': 85, '731.8': 85, '852.06': 85, 'V16.3': 85, '98.15': 85, '969.0': 85, '569.5': 85, '29.4': 85, '11.3': 84, '89.14': 84, '86.09': 84, '32.4': 84, '296.50': 84, '728.86': 84, 'V85.41': 84, '250.52': 84, '150.8': 83, 'E937.9': 83, '52.7': 83, '253.5': 83, '721.1': 83, '48.0': 83, '39.25': 83, '583.9': 83, 'E932.3': 82, '513.0': 82, '288.8': 82, '88.64': 82, '530.82': 82, 'V42.83': 82, '590.80': 82, '537.0': 82, 'V46.3': 82, '593.81': 81, '33.91': 81, '891.0': 81, '317': 81, '723.1': 81, 'E812.1': 81, '201.90': 80, '54.72': 80, '14.': 80, '821.01': 80, '568.89': 80, '438.83': 80, '041.02': 80, '402.91': 80, '183.0': 80, '713.5': 80, '438.82': 80, '51.37': 79, '53.51': 79, '89.61': 79, '31.5': 79, '77.89': 79, '999.8': 79, '737.10': 79, '96.33': 79, '442.2': 79, '97.55': 79, '79.15': 79, '416.9': 79, '487.0': 79, '868.04': 79, 'V44.4': 79, '342.92': 79, '710.1': 79, '427.69': 79, '414.12': 79, 'V85.1': 79, 'E870.8': 78, '998.13': 78, '81.64': 78, '723.0': 78, '79.31': 78, '453.2': 78, '20.6': 78, '453.82': 78, '518.52': 78, '351.0': 77, '852.22': 77, '39.71': 77, '53.49': 77, '730.27': 77, '784.5': 77, '211.1': 77, '273.1': 77, '778.4': 76, '45.79': 76, '34.22': 76, '560.2': 76, '282.49': 76, '969.3': 75, '304.21': 75, 'E882': 75, '996.69': 75, '43.99': 75, '79.06': 75, '789.2': 75, '934.9': 75, 'V12.79': 75, '244.1': 75, '253.2': 75, '369.60': 75, '303.00': 75, '305.61': 74, '88.73': 74, '778.3': 74, '89.49': 74, '786.6': 74, '425.5': 74, '218.9': 74, '54.3': 74, '782.0': 73, '776.1': 73, '441.3': 73, '478.31': 73, '715.35': 73, '807.05': 73, '532.00': 73, 'E878.4': 73, '301.83': 73, '729.81': 73, '707.04': 73, '786.2': 73, '39.75': 73, '432.9': 72, '20.2': 72, '428.41': 72, '79.02': 72, '552.21': 72, '820.8': 72, '796.3': 72, '240.9': 72, 'E935.8': 72, '358.00': 72, '46.01': 72, '573.0': 72, '96.59': 72, '053.9': 72, '569.89': 72, '787.29': 72, '39.57': 71, 'V42.82': 71, '079.99': 71, '410.81': 71, '958.7': 71, '807.4': 71, '511.0': 71, '42.41': 71, '93.56': 71, '174.8': 71, '342.91': 71, '296.90': 71, '719.45': 70, '50.22': 70, '51.14': 70, '997.62': 70, '95.43': 70, '996.67': 70, '99.28': 70, '907.2': 70, '250.92': 70, '813.42': 70, '753.12': 70, 'E936.1': 70, '197.5': 70, '696.0': 70, '35.9': 70, '46.75': 69, '777.5': 69, '242.00': 69, '21.01': 69, '991.6': 69, '300.3': 69, '441.02': 69, '155.1': 69, '37.49': 69, '188.8': 69, '191.1': 69, '719.46': 68, '281.1': 68, '283.9': 68, '965.09': 68, '438.9': 68, '852.02': 68, '455.3': 68, '46.23': 68, '041.89': 68, '138': 68, '162.2': 68, '429.3': 68, 'V44.6': 68, '780.65': 68, '455.6': 68, 'V07.1': 67, '582.81': 67, '285.8': 67, '333.2': 67, '709.8': 67, '87.03': 67, '730.17': 67, '070.71': 67, '907.0': 67, 'E850.2': 66, 'V42.81': 66, '38.85': 66, '784.2': 66, 'V10.07': 66, '771.6': 66, '593.4': 66, 'E888.1': 66, 'V14.0': 66, '205.10': 65, '805.01': 65, '275.0': 65, '457.1': 65, '805.06': 65, '995.0': 65, '698.9': 65, 'E966': 64, '79.62': 64, '76.76': 64, '50.3': 64, '536.42': 64, '441.7': 64, '84.3': 64, '801.22': 64, 'E937.8': 64, '78.67': 63, '851.81': 63, '437.2': 63, '275.49': 63, '722.93': 63, '995.93': 63, '910.0': 63, '793.11': 63, '711.06': 63, '607.84': 63, '608.86': 63, 'V45.61': 63, '37.64': 62, '753.29': 62, '55.93': 62, '35.39': 62, 'E884.2': 62, '227.0': 62, '281.0': 62, 'V10.72': 62, '853.01': 62, '770.0': 62, '444.21': 62, '807.06': 62, '337.9': 62, '710.2': 62, 'E850.4': 62, '722.52': 62, '934.8': 62, '823.00': 61, '398.90': 61, '27.51': 61, '369.00': 61, '820.09': 61, 'V55.1': 61, '44.13': 61, 'V16.1': 61, '262': 61, '730.18': 61, '440.8': 61, '295.30': 61, '43.7': 61, '747.81': 61, '996.49': 61, '12.5': 61, '437.9': 61, '789.09': 60, '35.14': 60, '530.20': 60, '510.0': 60, '820.22': 60, '038.40': 60, '50.91': 60, '434.01': 60, '478.74': 60, '799.1': 60, '795.5': 60, '336.1': 60, '35.11': 60, '78.69': 60, '478.29': 59, '531.00': 59, '76.79': 59, '150.5': 59, '343.9': 59, 'E931.0': 59, 'E823.0': 59, '935.1': 59, '48.24': 59, '729.73': 59, '31.74': 59, 'V55.3': 59, 'E936.3': 59, '491.22': 59, '84.17': 59, 'E928.8': 59, '284.89': 59, '345.50': 58, '79.65': 58, '682.4': 58, '824.8': 58, '56.51': 58, 'V10.02': 58, '808.41': 58, '295.62': 58, '742.4': 58, '81.54': 58, 'V45.85': 58, '34.52': 58, '37.31': 57, '50.61': 57, '996.79': 57, '996.01': 57, '283.0': 57, '484.6': 57, '041.10': 57, '331.3': 57, '250.83': 57, '787.6': 57, '46.1': 57, 'V45.88': 57, '733.42': 56, '43.0': 56, '89.62': 56, '87.53': 56, '38.14': 56, '37.71': 56, '420.91': 56, '403.00': 56, '429.83': 56, '522.5': 56, '36.16': 56, '282.5': 56, '174.9': 56, '791.9': 56, '45.8': 55, '42.42': 55, '569.49': 55, '51.0': 55, '485': 55, '372.72': 55, '44.42': 55, '32.28': 55, '530.21': 55, '571.6': 55, '443.81': 55, '455.8': 55, '461.9': 55, '57.71': 55, '715.95': 55, '778.6': 55, '54.23': 55, 'E929.3': 55, '444.0': 54, '996.04': 54, '996.02': 54, '48.1': 54, '338.19': 54, '34.1': 54, '37.99': 54, '78.65': 54, '296.89': 54, '487.1': 54, '574.91': 54, '771.7': 54, '851.86': 54, '577.9': 54, '98.02': 54, '350.1': 54, '22.1': 54, '446.5': 54, 'E826.1': 54, '719.06': 54, '530.5': 54, '416.2': 54, '584.8': 54, '342.00': 54, '32.49': 54, 'V15.52': 54, '99.12': 53, '537.82': 53, '394.1': 53, '733.82': 53, '47.09': 53, '81.51': 53, '287.3': 53, '620.2': 53, '96.35': 53, '86.3': 53, '354.0': 53, '87.74': 53, '996.66': 53, '93.41': 53, '518.1': 53, '394.2': 53, '52.22': 53, '425.9': 53, '46.2': 53, 'E853.2': 52, '423.2': 52, 'E960.0': 52, '444.81': 52, '041.84': 52, '42.52': 52, '721.90': 52, '191.2': 52, 'V10.04': 52, '38.86': 52, '290.0': 52, '753.0': 52, 'V55.2': 52, '97.03': 52, 'V10.89': 52, '47.19': 52, '36.99': 52, '446.4': 52, '786.1': 51, '197.1': 51, '34.24': 51, '53.59': 51, '786.03': 51, '368.2': 51, '39.42': 51, '786.06': 51, '789.51': 51, '88.67': 50, '324.0': 50, '396.0': 50, '805.05': 50, '537.84': 50, '410.92': 50, '45.94': 50, '873.63': 50, '569.41': 50, '560.39': 50, '348.0': 50, 'V85.43': 50, '296.30': 50, '780.93': 50, '86.27': 50, '746.89': 49, '76.74': 49, '76.92': 49, '433.31': 49, '39.74': 49, '447.6': 49, 'E816.1': 49, '294.9': 49, '825.25': 49, '39.52': 49, '37.74': 49, '806.00': 49, '720.0': 49, '351.8': 49, '34.06': 49, '285.3': 49, '40.29': 48, '87.75': 48, '88.91': 48, '491.20': 48, 'E888.8': 48, '440.31': 48, '37.77': 48, '738.4': 48, '569.82': 48, '309.0': 48, '39.2': 48, '305.40': 48, '528.01': 48, '358.01': 48, '99.60': 47, '873.44': 47, '580.89': 47, '588.89': 47, '793.1': 47, '996.71': 47, '772.6': 47, '801.20': 47, '33.28': 47, '305.51': 47, '359.81': 47, '426.82': 47, '345.40': 47, '535.61': 47, '457.8': 47, '789.01': 47, '477.9': 47, '996.63': 47, 'E930.9': 47, '196.3': 47, '571.42': 47, '191.9': 46, '45.74': 46, 'E968.9': 46, '359.4': 46, 'E854.3': 46, '250.10': 46, 'E939.8': 46, '053.19': 46, '801.01': 46, '338.4': 46, '866.02': 46, '84.12': 46, '277.4': 46, '46.52': 46, '813.44': 46, '43.41': 46, '21.71': 46, '443.29': 46, '997.79': 46, '305.63': 46, '93.54': 46, '211.2': 46, '810.02': 46, '443.21': 45, 'V10.47': 45, '731.0': 45, '155.2': 45, '83.95': 45, '623.8': 45, '966.3': 45, '519.2': 45, '550.92': 45, '535.10': 45, '426.12': 45, '783.0': 45, '83.32': 45, '279.00': 45, 'E816.2': 45, '771.8': 45, '997.69': 45, '345.80': 45, 'E942.4': 45, 'E934.8': 45, '198.82': 45, '442.84': 45, '759.89': 45, 'V45.77': 45, 'V13.51': 45, 'V85.42': 45, '722.0': 44, '596.0': 44, '789.06': 44, '197.3': 44, 'E942.1': 44, '424.90': 44, '980.0': 44, '765.22': 44, '865.03': 44, '410.51': 44, '88.54': 44, '429.71': 44, '88.74': 44, '252.01': 44, '571.0': 44, '491.9': 44, '250.22': 44, '227.3': 44, 'V58.11': 44, '46.85': 44, '415.12': 44, '853.00': 43, '763.83': 43, '79.05': 43, '970.8': 43, '492.0': 43, '45.19': 43, '038.10': 43, '271.3': 43, '314.00': 43, '536.49': 43, '414.10': 43, '786.8': 43, '76.72': 43, '454.9': 43, '999.32': 43, '478.5': 43, '425.8': 43, '33.29': 43, '596.8': 42, '965.00': 42, '335.20': 42, '368.8': 42, '338.11': 42, '17.': 42, '851.80': 42, '530.89': 42, '530.0': 42, '566': 42, '154.1': 42, 'E956': 42, '807.07': 42, 'V15.1': 42, '212.7': 42, '46.03': 42, '99.23': 42, '850.11': 42, '15.3': 42, '682.1': 42, 'V15.41': 42, '556.6': 42, '86.01': 41, '378.54': 41, '764.07': 41, '228.04': 41, '156.1': 41, '813.41': 41, '519.4': 41, '295.32': 41, '823.32': 41, '323.9': 41, '289.9': 41, '681.10': 41, 'E939.3': 41, '420.99': 41, '963.0': 41, 'E941.3': 41, '198.1': 41, 'E884.3': 41, '84.56': 41, '786.30': 41, '752.61': 40, '433.20': 40, '924.00': 40, '455.5': 40, '528.9': 40, 'E950.9': 40, '715.98': 40, '386.00': 40, '57.0': 40, '682.0': 40, '42.92': 40, '46.51': 40, '84.72': 40, '79.09': 40, '85.7': 40, '198.2': 39, '357.5': 39, '357.0': 39, '707.06': 39, '812.01': 39, '996.2': 39, '535.01': 39, '38.03': 39, '304.70': 39, '786.05': 39, '96.09': 39, '041.5': 39, '193': 39, '443.24': 39, '750.3': 39, '865.02': 39, '357.6': 39, '722.71': 39, '29.11': 39, '191.3': 38, '788.41': 38, '403.11': 38, '733.99': 38, '569.62': 38, '54.61': 38, '490': 38, '204.00': 38, '823.02': 38, '99.0': 38, '733.19': 38, '45.61': 38, '37.82': 38, '733.09': 38, '99.19': 38, '771.83': 38, '277.3': 38, '801.25': 38, '411.0': 38, '729.72': 38, '863.89': 38, '80.76': 38, '77.68': 38, '494.1': 38, '916.0': 38, '453.86': 38, '77.67': 38, '451.89': 38, '37.28': 38, '34.82': 38, '852.25': 38, '112.4': 37, '33.26': 37, '865.09': 37, '446.6': 37, '518.3': 37, '965.01': 37, '806.05': 37, '304.71': 37, 'E813.6': 37, '39.65': 37, '257.2': 37, '54.63': 37, '153.6': 37, 'E980.0': 37, '110.3': 37, '824.0': 37, '426.52': 37, '536.8': 37, '478.30': 37, '522.4': 37, '824.4': 37, '81.07': 37, '958.8': 37, '423.1': 37, '695.89': 37, '707.02': 37, '995.90': 37, '695.3': 37, '922.2': 37, '110.1': 37, '33.78': 37, '598.9': 37, '34.2': 37, '31.29': 36, '789.07': 36, '787.3': 36, '958.92': 36, 'E938.3': 36, '805.04': 36, 'E819.0': 36, 'V54.19': 36, '277.30': 36, '870.8': 36, '527.2': 36, '565.0': 36, '707.10': 36, '78.55': 36, '153.3': 36, '309.28': 36, '44.3': 36, '537.3': 36, '44.29': 36, '882.0': 36, '575.10': 36, '796.2': 36, 'V27.0': 36, 'E870.6': 36, 'E858.3': 36, 'V55.4': 36, '805.08': 36, '97.37': 36, '795.79': 36, 'V14.8': 36, '041.01': 36, '294.11': 36, '738.19': 36, '94.65': 35, '344.89': 35, '162.4': 35, '753.3': 35, '77.88': 35, '211.4': 35, '38.07': 35, '34.01': 35, '851.82': 35, '719.7': 35, 'V49.73': 35, '191.8': 35, '37.91': 35, 'E879.4': 35, '366.8': 35, '295.72': 35, '726.10': 35, '198.0': 35, '812.09': 35, 'E917.9': 35, '10.9': 35, 'E878.5': 35, '45.82': 35, '263.1': 35, '530.6': 35, '822.0': 34, '410.61': 34, '46.41': 34, 'E939.0': 34, '251.1': 34, '821.23': 34, '305.91': 34, '386.11': 34, '573.9': 34, '53.69': 34, '444.1': 34, '96.48': 34, '575.12': 34, '575.8': 34, '786.52': 34, '79.75': 34, '379.41': 34, '552.3': 34, '567.31': 34, '537.9': 34, '534.40': 34, '157.9': 34, 'V45.76': 34, '519.8': 34, '304.20': 34, '404.91': 34, '773.2': 34, '374.30': 34, '438.13': 34, '456.0': 33, '333.99': 33, '850.5': 33, '567.89': 33, '12.3': 33, '752.51': 33, '619.1': 33, '39.22': 33, '530.2': 33, '462': 33, '614.6': 33, '853.06': 33, '558.3': 33, '812.21': 33, '44.61': 33, '79.37': 33, '76.75': 33, '435.2': 33, '40.41': 33, '333.0': 33, '292.12': 33, '56.0': 33, '780.1': 33, '443.22': 33, '46.74': 33, '79.33': 33, '188.9': 33, '78.19': 33, '449': 33, 'V64.3': 33, '764.05': 33, '24.3': 33, '290.41': 33, '780.99': 33, '784.59': 33, '50.13': 33, '68.49': 33, '528.00': 33, '38.06': 32, '11.4': 32, '864.02': 32, '250.71': 32, '24.2': 32, 'E911': 32, '433.01': 32, '567.8': 32, '21.81': 32, '728.85': 32, '65.49': 32, '196.8': 32, '054.2': 32, 'E935.3': 32, '349.0': 32, '922.32': 32, '716.96': 32, '748.3': 32, '275.03': 32, '80.86': 32, '45.95': 32, '286.0': 32, '279.52': 32, '764.08': 31, 'V10.00': 31, '571.49': 31, '51.84': 31, '588.1': 31, '873.8': 31, '43.5': 31, '414.04': 31, '779.0': 31, '772.10': 31, '153.8': 31, '378.51': 31, '153.9': 31, '359.9': 31, 'V54.13': 31, '593.89': 31, '874.8': 31, '776.7': 31, '493.91': 31, '97.41': 31, '289.89': 31, '555.0': 31, '493.00': 31, '790.93': 31, '37.25': 31, '250.20': 31, '724.4': 31, '999.89': 31, '37.87': 31, '88.01': 31, '781.8': 31, '787.21': 31, '338.12': 31, '599.72': 31, '61.3': 30, 'E916': 30, '39.27': 30, '996.86': 30, '51.02': 30, '965.02': 30, '555.2': 30, '711.05': 30, '770.83': 30, '777.8': 30, '865.01': 30, '996.76': 30, '605': 30, '865.04': 30, '394.0': 30, '79.67': 30, '52.52': 30, '711.01': 30, '79.01': 30, '205.01': 30, '852.05': 30, 'E887': 30, '250.72': 30, '286.3': 30, '68.4': 30, '562.00': 30, '997.91': 30, '309.9': 30, '51.36': 30, '37.79': 30, '569.84': 30, '453.85': 30, '731.3': 30, '595.0': 30, '616.10': 30, '245.2': 30, '921.2': 30, '277.88': 30, 'E931.7': 29, '608.4': 29, '438.19': 29, '998.3': 29, 'V64.42': 29, '78.15': 29, '153.4': 29, '918.1': 29, '96.06': 29, '562.13': 29, '12.0': 29, '530.12': 29, '788.39': 29, '567.38': 29, 'V10.85': 29, '530.4': 29, '53.1': 29, '799.0': 29, '57.49': 29, '704.8': 29, '92.0': 29, '48.63': 29, '742.3': 29, '881.00': 29, '304.10': 29, '746.9': 29, '87.41': 29, '434.90': 29, '57.95': 29, '368.16': 29, '426.10': 29, '972.6': 29, '808.43': 29, '52.93': 29, '626.2': 29, '425.7': 29, '862.0': 29, '482.84': 29, '811.09': 29, '453.84': 29, '228.09': 28, '776.4': 28, '228.01': 28, '255.9': 28, '611.72': 28, '438.0': 28, '823.82': 28, '50.0': 28, '593.3': 28, '289.0': 28, '51.10': 28, '18.4': 28, '810.03': 28, '81.01': 28, '787.03': 28, '996.65': 28, '806.4': 28, '410.42': 28, '435.8': 28, '873.20': 28, '864.15': 28, 'E884.6': 28, '512.0': 28, '15.0': 28, '283.19': 28, '84.01': 28, '50.29': 28, '410.12': 28, '255.5': 28, '540.0': 28, '46.93': 28, '45.02': 28, '305.53': 28, '273.3': 28, '41.05': 28, '110.5': 28, '43.89': 28, '74.1': 28, '759.82': 28, '581.9': 28, '459.89': 28, '530.11': 28, '382.9': 28, '806.20': 28, '48.69': 28, '788.1': 28, '864.03': 28, '296.33': 27, '608.83': 27, '511.1': 27, '426.89': 27, '72.2': 27, '741.90': 27, '996.4': 27, '747.83': 27, '305.70': 27, '536.41': 27, '047.9': 27, '33.0': 27, '345.91': 27, 'V03.82': 27, '99.52': 27, '37.75': 27, '99.09': 27, '054.79': 27, '707.11': 27, '997.01': 27, '51.03': 27, '722.91': 27, '965.1': 27, '891.1': 27, '345.70': 27, 'E939.2': 27, '150.9': 27, '438.7': 27, '695.4': 27, '94.68': 27, '42.81': 27, '825.0': 27, '244.8': 27, '237.5': 27, '151.8': 27, '816.01': 27, '320.3': 27, '68.39': 27, 'V09.81': 27, '38.22': 27, '054.10': 27, '44.67': 27, '20.5': 27, '588.0': 27, '83.44': 27, '300.29': 26, '447.0': 26, '723.4': 26, '34.3': 26, '78.41': 26, '864.01': 26, '959.01': 26, '99.81': 26, 'E968.2': 26, '273.2': 26, '041.00': 26, '395.0': 26, '922.1': 26, '719.47': 26, 'E980.4': 26, 'E819.2': 26, '447.2': 26, '348.9': 26, '459.9': 26, '802.20': 26, 'V12.02': 26, '764.06': 26, '252.1': 26, '80.16': 26, '156.0': 26, '54.51': 26, '62.0': 26, '93.44': 26, 'E901.0': 26, '901.0': 26, '320.82': 26, '038.44': 26, '36.19': 26, '200.20': 26, 'V49.62': 26, '304.11': 26, '35.32': 26, '182.0': 26, '596.1': 26, '320.9': 26, '722.4': 26, '34.4': 26, '555.1': 26, '336.9': 26, '45.72': 26, '305.02': 26, '528.3': 26, '305.93': 26, '32.59': 26, '91.0': 25, '92.27': 25, '441.1': 25, '778.8': 25, '53.9': 25, '763.0': 25, '009.0': 25, '88.76': 25, '962.3': 25, '747.10': 25, '835.00': 25, '582.9': 25, '569.1': 25, '372.00': 25, 'E929.8': 25, '881.01': 25, '220': 25, '228.02': 25, 'V09.91': 25, '745.2': 25, '040.82': 25, '52.82': 25, '569.61': 25, '196.5': 25, '753.10': 25, '97.29': 25, '919.0': 25, '756.79': 25, 'E935.6': 25, '525.8': 25, '873.41': 25, '873.64': 25, '278.8': 25, '438.22': 25, '708.0': 25, '46.79': 25, 'E938.4': 25, '46.13': 25, '404.93': 25, 'E929.9': 25, '83.64': 25, '79.61': 25, '58.6': 25, '866.01': 25, '552.1': 25, '271.0': 25, 'V58.12': 25, '37.68': 25, '913.0': 24, '54.95': 24, '77.81': 24, '55.4': 24, '847.0': 24, '508.1': 24, '342.80': 24, 'V63.2': 24, 'V10.44': 24, '86.83': 24, '812.00': 24, '765.13': 24, '202.88': 24, '44.31': 24, '999.39': 24, '801.00': 24, '853.05': 24, '78.59': 24, '682.8': 24, 'E849.3': 24, '45.14': 24, '250.03': 24, '35.27': 24, '826.0': 24, '873.40': 24, '715.91': 24, '305.21': 24, '447.4': 24, '853.02': 24, '779.5': 24, '644.21': 24, '20.4': 24, '304.03': 24, '78.13': 24, '924.11': 24, 'E965.4': 24, '883.0': 24, '575.4': 24, 'V10.01': 24, '344.60': 24, '998.51': 24, '868.01': 24, 'V16.8': 24, '535.30': 24, '57.81': 24, '864.04': 24, '574.31': 24, '263.8': 24, '303.92': 24, 'V58.62': 24, '32.2': 24, '599.60': 24, '265.1': 24, '365.70': 24, '453.51': 24, 'V58.63': 24, '53.71': 24, '482.40': 23, '765.29': 23, '39.26': 23, '579.8': 23, '22.42': 23, '821.11': 23, '583.89': 23, '318.1': 23, '346.80': 23, '38.46': 23, '595.82': 23, '426.7': 23, '99.59': 23, '824.2': 23, '79.71': 23, '480.9': 23, '088.81': 23, 'E980.3': 23, '296.24': 23, '53.7': 23, '875.0': 23, '83.02': 23, '800.21': 23, '310.2': 23, '999.3': 23, 'E931.9': 23, '39.43': 23, '95.47': 23, '238.79': 23, '772.13': 23, '81.92': 23, '737.39': 23, '80.85': 23, '772.12': 23, '934.0': 23, '446.29': 23, '537.4': 23, '426.53': 23, '782.7': 23, '823.92': 23, '79.19': 23, '905.4': 23, '307.1': 23, '37.81': 23, '791.0': 23, '790.94': 23, '528.09': 23, '806.09': 23, '345.41': 23, 'V43.61': 23, '627.1': 23, '784.69': 23, '301.7': 23, '786.39': 23, '35.05': 23, '695.9': 23, '599.69': 23, '784.42': 23, 'V85.36': 23, 'V65.49': 23, '349.31': 23, '967.8': 22, 'V17.4': 22, '428.1': 22, '709.9': 22, '202.10': 22, '156.2': 22, '870.0': 22, '806.25': 22, '773.1': 22, '698.3': 22, '10.2': 22, '574.71': 22, '969.8': 22, '530.84': 22, '362.81': 22, '325': 22, '433.21': 22, '573.4': 22, '78.57': 22, '32.5': 22, '37.89': 22, '22.19': 22, '801.06': 22, 'V29.8': 22, '54.24': 22, '300.9': 22, '531.70': 22, '759.6': 22, '49.21': 22, '202.00': 22, '733.14': 22, '550.10': 22, '478.24': 22, '751.1': 22, '648.91': 22, '110.4': 22, '442.81': 22, '196.6': 22, '924.10': 22, '860.5': 22, '38.16': 22, '453.83': 22, '682.5': 22, '862.29': 22, '571.3': 22, '788.21': 22, 'E965.0': 22, '710.4': 22, '757.33': 22, '76.2': 22, '331.5': 22, '342.02': 22, '595.9': 22, '788.31': 22, '767.19': 21, 'V46.1': 21, '780.03': 21, '789.04': 21, '436': 21, '812.20': 21, '070.20': 21, '806.04': 21, '552.8': 21, 'V55.6': 21, '574.90': 21, '553.29': 21, '707.01': 21, '473.8': 21, '054.3': 21, '482.49': 21, '970.81': 21, '666.12': 21, '54.74': 21, '519.01': 21, '53.0': 21, '868.13': 21, '21.02': 21, '286.4': 21, '291.1': 21, '801.02': 21, '24.0': 21, 'V54.89': 21, '38.94': 21, '99.02': 21, '429.4': 21, '46.42': 21, 'V20.2': 21, '46.71': 21, '625.6': 21, '532.50': 21, '288.04': 21, '37.24': 21, '453.6': 21, '78.18': 21, 'V17.1': 21, '770.3': 21, '574.61': 21, '20.3': 21, 'E870.4': 21, '362.02': 21, '823.01': 21, '648.24': 21, '333.85': 21, '289.4': 21, '37.76': 21, '536.2': 21, '385.89': 21, '253.7': 21, '593.5': 21, '269.0': 21, '596.51': 21, 'V49.71': 21, '746.85': 21, '301.9': 21, '345.11': 21, '274.00': 21, '443.89': 20, 'V45.12': 20, '218.1': 20, '996.44': 20, '290.3': 20, '813.05': 20, '297.1': 20, '246.9': 20, '42.82': 20, '255.0': 20, 'E858.0': 20, '603.9': 20, '648.22': 20, '50.4': 20, '225.0': 20, '890.0': 20, 'E819.1': 20, '478.75': 20, '28.0': 20, '27.59': 20, '60.9': 20, '751.2': 20, '996.42': 20, '80.05': 20, 'E930.1': 20, '81.35': 20, '997.99': 20, '969.09': 20, '733.6': 20, '996.52': 20, '154.0': 20, 'E953.0': 20, '480.1': 20, '996.31': 20, '438.21': 20, '150.4': 20, '861.22': 20, '244.3': 20, '99.72': 20, '341.9': 20, '567.81': 20, 'V42.5': 20, 'V49.84': 20, '256.4': 20, '80.15': 20, '112.9': 20, 'E849.6': 20, '862.1': 20, '315.9': 20, '87.77': 20, 'V85.38': 20, '793.19': 20, '998.09': 20, '596.4': 19, '89.59': 19, '99.01': 19, '97.82': 19, '724.01': 19, '93.55': 19, '289.82': 19, '873.49': 19, '821.21': 19, '482.30': 19, '250.73': 19, 'E850.0': 19, '304.31': 19, '579.9': 19, '611.0': 19, '751.5': 19, '794.39': 19, '51.12': 19, '685.1': 19, '730.25': 19, '344.61': 19, '730.15': 19, 'E934.4': 19, '250.90': 19, '851.85': 19, '860.1': 19, '860.3': 19, '93.53': 19, '866.00': 19, '356.8': 19, '813.01': 19, '552.20': 19, '29.12': 19, '825.21': 19, '33.43': 19, '427.61': 19, '291.2': 19, '296.40': 19, '478.32': 19, '540.1': 19, '33.99': 19, '041.9': 19, '811.03': 19, '996.68': 19, 'V44.59': 19, '27.0': 19, '009.1': 19, '270.4': 19, '21.03': 19, '289.51': 19, '805.03': 19, '405.91': 19, '553.8': 19, 'V14.2': 19, '453.9': 19, '783.40': 19, '998.30': 19, '85.1': 19, '198.6': 19, '44.69': 19, '808.8': 19, '709.01': 19, '648.93': 19, '900.89': 19, '787.60': 19, '521.09': 19, '569.9': 19, '279.06': 19, 'V85.39': 19, 'V85.32': 19, '284.11': 19, '070.1': 18, '368.40': 18, '52.01': 18, '473.0': 18, '357.82': 18, '729.9': 18, '831.00': 18, '516.3': 18, '840.4': 18, '560.0': 18, 'E888': 18, '070.22': 18, '87.21': 18, '528.0': 18, 'E815.2': 18, '37.32': 18, '815.00': 18, '32.3': 18, '99.17': 18, '279.01': 18, 'E941.1': 18, '54.93': 18, '698.8': 18, '86.89': 18, '31.41': 18, '709.2': 18, '482.39': 18, '277.7': 18, '172.9': 18, '438.53': 18, '48.81': 18, '756.0': 18, '53.8': 18, '31.43': 18, '574.41': 18, '726.5': 18, '54.75': 18, '534.90': 18, '242.20': 18, '580.9': 18, '200.80': 18, '784.49': 18, '803.21': 18, '79.07': 18, '77.65': 18, '831.04': 18, '823.22': 18, '251.3': 18, 'E849.4': 18, 'V12.41': 18, '345.51': 18, '751.69': 18, '307.81': 18, '202.81': 18, '342.81': 18, '935.2': 18, '787.23': 18, 'V85.34': 18, 'V85.44': 18, '519.11': 18, '998.4': 17, '911.0': 17, '447.8': 17, '200.00': 17, '440.29': 17, '722.92': 17, '695.1': 17, '484.7': 17, '296.34': 17, '300.11': 17, '348.3': 17, '484.1': 17, '441.00': 17, '86.72': 17, '44.5': 17, '300.02': 17, 'V02.61': 17, '008.69': 17, '211.5': 17, '801.16': 17, 'E969': 17, '437.8': 17, '433.00': 17, '35.61': 17, '86.75': 17, '574.01': 17, '764.04': 17, '30.2': 17, '83.43': 17, 'E813.0': 17, '801.12': 17, '762.5': 17, '742.2': 17, '309.24': 17, '59.94': 17, '299.80': 17, '389.10': 17, '574.81': 17, '46.94': 17, '238.1': 17, '564.89': 17, '996.75': 17, '686.9': 17, '426.6': 17, '751.62': 17, 'V85.37': 17, '327.26': 17, '48.36': 17, '783.5': 17, '346.00': 17, '909.3': 17, '60.5': 17, '342.82': 17, '789.03': 17, '331.83': 17, '77.85': 17, '283.11': 17, 'E870.5': 17, 'E881.1': 17, '331.19': 17, '96.38': 17, '716.99': 17, '411.81': 17, '202.83': 17, '478.19': 17, '272.8': 17, '726.33': 17, '379.50': 17, '233.0': 17, '353.0': 17, '733.49': 17, '607.89': 17, '786.51': 17, '288.4': 17, '512.89': 17, '39.76': 17, 'E950.2': 16, '079.89': 16, '96.49': 16, '214.8': 16, '807.00': 16, '35.51': 16, '152.0': 16, '936': 16, '790.99': 16, '51.51': 16, '320.2': 16, '357.81': 16, 'E860.0': 16, '42.89': 16, '766.21': 16, 'E850.1': 16, '86.4': 16, '97.88': 16, '36.02': 16, '46.10': 16, '55.24': 16, '34.6': 16, '44.66': 16, '52.96': 16, '337.3': 16, '873.1': 16, '714.30': 16, '86.73': 16, '600.91': 16, '770.1': 16, '78.2': 16, '996.09': 16, 'E815.1': 16, '52.6': 16, '34.99': 16, '79.69': 16, '48.62': 16, '242.80': 16, '295.60': 16, '88.14': 16, '806.29': 16, '151.2': 16, '730.16': 16, '802.5': 16, '77.99': 16, '92.24': 16, '345.71': 16, '153.2': 16, '967.9': 16, '59.02': 16, '746.3': 16, '337.20': 16, '46.81': 16, '755.01': 16, '377.39': 16, '587': 16, '800.26': 16, '564.7': 16, '478.33': 16, 'E931.5': 16, '402.90': 16, '883.2': 16, '278.1': 16, '777.6': 16, '031.2': 16, '52.11': 16, '710.3': 16, '46.43': 16, '730.26': 16, '255.8': 16, '291.3': 16, '921.0': 16, '711.03': 16, '21.1': 16, '854.02': 16, '958.2': 16, '438.12': 16, '483.8': 16, '96.27': 16, '753.13': 16, '727.51': 16, '31.0': 16, 'V26.51': 16, '151.9': 16, '781.99': 16, '307.51': 16, '44.38': 16, '34.02': 16, '810.01': 16, '801.36': 16, '924.01': 16, '594.1': 16, '721.2': 16, '414.9': 16, '279.8': 16, '523.40': 16, '59.0': 16, 'V85.33': 16, 'V85.31': 16, 'E821.0': 15, '305.62': 15, 'V14.1': 15, '37.4': 15, 'V02.62': 15, '97.56': 15, '724.8': 15, '770.5': 15, '760.72': 15, '51.69': 15, '768.9': 15, '776.5': 15, '38.88': 15, '478.25': 15, '802.24': 15, '815.02': 15, '886.0': 15, '296.23': 15, '569.0': 15, 'V58.1': 15, '284.9': 15, '598.8': 15, '519.3': 15, '389.8': 15, 'E935.1': 15, 'E876.8': 15, '749.00': 15, '98.03': 15, '034.0': 15, '558.1': 15, 'V16.42': 15, '307.50': 15, '51.41': 15, '861.01': 15, '800.22': 15, '531.50': 15, '69.02': 15, 'V53.39': 15, '712.36': 15, '99.79': 15, '441.6': 15, 'V01.1': 15, '088.82': 15, '157.2': 15, '824.5': 15, '851.02': 15, '344.9': 15, '97.62': 15, '779.82': 15, '719.43': 15, '359.89': 15, '722.6': 15, '38.87': 15, '648.21': 15, '153.1': 15, '353.6': 15, '715.31': 15, '77.47': 15, '305.23': 15, '331.9': 15, '863.29': 15, '852.41': 15, '442.83': 15, '881.10': 15, '39.99': 15, '239.0': 15, '737.19': 15, '111.9': 15, '88.65': 15, '813.43': 15, '703.8': 15, 'E920.8': 15, '57.17': 15, '45.12': 15, '361.9': 15, '85.89': 15, '359.1': 15, '61.1': 15, '756.6': 15, '372.73': 15, '608.89': 15, '475': 15, '82.44': 15, 'E828.2': 15, '694.5': 15, '77.45': 15, '994.7': 15, '88.66': 15, '820.20': 15, '362.30': 15, '556.8': 15, '44.41': 15, '396.1': 15, '413.1': 15, '756.83': 15, '43.42': 15, 'V54.11': 15, '354.9': 15, 'V14.5': 15, 'V04.81': 15, 'E854.0': 15, '952.08': 15, 'V17.0': 14, 'V05.8': 14, '41.95': 14, '437.5': 14, '712.30': 14, '876.0': 14, '764.97': 14, '780.71': 14, '882.2': 14, '11.': 14, '16.0': 14, '136.9': 14, '457.0': 14, '21.72': 14, 'E934.7': 14, '54.92': 14, '68.9': 14, '289.7': 14, '482.89': 14, '414.11': 14, 'V85.35': 14, '51.21': 14, '354.2': 14, '761.7': 14, '99.39': 14, 'E853.8': 14, '586': 14, '764.02': 14, '196.9': 14, '730.05': 14, 'E933.0': 14, '599.4': 14, '756.19': 14, '14.24': 14, '996.57': 14, '762.6': 14, '764.98': 14, '802.21': 14, '801.30': 14, '334.1': 14, '737.34': 14, '760.79': 14, 'V04.82': 14, '117.5': 14, '705.1': 14, '863.39': 14, '996.39': 14, '760.0': 14, '799.89': 14, '881.02': 14, '362.10': 14, '751.0': 14, '173.3': 14, '576.4': 14, '48.5': 14, '995.29': 14, '972.9': 14, '611.1': 14, '94.0': 14, '821.00': 14, '35.52': 14, '765.12': 14, '304.23': 14, '79.34': 14, '902.89': 14, '581.81': 14, '729.39': 14, '427.9': 14, '97.01': 14, '648.23': 14, '924.8': 14, '85.43': 14, '601.9': 14, '967.0': 14, 'E942.2': 14, '994.1': 14, '600.90': 14, '881.20': 14, '282.2': 14, '34.84': 14, '872.01': 14, '32.41': 14, '596.3': 14, '464.30': 14, '358.0': 14, 'V02.9': 14, 'V10.91': 14, '45.3': 14, '861.32': 14, '35.25': 14, 'V62.0': 14, '282.60': 14, '031.0': 14, '88.77': 14, '279.50': 14, 'V16.7': 14, '726.0': 14, 'V17.41': 14, '438.10': 14, '477.8': 14, '824.9': 13, '711.07': 13, '364.3': 13, 'V45.78': 13, '996.56': 13, '801.11': 13, '802.1': 13, '86.67': 13, '68.29': 13, '60.29': 13, '39.62': 13, '38.48': 13, 'V62.5': 13, '87.61': 13, '808.3': 13, '417.8': 13, '969.7': 13, '86.63': 13, '835.01': 13, '737.41': 13, '446.0': 13, '737.43': 13, '582.1': 13, '39.41': 13, 'E880.1': 13, '51.79': 13, '36.04': 13, '823.20': 13, '464.10': 13, '429.0': 13, '801.10': 13, '46.20': 13, '748.60': 13, '968.4': 13, 'V14.6': 13, 'E919.4': 13, 'E878.9': 13, 'E944.3': 13, '716.91': 13, '38.82': 13, '38.09': 13, '813.22': 13, '779.7': 13, '83.85': 13, '702.19': 13, '574.70': 13, '525.9': 13, '789.30': 13, '070.41': 13, '727.00': 13, '77.71': 13, '420.0': 13, '304.90': 13, '78.68': 13, '83.19': 13, 'V45.75': 13, 'E852.9': 13, '761.2': 13, '39.24': 13, '952.05': 13, '903.5': 13, 'V54.16': 13, '972.4': 13, '277.87': 13, '794.5': 13, 'E917.4': 13, '320.1': 13, '461.8': 13, '690.10': 13, '532.70': 13, '535.31': 13, 'E935.4': 13, '753.19': 13, '44.22': 13, 'E823.1': 13, 'E930.7': 13, 'V10.81': 13, '99.63': 13, 'V15.29': 13, '041.2': 13, '451.19': 13, '618.1': 13, '38.99': 13, '373.00': 13, '151.4': 13, 'V12.42': 13, '345.00': 13, '815.01': 13, '388.61': 13, '812.03': 13, '29.3': 13, '205.02': 13, 'E883.0': 13, '532.10': 13, '158.8': 13, '905.0': 13, '380.10': 13, '564.81': 13, '25.51': 13, '453.0': 13, '34.81': 13, 'V58.31': 13, '585.1': 13, '756.12': 13, '998.01': 13, '78.51': 12, '41.04': 12, '802.29': 12, '34.25': 12, '22.63': 12, 'E855.0': 12, '574.21': 12, '866.12': 12, '710.9': 12, '982.8': 12, '426.2': 12, '980.9': 12, '760.8': 12, '79.26': 12, '770.84': 12, 'V03.81': 12, 'E927': 12, '977.8': 12, 'V64.4': 12, '354.3': 12, '516.31': 12, '902.53': 12, '948.00': 12, '823.30': 12, '877.0': 12, '52.59': 12, '905.3': 12, '304.91': 12, '803.20': 12, '362.74': 12, '799.3': 12, '764.96': 12, '41.2': 12, '27.49': 12, '965.8': 12, '40.7': 12, '759.81': 12, '243': 12, '300.21': 12, '81.53': 12, '158.0': 12, '049.9': 12, '802.22': 12, '80.06': 12, '772.0': 12, '715.89': 12, 'V16.9': 12, 'V45.74': 12, '767.0': 12, '825.31': 12, '552.29': 12, '788.42': 12, '789.1': 12, '971.3': 12, '762.3': 12, '52.4': 12, '356.1': 12, 'V01.79': 12, '88.61': 12, '461.0': 12, '868.14': 12, '52.09': 12, '786.04': 12, '434.10': 12, '728.71': 12, '252.08': 12, '041.82': 12, '27.57': 12, '33.2': 12, '45.22': 12, '813.21': 12, '29.1': 12, '282.62': 12, '426.9': 12, '428.9': 12, '50.69': 12, '813.11': 12, '912.0': 12, '800.20': 12, '507.8': 12, '273.9': 12, '975.2': 12, '723.5': 12, '908.9': 12, '351.9': 12, '903.1': 12, '239.6': 12, '788.99': 12, '80.13': 12, 'V62.6': 12, '701.5': 12, '65.0': 12, '97.51': 12, '80.81': 12, '617.0': 12, '709.3': 12, '279.03': 12, '891.2': 12, '47.9': 12, '79.64': 12, '110.8': 12, '839.08': 12, '813.54': 12, '139.8': 12, 'V85.22': 12, '323.81': 12, '814.01': 12, '342.01': 12, 'V85.30': 12, '276.61': 12, 'V12.61': 12, '84.71': 12, 'E945.7': 12, 'E943.0': 12, '437.1': 11, '772.14': 11, '289.83': 11, '37.66': 11, '761.5': 11, '518.7': 11, '772.4': 11, 'V12.03': 11, '756.51': 11, '872.00': 11, '42.24': 11, '879.4': 11, '478.4': 11, '51.11': 11, '349.89': 11, '429.79': 11, '211.6': 11, '422.91': 11, '802.26': 11, '765.04': 11, '333.92': 11, 'E854.2': 11, '33.39': 11, '808.42': 11, '40.42': 11, '253.0': 11, '79.74': 11, '508.0': 11, '38.05': 11, '779.8': 11, '824.6': 11, '900.03': 11, '37.65': 11, '379.23': 11, '921.3': 11, '84.57': 11, '786.4': 11, '292.9': 11, '40.64': 11, '806.01': 11, '965.61': 11, '57.18': 11, '075': 11, '161.8': 11, '99.11': 11, 'V71.4': 11, '760.75': 11, '839.05': 11, '51.64': 11, '904.7': 11, '565.1': 11, '079.6': 11, '56.31': 11, '654.21': 11, '601.0': 11, '40.23': 11, '442.89': 11, '41.03': 11, '310.0': 11, '237.70': 11, '280.8': 11, '590.2': 11, '472.0': 11, '74.0': 11, '282.4': 11, '227.1': 11, '39.53': 11, '56.71': 11, '225.4': 11, 'E941.2': 11, '123.1': 11, '959.12': 11, '389.7': 11, '34.': 11, '34.59': 11, '851.41': 11, 'V10.49': 11, '710.8': 11, '69.09': 11, '851.40': 11, '253.8': 11, '659.71': 11, '952.00': 11, '821.20': 11, 'E968.8': 11, '488.1': 11, 'V26.52': 11, 'V12.52': 11, '812.31': 11, '225.1': 11, '349.81': 11, '327.21': 11, '604.90': 11, '35.72': 11, '787.99': 11, '861.31': 11, 'V65.3': 11, 'E818.1': 11, 'E917.3': 11, 'V16.6': 11, 'V58.64': 11, 'E939.1': 11, '344.03': 11, '326': 11, 'E813.2': 11, '582.89': 11, '369.8': 11, '427.2': 11, '376.01': 11, '284.12': 11, '347.00': 11, 'V85.45': 11, '969.05': 11, '795.51': 11, '32.39': 10, 'V15.07': 10, 'V65.2': 10, '532.60': 10, 'V10.53': 10, '378.52': 10, 'E980.9': 10, '722.72': 10, '733.11': 10, '31.44': 10, 'E924.8': 10, '377.41': 10, '38.21': 10, '923.00': 10, '533.40': 10, '44.62': 10, '614.4': 10, '747.42': 10, '274.82': 10, '235.2': 10, '666.32': 10, '79.03': 10, '574.40': 10, '574.30': 10, '273.4': 10, '368.13': 10, '357.4': 10, '762.1': 10, '310.9': 10, '380.4': 10, '461.3': 10, '880.03': 10, '54.64': 10, '806.26': 10, '72.0': 10, '68.8': 10, '287.30': 10, 'V32.01': 10, '250.30': 10, '506.0': 10, '596.7': 10, '802.9': 10, '533.70': 10, 'V06.1': 10, '742.9': 10, '45.34': 10, '239.7': 10, '445.02': 10, '358.9': 10, '728.0': 10, '968.0': 10, '525.10': 10, '747.21': 10, '379.43': 10, '862.39': 10, '336.8': 10, '39.23': 10, '88.03': 10, '279.51': 10, '520.6': 10, '780.96': 10, '821.29': 10, '803.22': 10, '757.32': 10, '816.11': 10, '362.34': 10, '202.40': 10, '85.2': 10, '659.61': 10, 'E944.7': 10, '706.2': 10, '277.1': 10, '154.8': 10, '141.0': 10, '766.0': 10, '255.10': 10, '161.9': 10, '823.10': 10, '863.0': 10, '372.39': 10, '55.53': 10, '304.73': 10, '041.83': 10, '830.0': 10, '770.4': 10, '79.14': 10, '823.12': 10, '902.41': 10, '874.4': 10, '81.38': 10, '218.2': 10, '748.5': 10, '438.84': 10, '68.16': 10, 'V16.41': 10, '581.1': 10, '388.69': 10, 'V54.12': 10, '51.43': 10, '540.9': 10, '47.01': 10, '851.46': 10, '027.0': 10, '531.10': 10, '241.9': 10, '237.71': 10, '778.0': 10, '57.83': 10, '244.2': 10, '79.63': 10, '596.6': 10, '903.2': 10, '44.9': 10, '729.2': 10, '562.02': 10, '212.6': 10, '743.65': 10, '885.0': 10, '84.21': 10, '83.88': 10, '447.9': 10, '777.4': 10, '816.00': 10, '788.63': 10, '424.3': 10, '721.8': 10, '863.1': 10, '800.25': 10, '825.23': 10, '38.63': 10, '464.00': 10, '729.82': 10, '238.76': 10, '926.19': 10, '957.1': 10, '691.8': 10, '12.2': 10, '517.2': 10, '76.39': 10, 'E945.2': 10, '84.0': 10, '44.63': 10, '414.05': 10, '516.0': 10, '648.94': 10, '996.41': 10, '288.09': 10, '811.01': 10, '958.93': 10, '790.8': 10, '952.03': 10, '785.1': 10, '21.21': 10, '524.60': 10, '349.2': 10, '999.33': 10, '200.30': 10, '288.66': 10, '53.72': 10, '781.6': 10, '348.82': 10, '714.2': 10, '62.3': 9, '57.92': 9, '754.70': 9, 'V54.17': 9, '904.41': 9, '86.62': 9, '969.6': 9, '806.21': 9, '38.04': 9, '440.30': 9, '345.01': 9, '884.0': 9, '950.0': 9, '252.0': 9, '764.92': 9, 'E957.1': 9, '87.76': 9, '839.01': 9, '756.10': 9, '803.25': 9, '34.73': 9, '742.1': 9, '89.38': 9, '880.02': 9, '453.3': 9, '17.55': 9, '620.8': 9, '706.1': 9, '825.35': 9, '318.0': 9, '762.2': 9, '755.63': 9, '881.22': 9, '569.42': 9, '45.41': 9, '617.9': 9, '996.03': 9, '821.33': 9, '171.3': 9, '425.3': 9, '765.21': 9, '851.01': 9, '61.': 9, '747.49': 9, '526.4': 9, 'E850.8': 9, '77.77': 9, '37.97': 9, '532.20': 9, '543.9': 9, '287.9': 9, '746.2': 9, '626.8': 9, '802.38': 9, '453.89': 9, '344.01': 9, '269.9': 9, '716.95': 9, '52.0': 9, '704.00': 9, '99.03': 9, '466.19': 9, '37.11': 9, '189.1': 9, '281.2': 9, '845.00': 9, '588.8': 9, '368.9': 9, '68.6': 9, '438.30': 9, 'V30.1': 9, '301.22': 9, '99.82': 9, '867.6': 9, '357.7': 9, '764.03': 9, 'V43.4': 9, '764.95': 9, '765.01': 9, 'E931.3': 9, '130.7': 9, '12.': 9, '808.53': 9, '807.5': 9, '70.0': 9, '802.28': 9, '250.33': 9, '22.2': 9, '86.26': 9, '705.83': 9, '355.3': 9, '338.21': 9, '535.11': 9, '040.0': 9, '464.31': 9, '783.3': 9, '094.9': 9, '133.0': 9, '483.0': 9, '722.73': 9, '98.14': 9, 'E950.1': 9, '214.3': 9, '794.15': 9, '771.4': 9, '45.33': 9, '796.4': 9, '746.7': 9, '453.87': 9, '344.09': 9, '873.59': 9, '377.75': 9, '614.3': 9, '814.00': 9, '279.4': 9, 'V15.09': 9, '777.1': 9, '747.61': 9, '552.00': 9, '754.81': 9, '78.07': 9, '995.27': 9, '464.50': 9, '171.5': 9, 'E823.2': 9, '69.52': 9, '305.41': 9, '523.9': 9, '953.4': 9, '493.02': 9, '863.99': 9, '815.03': 9, '415.0': 9, '97.61': 9, '82.36': 9, '251.5': 9, '76.31': 9, '614.0': 9, 'V18.3': 9, '590.00': 9, '312.9': 9, '831.01': 9, '438.50': 9, '12.6': 9, '806.06': 9, '235.7': 9, '794.4': 9, '191.4': 9, '40.4': 9, '453.75': 9, '864.14': 9, '299.00': 9, '369.9': 9, 'E928.3': 9, '282.40': 9, '854.06': 9, '388.30': 9, '701.1': 9, '800.01': 9, '38.02': 9, '195.0': 9, '81.47': 9, '323.82': 9, '77.7': 9, '958.91': 9, '746.83': 9, '478.22': 9, 'V85.23': 9, '50.14': 9, '425.11': 9, '444.09': 9, 'E001.0': 9, 'E000.9': 9, '793.89': 9, '999.82': 9, '46.86': 9, '78.61': 8, '780.8': 8, '767.8': 8, '788.43': 8, '720.2': 8, '851.42': 8, '82.45': 8, '369.3': 8, '250.93': 8, '793.5': 8, '30.3': 8, '990': 8, '567.82': 8, '478.1': 8, '854.05': 8, '808.49': 8, '410.82': 8, '88.71': 8, '701.9': 8, '99.41': 8, '529.0': 8, '681.00': 8, '674.54': 8, '453.52': 8, '204.11': 8, '40.24': 8, '38.67': 8, '550.12': 8, '51.04': 8, '867.8': 8, '971.1': 8, '764.94': 8, '156.9': 8, 'E852.8': 8, 'E919.8': 8, '84.22': 8, '575.5': 8, '292.84': 8, 'E855.1': 8, '720.9': 8, '63.': 8, '62.5': 8, '383.00': 8, '303.02': 8, '868.02': 8, '99.99': 8, '77.31': 8, '286.5': 8, '745.69': 8, '198.81': 8, '252.02': 8, '863.21': 8, '223.0': 8, '96.36': 8, '70.77': 8, '821.30': 8, '726.11': 8, '956.3': 8, '86.23': 8, 'V55.5': 8, '79.17': 8, '892.0': 8, 'E922.9': 8, 'E958.8': 8, '715.34': 8, '527.5': 8, '80.87': 8, '31.93': 8, '647.83': 8, '666.24': 8, '150.3': 8, '88.79': 8, '493.21': 8, '812.40': 8, 'E989': 8, '173.7': 8, '79.76': 8, '772.8': 8, '201.98': 8, '49.45': 8, '78.49': 8, '85.21': 8, '79.12': 8, '451.83': 8, '903.3': 8, '38.83': 8, '972.0': 8, '26.32': 8, '172.5': 8, '813.33': 8, '45.11': 8, '81.81': 8, '174.4': 8, '97.12': 8, '719.65': 8, '414.19': 8, '864.12': 8, '879.2': 8, '60.2': 8, '517.3': 8, '719.49': 8, '55.02': 8, '751.7': 8, '360.00': 8, '747.29': 8, '839.42': 8, '57.89': 8, '85.42': 8, '747.69': 8, '890.1': 8, '646.63': 8, '756.89': 8, '376.30': 8, 'V06.6': 8, '757.39': 8, '404.03': 8, '729.4': 8, '596.9': 8, '300.14': 8, '865.13': 8, '344.40': 8, '164.0': 8, '53.41': 8, '955.3': 8, '442.0': 8, '721.7': 8, 'E888.0': 8, 'E917.0': 8, '421.9': 8, '300.22': 8, '716.80': 8, '714.9': 8, '922.31': 8, '775.8': 8, '583.1': 8, '78.47': 8, '996.78': 8, '801.15': 8, '438.6': 8, '535.00': 8, '596.59': 8, '625.8': 8, '621.0': 8, '541': 8, '343.2': 8, '38.89': 8, '327.27': 8, '736.89': 8, '344.81': 8, '191.0': 8, '728.2': 8, '617.1': 8, '575.6': 8, '902.22': 8, '83.63': 8, '738.5': 8, '75.69': 8, '790.2': 8, '523.8': 8, '011.90': 8, '816.02': 8, '811.02': 8, 'E819.9': 8, '83.13': 8, '57.6': 8, '171.4': 8, '56.74': 8, '336.0': 8, '354.8': 8, '813.23': 8, '78.09': 8, '348.2': 8, '780.50': 8, '22.11': 8, '682.9': 8, '648.44': 8, '88.38': 8, '453.77': 8, 'V10.61': 8, '153.7': 8, '433.80': 8, '803.15': 8, '301.81': 8, 'V15.08': 8, '951.4': 8, '76.43': 8, 'E955.0': 8, '473.2': 8, '78.63': 8, '707.00': 8, '96.55': 8, '320.7': 8, '730.06': 8, '803.26': 8, 'V15.89': 8, '868.09': 8, '27.56': 8, '375.15': 8, '491.8': 8, '79.04': 8, '365.10': 8, '701.0': 8, '37.98': 8, '806.08': 8, '202.82': 8, '180.9': 8, '279.53': 8, '96.57': 8, 'V86.0': 8, '200.70': 8, '879.8': 8, '338.28': 8, '789.39': 8, '480.2': 8, '204.12': 8, '780.33': 8, '969.72': 8, '209.72': 8, '274.03': 8, '81.37': 8, '200.50': 8, '997.32': 8, 'V02.53': 8, '94.25': 7, '861.13': 7, '752.63': 7, '191.5': 7, '871.1': 7, '203.01': 7, '78.05': 7, '053.79': 7, '358.1': 7, '569.2': 7, '117.7': 7, '279.49': 7, '286.2': 7, '200.28': 7, '574.60': 7, '44.1': 7, 'V02.51': 7, '039.1': 7, '93.': 7, '824.1': 7, '188.2': 7, '51.83': 7, '157.1': 7, '34.93': 7, '33.93': 7, '235.5': 7, 'E918': 7, '80.11': 7, '726.91': 7, '952.04': 7, '25.1': 7, '29.5': 7, '804.25': 7, '820.03': 7, '25.2': 7, '771.82': 7, '864.00': 7, '756.3': 7, '801.31': 7, '34.72': 7, '304.60': 7, '466.11': 7, '37.9': 7, '51.63': 7, '796.1': 7, '77.48': 7, '30.4': 7, '83.49': 7, '301.4': 7, '318.2': 7, '365.63': 7, '55.01': 7, 'E934.6': 7, '751.3': 7, '773.0': 7, '40.21': 7, '513.1': 7, '49.01': 7, '75.34': 7, '708.9': 7, '786.07': 7, '867.2': 7, 'V06.8': 7, '473.3': 7, '724.9': 7, '42.87': 7, '648.43': 7, '803.12': 7, '721.42': 7, '54.99': 7, '054.19': 7, '959.09': 7, 'E940.1': 7, '73.4': 7, '73.09': 7, '272.6': 7, '823.21': 7, '38.68': 7, '722.90': 7, '56.41': 7, '55.0': 7, '571.9': 7, 'V59.6': 7, '079.51': 7, '148.1': 7, '50.51': 7, '746.86': 7, '782.5': 7, '337.22': 7, '569.86': 7, '955.2': 7, '470': 7, '38.34': 7, '83.42': 7, '294.0': 7, '71.0': 7, '770.12': 7, '564.2': 7, '268.2': 7, 'V88.12': 7, '39.66': 7, '53.01': 7, '711.09': 7, '844.9': 7, '344.04': 7, '344.30': 7, '783.1': 7, '615.9': 7, '729.71': 7, '959.7': 7, '79.16': 7, '34.71': 7, '309.4': 7, '964.2': 7, '153.0': 7, '85.41': 7, 'V10.84': 7, '862.22': 7, '812.50': 7, '480.8': 7, '296.32': 7, '923.03': 7, '701.8': 7, 'V16.49': 7, '51.32': 7, '836.1': 7, '80.83': 7, '534.00': 7, '478.79': 7, '333.72': 7, 'V56.0': 7, '259.4': 7, '863.30': 7, '755.50': 7, '794.9': 7, '141.8': 7, '65.39': 7, '882.1': 7, '722.11': 7, '727.61': 7, '443.23': 7, '808.9': 7, '601.1': 7, 'E813.1': 7, '709.09': 7, '996.77': 7, '078.11': 7, '844.1': 7, '377.30': 7, '278.02': 7, '85.34': 7, '737.12': 7, '719.16': 7, '49.39': 7, '458.1': 7, '620.1': 7, '76.5': 7, '378.87': 7, '81.33': 7, '17.42': 7, '052.9': 7, '383.9': 7, '528.2': 7, '235.3': 7, '686.01': 7, '646.64': 7, '173.4': 7, '444.9': 7, '30.09': 7, '482.81': 7, '341.8': 7, '205.11': 7, '24.5': 7, '78.12': 7, '345.81': 7, '81.13': 7, '493.81': 7, '601.2': 7, '88.97': 7, '377.49': 7, '97.89': 7, '51.96': 7, 'V43.0': 7, '378.9': 7, '302.50': 7, '341.0': 7, 'V43.1': 7, '377.01': 7, '80.88': 7, '788.38': 7, '969.1': 7, '615.0': 7, '537.1': 7, '333.6': 7, '12.8': 7, '521.81': 7, '17.36': 7, 'V85.25': 7, '627.9': 7, '530.13': 7, '415.13': 7, '596.89': 7, '995.61': 7, '52.12': 6, '800.12': 6, '194.0': 6, '464.51': 6, '82.29': 6, '170.0': 6, '568.82': 6, '89.60': 6, '775.81': 6, '764.93': 6, '874.02': 6, '14.74': 6, '753.15': 6, '753.22': 6, '733.15': 6, '796.0': 6, '975.4': 6, '719.02': 6, '607.82': 6, '437.7': 6, '410.02': 6, '295.92': 6, '21.09': 6, '879.6': 6, 'E818.7': 6, '38.65': 6, '905.1': 6, '873.52': 6, '189.2': 6, '96.23': 6, '286.1': 6, '192.1': 6, '921.1': 6, '53.02': 6, '260': 6, '762.4': 6, '321.0': 6, '755.59': 6, '742.59': 6, '58.39': 6, '56.82': 6, '823.81': 6, '804.26': 6, '386.30': 6, '22.64': 6, '608.9': 6, '453.72': 6, '789.05': 6, '386.12': 6, '482.32': 6, '532.41': 6, '900.82': 6, '272.2': 6, '391.1': 6, '46.76': 6, '81.32': 6, '63.9': 6, '51.94': 6, '892.1': 6, '34.27': 6, '289.3': 6, '674.32': 6, '572.1': 6, '834.02': 6, '216.5': 6, '180.8': 6, '618.01': 6, '825.1': 6, '642.44': 6, '32.0': 6, '301.20': 6, '625.3': 6, 'E906.8': 6, '370.34': 6, '202.60': 6, '813.83': 6, '733.41': 6, '927.20': 6, '694.4': 6, '88.63': 6, 'E855.6': 6, '69.51': 6, '239.5': 6, '754.89': 6, '670.04': 6, '666.34': 6, '46.72': 6, '007.4': 6, '379.91': 6, '31.64': 6, '473.1': 6, '780.94': 6, '99.78': 6, '362.11': 6, 'E927.8': 6, '79.11': 6, '756.14': 6, '333.82': 6, '053.0': 6, '745.10': 6, '236.91': 6, '362.31': 6, '80.6': 6, '702.0': 6, '648.61': 6, '647.81': 6, '750.4': 6, '850.9': 6, '863.45': 6, 'V45.09': 6, '767.1': 6, '53.00': 6, '305.52': 6, '906.4': 6, 'E929.1': 6, '127.2': 6, '161.1': 6, '550.11': 6, '51.59': 6, '149.0': 6, '39.55': 6, 'E910.8': 6, '289.8': 6, '813.82': 6, '821.31': 6, '242.91': 6, '36.31': 6, '269.8': 6, '55.04': 6, '758.7': 6, '996.43': 6, 'V10.62': 6, '84.18': 6, '79.13': 6, '376.33': 6, '574.11': 6, '295.20': 6, '451.0': 6, '287.1': 6, '751.4': 6, '595.1': 6, '200.10': 6, '078.10': 6, '28.2': 6, '780.59': 6, '233.7': 6, '53.10': 6, '822.1': 6, '372.03': 6, '204.01': 6, '44.99': 6, '161.0': 6, '604.0': 6, '695.2': 6, '83.94': 6, '239.1': 6, '156.8': 6, '96.01': 6, '440.32': 6, '727.05': 6, '735.4': 6, '218.0': 6, '37.62': 6, '46.82': 6, 'E930.6': 6, 'E920.1': 6, 'V10.88': 6, 'E950.7': 6, 'E944.5': 6, 'E919.2': 6, '903.8': 6, '711.02': 6, 'V54.81': 6, '620.5': 6, '801.24': 6, '346.20': 6, '864.13': 6, '80.21': 6, '395.2': 6, '746.09': 6, '81.65': 6, '560.32': 6, 'E938.5': 6, '723.8': 6, '904.53': 6, '208.90': 6, '955.7': 6, '825.22': 6, '495.9': 6, '363.20': 6, '51.': 6, '728.9': 6, '45.15': 6, 'V69.4': 6, '85.85': 6, '526.89': 6, '275.1': 6, '873.74': 6, '77.87': 6, '512.2': 6, '747.40': 6, '802.25': 6, '277.89': 6, '45.51': 6, '772.2': 6, '727.03': 6, '359.2': 6, '158.9': 6, '38.47': 6, '432.0': 6, '80.36': 6, '115.99': 6, '802.39': 6, '558.2': 6, '79.72': 6, 'E854.8': 6, '441.9': 6, '333.90': 6, '916.2': 6, '852.42': 6, '453.76': 6, '674.84': 6, '173.5': 6, '785.9': 6, '17.33': 6, '359.71': 6, '163.9': 6, '289.50': 6, '308.0': 6, '76.73': 6, '502': 6, '71.3': 6, '270.6': 6, '872.8': 6, '880.00': 6, '53.29': 6, '616.4': 6, 'V15.04': 6, '955.1': 6, '008.63': 6, '996.89': 6, 'V13.89': 6, '094.0': 6, '425.18': 6, '371.40': 6, '532.30': 6, '574.80': 6, '35.62': 6, '813.32': 6, '730.12': 6, '881.11': 6, '50.24': 6, '12.7': 6, '378.10': 6, '171.0': 6, '451.9': 6, '607.2': 6, '820.02': 6, '275.5': 6, '722.2': 6, '980.2': 6, '389.15': 6, 'V60.2': 6, '839.03': 6, '203.02': 6, '783.6': 6, '738.3': 6, 'V85.24': 6, '362.03': 6, '355.1': 6, '315.8': 6, '150.1': 6, '716.94': 6, '48.52': 6, '85.74': 6, '453.50': 6, '573.5': 6, '275.01': 6, 'V15.05': 6, '730.24': 6, '69.0': 6, '282.46': 6, '191.7': 5, '745.61': 5, 'V45.69': 5, '360.01': 5, '379.00': 5, '721.41': 5, '39.54': 5, '76.91': 5, '99.00': 5, 'E821.7': 5, '56.52': 5, 'E854.1': 5, '966.1': 5, '970.1': 5, 'E910.2': 5, '323.8': 5, '823.80': 5, '81.46': 5, '771.89': 5, '036.0': 5, 'V06.3': 5, '780.51': 5, '405.01': 5, '958.1': 5, '152.2': 5, '805.00': 5, '618.0': 5, '304.41': 5, '958.3': 5, '052.1': 5, '009.2': 5, '32.6': 5, '839.02': 5, '237.3': 5, '52.9': 5, '71.71': 5, '270.0': 5, '758.6': 5, '625.9': 5, '292.85': 5, '375.55': 5, '804.22': 5, '22.79': 5, '31.75': 5, '242.01': 5, '745.8': 5, '728.84': 5, '77.64': 5, '82.56': 5, '253.1': 5, '793.4': 5, '25.01': 5, '437.4': 5, 'V29.1': 5, '97.39': 5, '755.13': 5, '79.87': 5, '87.62': 5, '507.1': 5, '68.59': 5, '046.3': 5, '16.82': 5, '575.3': 5, '718.46': 5, '648.63': 5, '804.21': 5, '70.4': 5, '632': 5, '422.90': 5, '917.2': 5, '293.1': 5, '77.39': 5, '310.1': 5, '283.2': 5, 'V13.8': 5, '357.89': 5, '950.9': 5, '556.3': 5, '53.11': 5, '482.31': 5, '665.51': 5, '746.1': 5, '784.41': 5, '873.65': 5, '902.21': 5, '684': 5, '308.2': 5, '599.6': 5, '648.04': 5, '647.84': 5, '304.61': 5, '775.4': 5, '45.03': 5, '201.50': 5, '801.32': 5, 'V19.5': 5, '86.61': 5, '755.02': 5, '607.83': 5, 'V58.41': 5, '18.79': 5, '77.91': 5, '89.39': 5, '38.43': 5, '162.0': 5, 'E825.7': 5, '642.51': 5, 'V27.2': 5, '290.43': 5, '80.39': 5, '070.31': 5, '238.2': 5, '250.32': 5, '405.99': 5, '88.5': 5, '305.22': 5, '715.09': 5, '525.50': 5, '277.9': 5, 'V15.02': 5, '704.1': 5, '767.2': 5, '763.82': 5, '448.9': 5, '733.16': 5, '836.0': 5, '648.81': 5, '142.0': 5, '995.3': 5, '729.6': 5, '76.78': 5, '813.52': 5, '872.02': 5, '304.02': 5, '362.89': 5, '815.09': 5, '904.2': 5, '754.0': 5, '097.1': 5, '87.69': 5, '69.5': 5, '992.0': 5, '741.93': 5, '48.75': 5, '793.0': 5, '208.00': 5, '184.4': 5, '40.54': 5, '395.1': 5, '648.13': 5, 'V50.41': 5, '89.69': 5, '478.34': 5, '370.00': 5, 'V04.0': 5, '45.71': 5, '692.4': 5, 'E858.1': 5, '82.12': 5, '48.32': 5, '708.8': 5, '878.2': 5, '61.41': 5, '584.6': 5, '322.0': 5, '89.44': 5, '337.29': 5, '802.7': 5, 'E817.1': 5, '68.3': 5, '89.48': 5, 'V74.1': 5, '39.58': 5, '87.44': 5, '130.0': 5, '31.71': 5, '815.19': 5, 'V58.49': 5, '294.21': 5, '761.3': 5, '669.32': 5, '53.04': 5, '44.49': 5, '40.1': 5, '763.1': 5, '746.87': 5, '342.10': 5, '880.13': 5, '170.7': 5, '60.11': 5, '53.21': 5, '713.1': 5, '42.7': 5, 'E938.2': 5, '719.03': 5, 'E885.4': 5, 'V19.8': 5, '199.0': 5, '901.1': 5, '70.24': 5, 'E922.0': 5, '389.00': 5, '288.61': 5, '893.0': 5, '40.59': 5, '442.82': 5, '36.91': 5, '724.03': 5, '582.2': 5, '146.0': 5, '154.3': 5, '529.8': 5, 'V45.87': 5, '556.2': 5, '903.01': 5, '71.09': 5, '44.14': 5, '77.62': 5, '464.11': 5, '758.9': 5, '87.73': 5, '722.51': 5, '81.36': 5, '429.1': 5, '048': 5, '45.31': 5, '426.51': 5, '56.75': 5, '22.62': 5, '77.66': 5, '683': 5, '715.97': 5, '32.1': 5, '65.29': 5, '712.33': 5, '794.02': 5, '078.19': 5, '627.3': 5, '454.0': 5, '851.05': 5, '603.8': 5, 'V61.8': 5, 'E937.0': 5, '53.81': 5, 'E917.5': 5, '84.07': 5, '27.52': 5, '805.8': 5, '83.5': 5, '36.2': 5, '287.0': 5, '33.34': 5, '802.36': 5, '733.81': 5, '203.80': 5, '573.1': 5, '863.40': 5, '995.83': 5, '147.8': 5, '373.13': 5, '57.87': 5, '388.70': 5, '839.20': 5, 'V16.2': 5, '900.1': 5, '801.76': 5, 'E917.7': 5, '801.65': 5, '78.03': 5, '57.33': 5, '446.7': 5, 'E935.7': 5, '800.10': 5, '44.12': 5, '832.00': 5, '453.1': 5, '192.2': 5, '991.1': 5, '813.03': 5, 'E858.8': 5, '172.3': 5, '79.85': 5, '839.06': 5, '81.11': 5, '448.0': 5, '622.10': 5, '921.9': 5, '951.0': 5, '758.5': 5, '813.02': 5, '697.0': 5, '842.00': 5, '972.1': 5, '790.22': 5, '22.52': 5, '304.81': 5, 'E901.9': 5, '759.5': 5, '99.57': 5, '806.03': 5, '759.3': 5, '852.31': 5, '49.42': 5, '44.95': 5, '053.29': 5, '873.73': 5, '753.17': 5, '793.7': 5, 'E860.3': 5, 'E938.7': 5, '288.62': 5, '434.00': 5, 'E885.2': 5, '200.40': 5, '789.02': 5, '41.06': 5, '76.99': 5, '80.12': 5, '648.92': 5, '112.81': 5, '304.93': 5, '933.0': 5, '716.97': 5, '323.61': 5, 'V10.60': 5, 'V13.65': 5, '359.21': 5, '597.0': 5, '334.3': 5, '147.9': 5, '530.87': 5, '447.71': 5, '209.21': 5, '249.01': 5, '788.64': 5, '368.15': 5, '518.53': 5, '35.99': 5, '998.00': 5, 'E906.4': 5, '998.02': 5, '998.33': 5, '207.80': 5, '863.84': 5, '523.10': 5, 'E967.0': 5, '453.71': 5, '955.6': 5, '64.49': 5, '727.09': 5, '626.0': 5, 'E960.1': 5, '246.8': 5, '715.80': 5, '790.21': 5, 'E946.0': 5, '746.6': 4, '404.92': 4, '376.32': 4, '85.9': 4, '394.9': 4, '500': 4, '164.2': 4, '750.9': 4, '852.46': 4, '379.92': 4, '758.89': 4, '36.03': 4, '904.1': 4, '89.63': 4, '130.9': 4, '81.45': 4, '730.22': 4, '303.03': 4, '780.9': 4, '288.01': 4, '922.8': 4, '868.19': 4, '806.07': 4, 'V15.5': 4, '009.3': 4, '036.2': 4, '759.9': 4, '273.0': 4, '40.2': 4, '595.89': 4, '70.50': 4, 'E924.1': 4, '30.29': 4, '786.02': 4, 'E939.7': 4, 'V01.7': 4, '869.0': 4, '599.1': 4, '867.1': 4, '58.22': 4, '83.31': 4, '802.27': 4, '292.11': 4, '887.2': 4, '730.04': 4, '226': 4, '333.91': 4, '784.1': 4, '755.67': 4, '562.01': 4, '164.8': 4, '726.2': 4, 'E819.7': 4, '715.94': 4, '88.93': 4, '442.1': 4, 'V53.91': 4, '939.2': 4, '290.12': 4, '141.9': 4, '753.8': 4, '427.60': 4, '82.09': 4, '28.11': 4, '845.03': 4, '42.58': 4, '923.20': 4, '402.01': 4, '52.80': 4, '59.09': 4, '823.90': 4, '996.45': 4, '45.92': 4, '673.23': 4, 'E958.5': 4, '71.5': 4, '59.79': 4, '534.50': 4, '65.63': 4, '821.10': 4, '902.54': 4, '295.74': 4, '801.72': 4, '35.93': 4, '864.09': 4, '38.84': 4, '719.07': 4, '137.0': 4, '794.2': 4, '53.05': 4, '77.41': 4, '863.50': 4, '49.95': 4, '764.09': 4, 'V13.09': 4, '863.81': 4, '593.82': 4, '40.22': 4, '77.19': 4, '86.06': 4, '16.': 4, '708.3': 4, '20.49': 4, '927.3': 4, '747.5': 4, '615.1': 4, '763.5': 4, '435.3': 4, '663.31': 4, '659.51': 4, '800.31': 4, '250.31': 4, '34.26': 4, '027.2': 4, '22.41': 4, '868.12': 4, '680.2': 4, '34.23': 4, '355.2': 4, '863.52': 4, '758.2': 4, '237.6': 4, '734': 4, '83.75': 4, '93.91': 4, '728.13': 4, '526.9': 4, '293.89': 4, '21.88': 4, 'E825.1': 4, '792.0': 4, '879.7': 4, '98.51': 4, '908.1': 4, '850.0': 4, '259.9': 4, '85.0': 4, '018.94': 4, '48.49': 4, '85.94': 4, 'E943.8': 4, 'E942.5': 4, '39.91': 4, '730.88': 4, '10.1': 4, '762.8': 4, '700': 4, '757.0': 4, '977.9': 4, '87.72': 4, '56.2': 4, '79.29': 4, '808.1': 4, 'E958.9': 4, '42.55': 4, '35.53': 4, '996.84': 4, '852.29': 4, '55.54': 4, '753.9': 4, '927.10': 4, '31.73': 4, '31.98': 4, '902.9': 4, '595.2': 4, '694.8': 4, '602.3': 4, 'E917.8': 4, '38.66': 4, '756.4': 4, '39.4': 4, '756.17': 4, '404.01': 4, '701.2': 4, '887.0': 4, '214.1': 4, '85.6': 4, '031.9': 4, '42.32': 4, '812.41': 4, '277.6': 4, 'E930.3': 4, '844.2': 4, '88.75': 4, '59.93': 4, '806.24': 4, '305.80': 4, '42.54': 4, '747.41': 4, 'E850.3': 4, '619.0': 4, '779.2': 4, '296.99': 4, '48.79': 4, '274.11': 4, '212.3': 4, '863.20': 4, '801.35': 4, '48.9': 4, '800.76': 4, '535.70': 4, '379.21': 4, '478.21': 4, '952.9': 4, '744.1': 4, '53.03': 4, '524.00': 4, '250.21': 4, '344.2': 4, '727.04': 4, '13.2': 4, 'V42.1': 4, 'V54.9': 4, 'E936.4': 4, '61.49': 4, '758.81': 4, '692.82': 4, '719.01': 4, 'E814.1': 4, '084.0': 4, '616.0': 4, '808.51': 4, '80.98': 4, '78.1': 4, '528.5': 4, '256.1': 4, '304.30': 4, '763.89': 4, '89.26': 4, '741.00': 4, '847.1': 4, '914.0': 4, '618.5': 4, '24.4': 4, 'E824.1': 4, '617.3': 4, '746.01': 4, 'E970': 4, '37.86': 4, '722.83': 4, '20.09': 4, '664.01': 4, '611.8': 4, 'V49.60': 4, '793.2': 4, '58.5': 4, '426.50': 4, '762.7': 4, '171.6': 4, '814.03': 4, '996.53': 4, '332.1': 4, '800.00': 4, '718.49': 4, '556.4': 4, '84.94': 4, '35.91': 4, '215.5': 4, '082.40': 4, '81.66': 4, '120.9': 4, '92.3': 4, '308.3': 4, 'E901.8': 4, '92.39': 4, '758.1': 4, '767.6': 4, '172.4': 4, '746.81': 4, '81.12': 4, '718.87': 4, '35.82': 4, '648.03': 4, '53.62': 4, 'V44.50': 4, 'nan': 4, '422.93': 4, '939.0': 4, '85.36': 4, '82.46': 4, '623.5': 4, 'E934.5': 4, '377.00': 4, '233.9': 4, '377.10': 4, '377.16': 4, '692.6': 4, '816.12': 4, '427.42': 4, '309.89': 4, '052.7': 4, '008.5': 4, '824.3': 4, '333.7': 4, '649.03': 4, '754.32': 4, '775.89': 4, '875.1': 4, '118': 4, '307.89': 4, '79.08': 4, '44.03': 4, '367.1': 4, '22.6': 4, '989.89': 4, '87.78': 4, '65.91': 4, '787.1': 4, '341.20': 4, '851.00': 4, 'E885.3': 4, '607.9': 4, 'E959': 4, '707.9': 4, '648.64': 4, '144.0': 4, '471.8': 4, 'V54.01': 4, '086.0': 4, '759.2': 4, '83.79': 4, '379.24': 4, '238.6': 4, '812.42': 4, '726.60': 4, '742.0': 4, '79.38': 4, '516.36': 4, '553.9': 4, '759.0': 4, '92.05': 4, '296.44': 4, '85.91': 4, 'V17.5': 4, '519.9': 4, '674.04': 4, '879.5': 4, '246.2': 4, '556.0': 4, '880.01': 4, 'V07.4': 4, '23.01': 4, '703.0': 4, '45.01': 4, '041.05': 4, '648.14': 4, 'E884.1': 4, '995.81': 4, '872.61': 4, '172.6': 4, '789.33': 4, '88.96': 4, 'E825.0': 4, '38.62': 4, '706.8': 4, '300.7': 4, '357.3': 4, '958.5': 4, '378.81': 4, '642.54': 4, '804.20': 4, '152.8': 4, '334.0': 4, '996.88': 4, '99.88': 4, '881.21': 4, '652.21': 4, '265.2': 4, '85.84': 4, 'V15.59': 4, '714.89': 4, '041.03': 4, '924.9': 4, 'V72.81': 4, '80.17': 4, '719.40': 4, '255.42': 4, '343.1': 4, '844.8': 4, 'V42.89': 4, '627.2': 4, '21.0': 4, '446.21': 4, '852.24': 4, '31.3': 4, '780.64': 4, '422.0': 4, '863.42': 4, '599.84': 4, '376.89': 4, '718.88': 4, '681.11': 4, 'E858.2': 4, '211.8': 4, '79.73': 4, '153.5': 4, '77.63': 4, '322.2': 4, 'E818.2': 4, 'E812.3': 4, '81.28': 4, '201.58': 4, '27.92': 4, '730.01': 4, '996.47': 4, '388.8': 4, '038.41': 4, 'E825.2': 4, 'E871.6': 4, '235.4': 4, '96.7': 4, '258.01': 4, '902.23': 4, '692.3': 4, 'E860.9': 4, '686.1': 4, '111.0': 4, '952.8': 4, '77.0': 4, '249.81': 4, '42.4': 4, '200.23': 4, '727.3': 4, '239.2': 4, 'V61.42': 4, '821.39': 4, '918.0': 4, '942.04': 4, '542': 4, '924.20': 4, '202.70': 4, '787.24': 4, 'E906.0': 4, '564.3': 4, '362.06': 4, '780.02': 4, '784.92': 4, 'E871.0': 4, '339.00': 4, '209.29': 4, 'V49.85': 4, '854.01': 4, '202.78': 4, '44.01': 4, 'V61.41': 4, '381.4': 4, '26.31': 4, '282.3': 4, '379.99': 4, '334.2': 4, '715.00': 4, 'V10.90': 4, 'V85.21': 4, 'V87.2': 4, '195.8': 4, '35.55': 4, '374.32': 4, '259.2': 4, '736.81': 4, '76.1': 4, '714.81': 4, '37.73': 4, '539.89': 4, '98.2': 3, '969.5': 3, '270.7': 3, '622.11': 3, '957.0': 3, '51.62': 3, '191.6': 3, '83.61': 3, '957.9': 3, '93.42': 3, '057.9': 3, '157.3': 3, '999.1': 3, 'E931.8': 3, '293.83': 3, '575.9': 3, '99.22': 3, 'E866.8': 3, '77.42': 3, '924.21': 3, '718.47': 3, '793.6': 3, '597.89': 3, '719.05': 3, '70.71': 3, 'E858.6': 3, '75.0': 3, '60.94': 3, '812.44': 3, '96.26': 3, '39.30': 3, 'E829.8': 3, '753.4': 3, '200.02': 3, 'E958.0': 3, 'V49.81': 3, '804.35': 3, '035': 3, '27.24': 3, '754.79': 3, '047.8': 3, '049.8': 3, '952.09': 3, '744.29': 3, '445.81': 3, '604.99': 3, '89.22': 3, '272.9': 3, '914.2': 3, '77.11': 3, '550.91': 3, '989.4': 3, 'E950.6': 3, '996.93': 3, 'E919.0': 3, '861.00': 3, '922.4': 3, '810.10': 3, '564.01': 3, '245.9': 3, '81.17': 3, '987.8': 3, 'E869.8': 3, '556.1': 3, '361.89': 3, '753.5': 3, '813.18': 3, '870.2': 3, '727.81': 3, '776.2': 3, '923.10': 3, '68.41': 3, '94.49': 3, '775.9': 3, '825.29': 3, '901.3': 3, '928.20': 3, '85.44': 3, '75.52': 3, '57.19': 3, '323.41': 3, '65.1': 3, '31.48': 3, '801.82': 3, '31.69': 3, '173.2': 3, 'E822.7': 3, '524.10': 3, '80.26': 3, '66.32': 3, '753.23': 3, 'V15.2': 3, '771.2': 3, '212.1': 3, '295.80': 3, '301.51': 3, '852.40': 3, '10.0': 3, '55.92': 3, '304.83': 3, '82.41': 3, '86.86': 3, '471.0': 3, '44.15': 3, '40.9': 3, '296.00': 3, '748.0': 3, '97.71': 3, '652.61': 3, '656.51': 3, '75.61': 3, '942.14': 3, '836.50': 3, '873.53': 3, '11.51': 3, '38.38': 3, '440.9': 3, '873.61': 3, 'V01.89': 3, 'E818.0': 3, '782.61': 3, '78.66': 3, '477.0': 3, '161.2': 3, '782.2': 3, '642.24': 3, '277.00': 3, '698.1': 3, '693.1': 3, '923.11': 3, '77.6': 3, '781.1': 3, 'E932.8': 3, '646.83': 3, '800.61': 3, '96.52': 3, '85.96': 3, 'E987.1': 3, '938': 3, '647.61': 3, 'E891.8': 3, '82.21': 3, '686.09': 3, '49.46': 3, 'V31.1': 3, '641.31': 3, '771.1': 3, '27.42': 3, '634.91': 3, '463': 3, '814.08': 3, '35.28': 3, 'E930.4': 3, '765.05': 3, '840.9': 3, '758.39': 3, '522.6': 3, '48.71': 3, '745.60': 3, '788.8': 3, '297.9': 3, '571.40': 3, 'E812.9': 3, '52.': 3, '455.9': 3, '35.31': 3, '209.20': 3, '249.11': 3, '44.93': 3, 'E980.5': 3, '254.8': 3, '968.5': 3, '696.2': 3, '42.11': 3, '369.70': 3, '338.0': 3, '851.89': 3, '881.12': 3, '38.33': 3, '86.94': 3, '96.18': 3, '053.13': 3, '641.21': 3, '35.7': 3, 'E884.5': 3, '49.04': 3, '959.9': 3, 'V63.8': 3, '746.84': 3, 'V10.29': 3, '054.72': 3, '004.1': 3, '702.8': 3, '238.8': 3, '230.1': 3, '764.91': 3, '22.01': 3, '666.02': 3, '815.10': 3, '718.44': 3, 'E920.9': 3, 'E814.6': 3, '817.0': 3, '77.73': 3, '55.86': 3, '89.32': 3, '39.93': 3, '753.21': 3, '788.91': 3, '378.00': 3, '674.03': 3, '23.1': 3, '952.14': 3, '718.31': 3, '803.75': 3, 'E929.2': 3, '228.1': 3, '712.38': 3, '29.2': 3, 'V02.3': 3, '780.66': 3, '258.9': 3, '999.5': 3, 'V53.09': 3, '732.1': 3, '985.8': 3, '983.9': 3, '989.9': 3, '349.9': 3, '537.81': 3, '783.41': 3, '748.1': 3, 'E812.6': 3, '752.49': 3, '50.19': 3, '184.0': 3, '635.72': 3, '635.22': 3, '635.12': 3, '55.81': 3, 'E965.1': 3, 'V10.69': 3, '454.1': 3, '474.11': 3, '28.7': 3, '806.60': 3, 'V13.69': 3, 'V18.2': 3, '733.3': 3, '96.02': 3, '455.1': 3, '238.0': 3, '850.4': 3, '88.9': 3, '748.69': 3, '826.1': 3, '88.92': 3, '365.65': 3, '438.40': 3, '82.01': 3, '484.8': 3, '802.23': 3, '874.9': 3, '52.8': 3, '306.1': 3, '495.8': 3, '57.93': 3, '478.20': 3, '121.1': 3, '78.16': 3, '614.1': 3, '70.5': 3, '852.11': 3, '69.01': 3, '904.0': 3, '552.9': 3, '581.2': 3, '878.0': 3, '62.69': 3, '533.00': 3, '774.39': 3, '417.1': 3, '256.39': 3, '73.59': 3, '97.85': 3, '564.0': 3, 'E938.9': 3, 'V49.65': 3, '601.8': 3, '389.12': 3, '900.01': 3, '866.03': 3, '85.31': 3, '187.4': 3, '58.0': 3, '304.22': 3, '80.75': 3, '38.81': 3, 'V10.86': 3, '429.6': 3, '592.9': 3, '768.5': 3, '63.3': 3, '65.89': 3, 'V10.12': 3, '576.9': 3, '800.30': 3, '813.08': 3, '865.14': 3, '512.84': 3, '624.8': 3, '747.63': 3, '715.37': 3, 'V18.59': 3, '64.91': 3, '36.09': 3, '883.1': 3, '645.11': 3, 'V18.9': 3, '583.2': 3, '38.36': 3, '752.0': 3, '863.49': 3, '471.9': 3, '903.9': 3, '873.60': 3, '368.41': 3, '45.30': 3, '794.09': 3, 'V54.27': 3, '410.90': 3, '176.9': 3, '727.43': 3, '908.0': 3, 'E886.0': 3, '68.23': 3, '82.0': 3, '617.5': 3, '79.27': 3, '013.00': 3, '874.12': 3, '800.71': 3, '958.0': 3, '039.8': 3, '719.42': 3, '164.9': 3, '71.79': 3, '716.98': 3, '39.3': 3, '306.9': 3, '661.11': 3, '864.19': 3, '172.8': 3, '145.0': 3, '29.51': 3, '714.1': 3, '13.41': 3, '13.71': 3, '389.03': 3, '67.12': 3, '78.53': 3, '173.9': 3, '334.9': 3, '576.3': 3, '404.90': 3, '674.34': 3, '237.1': 3, '39.92': 3, '655.83': 3, 'V54.15': 3, '38.42': 3, '598.2': 3, '99.76': 3, '81.95': 3, '620.0': 3, '292.89': 3, '901.82': 3, '79.88': 3, '909.9': 3, '863.43': 3, '354.5': 3, '171.2': 3, '744.41': 3, '750.19': 3, '342.11': 3, '524.69': 3, '788.37': 3, '18.09': 3, '307.23': 3, '93.93': 3, '536.1': 3, '922.0': 3, '374.31': 3, '81.83': 3, 'V60.4': 3, '58.31': 3, '352.6': 3, '951.3': 3, '801.75': 3, '331.89': 3, '052.0': 3, '83.03': 3, '634.11': 3, '743.49': 3, '79.52': 3, '840.8': 3, '870.1': 3, '814.09': 3, '97.38': 3, '382.01': 3, '782.8': 3, '530.9': 3, '680.6': 3, '873.22': 3, '488.01': 3, 'E906.3': 3, '770.18': 3, '812.59': 3, '097.9': 3, '78.52': 3, '851.06': 3, '44.2': 3, '525.3': 3, '237.0': 3, '70.12': 3, 'E879.7': 3, '865.12': 3, '861.30': 3, '93.16': 3, '80.82': 3, '312.39': 3, '727.89': 3, '906.8': 3, '079.0': 3, '435.0': 3, '618.4': 3, '16.59': 3, '70.72': 3, '308.9': 3, '12.92': 3, '365.22': 3, '275.40': 3, '770.16': 3, '803.36': 3, '80.73': 3, '37.29': 3, '27.31': 3, '614.9': 3, '852.03': 3, '712.26': 3, 'E932.2': 3, '379.56': 3, 'V16.51': 3, '73.0': 3, '751.60': 3, '569.44': 3, '816.13': 3, '988.1': 3, 'E865.5': 3, '642.31': 3, 'E945.1': 3, '354.1': 3, '909.4': 3, '593.70': 3, '93.46': 3, '402.00': 3, '18.0': 3, '32.24': 3, '216.7': 3, '172.7': 3, '85.82': 3, '728.3': 3, '802.35': 3, '188.4': 3, '22.9': 3, 'V58.81': 3, '786.01': 3, '71.1': 3, '728.6': 3, '335.22': 3, '335.23': 3, '627.0': 3, '49.2': 3, '971.2': 3, '42.1': 3, '307.59': 3, '236.7': 3, '188.3': 3, '642.41': 3, '651.01': 3, 'V10.9': 3, '525.11': 3, 'E874.4': 3, '669.34': 3, '327.42': 3, '455.4': 3, '868.10': 3, '839.04': 3, '831.03': 3, '602.8': 3, '60.0': 3, '255.2': 3, '115.90': 3, 'E957.0': 3, '44.92': 3, '648.41': 3, '793.99': 3, '791.3': 3, '289.52': 3, '80.5': 3, '373.2': 3, '300.15': 3, '301.3': 3, '783.43': 3, 'E938.0': 3, '863.53': 3, '958.99': 3, '784.61': 3, '736.6': 3, '367.4': 3, '147.1': 3, '873.71': 3, '58.91': 3, '80.0': 3, '410.22': 3, '253.4': 3, '433.81': 3, '614.5': 3, '343.0': 3, '176.4': 3, '288.02': 3, '437.6': 3, '716.15': 3, '53.83': 3, '83.77': 3, '389.22': 3, '249.60': 3, '795.39': 3, '51.93': 3, '788.33': 3, '368.47': 3, '85.11': 3, '908.6': 3, '649.14': 3, '80.95': 3, '32.27': 3, '188.1': 3, '355.0': 3, '378.53': 3, '801.71': 3, '209.79': 3, '78.14': 3, '551.8': 3, '569.71': 3, '209.61': 3, '288.1': 3, '50.23': 3, '861.20': 3, '616.89': 3, '575.2': 3, '362.52': 3, '53.84': 3, '865.10': 3, '800.36': 3, '348.81': 3, 'V90.81': 3, 'E976': 3, '274.02': 3, '372.40': 3, '359.22': 3, '974.7': 3, '204.02': 3, '24.32': 3, '17.35': 3, 'E927.0': 3, 'E000.8': 3, 'V15.01': 3, '780.72': 3, '312.30': 3, '81.93': 3, '202.08': 3, '176.0': 3, 'E920.3': 3, '902.42': 3, '531.41': 3, '378.55': 3, '265.0': 3, 'V54.82': 3, 'V88.21': 3, '206.00': 3, '904.51': 3, '84.73': 3, '85.73': 3, '339.3': 3, '209.73': 3, '136.1': 3, '611.79': 3, '642.33': 3, '787.04': 3, '41.1': 3, '628.9': 3, 'V15.42': 3, '239.89': 3, '732.4': 3, 'E967.9': 3, '695.10': 3, 'V42.6': 3, '200.08': 2, '333.4': 2, '360.19': 2, '364.00': 2, '88.26': 2, '39.28': 2, '839.00': 2, '16.09': 2, 'V58.43': 2, '215.6': 2, '235.6': 2, '839.61': 2, '33.41': 2, '323.6': 2, '98.04': 2, '820.32': 2, 'V04.89': 2, '800.70': 2, '755.26': 2, '37.95': 2, '51.24': 2, '41.32': 2, '253.9': 2, '942.24': 2, '96.70': 2, '960.4': 2, '967.1': 2, '730.09': 2, '975.5': 2, '945.36': 2, '040.89': 2, '824.7': 2, '803.01': 2, '747.82': 2, '862.9': 2, '865.11': 2, '72.1': 2, '959.8': 2, '743.61': 2, 'E848': 2, '305.30': 2, '863.44': 2, '801.14': 2, 'E834.8': 2, '997.71': 2, '748.4': 2, '144.8': 2, '43.': 2, '355.6': 2, '84.02': 2, '81.75': 2, '987.9': 2, 'E890.2': 2, '205.30': 2, '755.39': 2, '82.72': 2, '748.8': 2, '580.0': 2, '152.1': 2, '956.1': 2, '337.0': 2, '78.37': 2, '871.2': 2, '78.58': 2, '079.3': 2, '763.84': 2, '180.0': 2, '986': 2, '99.9': 2, '789.31': 2, '195.3': 2, '093.1': 2, '639.1': 2, '070.33': 2, '217': 2, '836.2': 2, '737.22': 2, '729.30': 2, '81.34': 2, '89.65': 2, '523.3': 2, 'E949.6': 2, '370.8': 2, '863.54': 2, '880.12': 2, 'V83.81': 2, '730.23': 2, '801.62': 2, '928.10': 2, '38.8': 2, '139.0': 2, '727.40': 2, '639.8': 2, 'V05.2': 2, '52.13': 2, '813.51': 2, '137.3': 2, 'E934.3': 2, '852.09': 2, '201.52': 2, '140.9': 2, 'V25.2': 2, '212.5': 2, '64.': 2, '110.9': 2, '21.69': 2, '42.51': 2, '843.9': 2, '642.03': 2, '821.22': 2, '956.2': 2, '800.15': 2, '917.1': 2, '86.9': 2, '032.85': 2, '658.01': 2, '69.59': 2, '806.16': 2, '532.01': 2, '873.51': 2, '053.20': 2, '902.87': 2, '151.3': 2, '755.55': 2, '932': 2, '804.85': 2, '641.11': 2, '669.11': 2, '87.64': 2, 'V58.0': 2, '788.32': 2, '866.10': 2, '803.32': 2, '161.3': 2, '744.9': 2, '983.1': 2, 'E864.1': 2, '806.39': 2, '659.63': 2, '654.23': 2, '188.0': 2, '077.99': 2, '800.75': 2, '836.51': 2, '779.9': 2, '996.40': 2, '238.77': 2, '642.01': 2, '646.21': 2, '674.82': 2, '648.42': 2, '478.0': 2, '21.62': 2, '379.40': 2, '962.7': 2, '814.02': 2, '81.79': 2, '76.93': 2, 'E988.9': 2, '383.1': 2, '381.00': 2, '20.41': 2, 'E932.5': 2, '656.41': 2, '279.3': 2, '229.8': 2, '296.22': 2, '812.10': 2, '79.22': 2, '84.14': 2, '096': 2, '70.92': 2, '754.60': 2, '753.6': 2, '23.3': 2, '333.5': 2, '015.04': 2, '320.0': 2, 'V51': 2, '200.21': 2, 'E879.3': 2, '94.27': 2, '45.28': 2, '747.62': 2, '99.95': 2, '916.1': 2, '45.63': 2, '151.1': 2, '175.9': 2, '282.9': 2, '84.55': 2, '38.98': 2, '52.51': 2, '900.81': 2, 'E855.2': 2, '478.9': 2, '461.2': 2, '018.03': 2, '295.40': 2, '110.0': 2, 'E871.4': 2, '71.11': 2, '37.85': 2, '363.62': 2, '820.01': 2, '768.3': 2, '821.32': 2, '32.': 2, '902.33': 2, '674.51': 2, '726.69': 2, '172.0': 2, '202.85': 2, '760.2': 2, '82.57': 2, '238.72': 2, '76.3': 2, '767.11': 2, '84.23': 2, '747.22': 2, '70.23': 2, '755.00': 2, '748.2': 2, '737.32': 2, '215.4': 2, 'E862.4': 2, '851.25': 2, '726.71': 2, '754.1': 2, '433.91': 2, '654.51': 2, '669.41': 2, '69.96': 2, '78.64': 2, '324.9': 2, '89.37': 2, 'E968.7': 2, '344.02': 2, '474.12': 2, '996.54': 2, '726.90': 2, '850.2': 2, '79.25': 2, '80.14': 2, '902.20': 2, '756.71': 2, 'E988.8': 2, '375.56': 2, '78.04': 2, '814.05': 2, '814.06': 2, '005.81': 2, '654.41': 2, '078.89': 2, '44.97': 2, '995.63': 2, '282.7': 2, '356.2': 2, 'E821.2': 2, '704.09': 2, '844.0': 2, '57.84': 2, '97.54': 2, 'V18.8': 2, '78.45': 2, '767.3': 2, '97.33': 2, '600.10': 2, '801.05': 2, '873.30': 2, '580.4': 2, '752.9': 2, '97.59': 2, '909.0': 2, '85.54': 2, '812.51': 2, '749.20': 2, '194.1': 2, '88.62': 2, '866.11': 2, '743.62': 2, '474.8': 2, '97.14': 2, '581.89': 2, '763.3': 2, '619.8': 2, '553.00': 2, '141.4': 2, '25.3': 2, '743.63': 2, '897.0': 2, '80.84': 2, '96.22': 2, 'V09.50': 2, '762.0': 2, '69.41': 2, '820.19': 2, '813.31': 2, '863.93': 2, '80.89': 2, '96.53': 2, '801.60': 2, '815.12': 2, '410.32': 2, 'E931.4': 2, '35.95': 2, '47.2': 2, '284.0': 2, '754.2': 2, '64.96': 2, '48.82': 2, '49.49': 2, '812.49': 2, '38.15': 2, '79.68': 2, 'V44.8': 2, '82.11': 2, '861.03': 2, '788.34': 2, '959.11': 2, '51.13': 2, '995.2': 2, '626.4': 2, '78.33': 2, '81.72': 2, '201.92': 2, '202.18': 2, '173.6': 2, 'V16.59': 2, '391.0': 2, '271.8': 2, '998.9': 2, '853.09': 2, '98.21': 2, '79.78': 2, '811.10': 2, '902.26': 2, '164.1': 2, '715.16': 2, '997.72': 2, '629.89': 2, 'V84.01': 2, '79.56': 2, '814.19': 2, '77.84': 2, '86.7': 2, '596.55': 2, '27.22': 2, '646.23': 2, '64.3': 2, '608.2': 2, '43.3': 2, 'E820.0': 2, 'E813.3': 2, '852.39': 2, '267': 2, '70.52': 2, '800.06': 2, '44.11': 2, '873.54': 2, '560.31': 2, '233.3': 2, '71.61': 2, '053.71': 2, '42.22': 2, '626.6': 2, '945.34': 2, '991.3': 2, '350.9': 2, '941.27': 2, '12.91': 2, '83.01': 2, '806.8': 2, '33.9': 2, '735.0': 2, '81.14': 2, '77.59': 2, '761.8': 2, '752.69': 2, '73.6': 2, '692.89': 2, '99.53': 2, '861.12': 2, '901.41': 2, '460': 2, '013.25': 2, '946.2': 2, 'E880.0': 2, '174.3': 2, '054.5': 2, '45.21': 2, '93.52': 2, '804.00': 2, 'V58.3': 2, '66.69': 2, '900.02': 2, 'V61.0': 2, '120.8': 2, '84.04': 2, '213.0': 2, 'V58.73': 2, 'E929.4': 2, '955.9': 2, '905.5': 2, 'E826.0': 2, '152.9': 2, '945.32': 2, '534.41': 2, 'E865.4': 2, '880.20': 2, '359.0': 2, '38.57': 2, '719.13': 2, '40.69': 2, '355.5': 2, '077.8': 2, '803.11': 2, '29.39': 2, '755.21': 2, '37.96': 2, '233.4': 2, '96.19': 2, '258.1': 2, '621.3': 2, '897.2': 2, '989.5': 2, '011.94': 2, '730.13': 2, '31.61': 2, '14.54': 2, '58.93': 2, '879.3': 2, 'E920.4': 2, '40.52': 2, '892.2': 2, '352.2': 2, '159.9': 2, 'V62.89': 2, '87.59': 2, '010.85': 2, '305.71': 2, '838.09': 2, '410.00': 2, '901.9': 2, '551.3': 2, '764.99': 2, '647.82': 2, '343.4': 2, '81.55': 2, '236.2': 2, '31.45': 2, '171.8': 2, '812.43': 2, '366.41': 2, '680.5': 2, '976.6': 2, '776.3': 2, '216.9': 2, '807.3': 2, '764.16': 2, '176.3': 2, '112.85': 2, '202.13': 2, '78.56': 2, 'V43.82': 2, '435.1': 2, '804.76': 2, 'V22.2': 2, '61.2': 2, '956.9': 2, '600.2': 2, '60.3': 2, '952.06': 2, '203.10': 2, '37.2': 2, '300.16': 2, '38.61': 2, '83.87': 2, '68.12': 2, '801.52': 2, '851.45': 2, '851.96': 2, '813.07': 2, '516.9': 2, '57.59': 2, 'V56.1': 2, '947.1': 2, '743.30': 2, '743.20': 2, '789.34': 2, '863.31': 2, '306.8': 2, 'E876.4': 2, '49.71': 2, '847.9': 2, '031.1': 2, 'E936.0': 2, '719.09': 2, '928.00': 2, '242.10': 2, '880.23': 2, '736.29': 2, '438.81': 2, '445.89': 2, '380.22': 2, '112.83': 2, '717.6': 2, '800.23': 2, '85.47': 2, '754.61': 2, '52.53': 2, '52.92': 2, '738.0': 2, '379.09': 2, '567.0': 2, '202.87': 2, '803.16': 2, 'E870.3': 2, '963.1': 2, '804.70': 2, '76.0': 2, '18.29': 2, '801.41': 2, '216.6': 2, '744.89': 2, 'V53.99': 2, '580.81': 2, '960.5': 2, '75.32': 2, '951.5': 2, '80.46': 2, '642.64': 2, '801.80': 2, '79.21': 2, 'E871.8': 2, '736.72': 2, '079.4': 2, '966.4': 2, '527.3': 2, '806.02': 2, '144.9': 2, '919.8': 2, '744.23': 2, '634.01': 2, '339.89': 2, '712.22': 2, '727.82': 2, '84.19': 2, '862.32': 2, 'E858.5': 2, '618.3': 2, '616.2': 2, 'E841.5': 2, '92.15': 2, '37.1': 2, '733.12': 2, '378.42': 2, '770.87': 2, '461.1': 2, '991.2': 2, '841.8': 2, '626.9': 2, '995.80': 2, 'E967.3': 2, 'E851': 2, 'E811.0': 2, '607.3': 2, '84.03': 2, '82.81': 2, 'E876.9': 2, '77.33': 2, '782.62': 2, '89.41': 2, '945.22': 2, 'E924.2': 2, '86.1': 2, '15.7': 2, '79.86': 2, '57.34': 2, '790.1': 2, '20.7': 2, '386.10': 2, '42.84': 2, '214.2': 2, '289.1': 2, '45.29': 2, '360.03': 2, '14.75': 2, '77.78': 2, '621.4': 2, '388.72': 2, '160.2': 2, '370.9': 2, '803.60': 2, '66.62': 2, '287.2': 2, '438.52': 2, '48.35': 2, 'V18.1': 2, '600.11': 2, '86.65': 2, '804.36': 2, '837.0': 2, '953.0': 2, '92.30': 2, '681.02': 2, '301.50': 2, '995.67': 2, '201.91': 2, '648.31': 2, 'V29.2': 2, '335.10': 2, '334.8': 2, '744.42': 2, 'V55.8': 2, '163.8': 2, '727.42': 2, '794.01': 2, '719.66': 2, '355.79': 2, '145.2': 2, '644.03': 2, 'E876.2': 2, '202.01': 2, '343.8': 2, '93.26': 2, '904.42': 2, '673.24': 2, '397.1': 2, '21.61': 2, '417.0': 2, '616.50': 2, '733.02': 2, '741.03': 2, '239.4': 2, '252.8': 2, '813.93': 2, '287.33': 2, '815.13': 2, '478.70': 2, '29.33': 2, '590.11': 2, '369.01': 2, '670.02': 2, '791.6': 2, 'E900.0': 2, 'E955.4': 2, '188.5': 2, '008.62': 2, '208.91': 2, '994.8': 2, '070.59': 2, '304.80': 2, '614.2': 2, '866.13': 2, '876.1': 2, '80.71': 2, '80.31': 2, '924.3': 2, 'E943.3': 2, '55.39': 2, '800.16': 2, '24.11': 2, '361.07': 2, 'E954': 2, '230.9': 2, '233.1': 2, '672.02': 2, '78.29': 2, '54.73': 2, '373.11': 2, '86.95': 2, '910.8': 2, '003.0': 2, '33.42': 2, '828.0': 2, '864.11': 2, '812.30': 2, '995.64': 2, 'V11.8': 2, '831.09': 2, '250.23': 2, '995.7': 2, '621.8': 2, '65.2': 2, '378.71': 2, '360.43': 2, '230.0': 2, '536.40': 2, '648.01': 2, '250.91': 2, '768.7': 2, '74.4': 2, 'V10.59': 2, '64.11': 2, '60.91': 2, '374.20': 2, 'V18.51': 2, '49.47': 2, 'E871.7': 2, '911.2': 2, '379.8': 2, '736.09': 2, '908.2': 2, '727.41': 2, '186.9': 2, '851.35': 2, '944.23': 2, '523.31': 2, '76.11': 2, '800.24': 2, '40.19': 2, '64.2': 2, '454.8': 2, '48.29': 2, '365.20': 2, '906.7': 2, '621.30': 2, '053.12': 2, '770.88': 2, '157.4': 2, '812.19': 2, '839.69': 2, '44.91': 2, '51.99': 2, '611.71': 2, '93.99': 2, '159.8': 2, '610.1': 2, '384.20': 2, '532.51': 2, '81.22': 2, '374.43': 2, '736.71': 2, '211.9': 2, '832.09': 2, '807.6': 2, '716.93': 2, '366.16': 2, 'E899': 2, '85.81': 2, 'E944.1': 2, '216.3': 2, '624.9': 2, '365.89': 2, '76.77': 2, '374.10': 2, 'E831.1': 2, '084.4': 2, '726.12': 2, '608.20': 2, 'E855.5': 2, '523.33': 2, '146.8': 2, '18.39': 2, 'V58.44': 2, '730.20': 2, '523.30': 2, '151.6': 2, '733.29': 2, '304.40': 2, '163.0': 2, '429.2': 2, '995.89': 2, '813.13': 2, '925.2': 2, '297.8': 2, 'E030': 2, '45.81': 2, '969.03': 2, '362.83': 2, '796.9': 2, '87.66': 2, '132.0': 2, '79.89': 2, '305.73': 2, '213.7': 2, '737.11': 2, '819.0': 2, '302.85': 2, '902.25': 2, 'V83.89': 2, '290.42': 2, '959.2': 2, '003.8': 2, '916.5': 2, '947.2': 2, '320.89': 2, '845.09': 2, 'E000.0': 2, '054.12': 2, '361.2': 2, '669.14': 2, '983.2': 2, 'E950.5': 2, '347.10': 2, '825.24': 2, '80.91': 2, '907.4': 2, 'V86.1': 2, '789.60': 2, '718.55': 2, '712.35': 2, '695.50': 2, '846.1': 2, '333.81': 2, '42.59': 2, '275.02': 2, '801.50': 2, '802.34': 2, 'E002.6': 2, '836.4': 2, '249.80': 2, 'E938.6': 2, '670.14': 2, '531.91': 2, 'V07.8': 2, '812.54': 2, '349.39': 2, '843.8': 2, '21.4': 2, '923.9': 2, '848.8': 2, '531.60': 2, '33.79': 2, '176.1': 2, '719.12': 2, 'V01.6': 2, '525.40': 2, '863.94': 2, '17.56': 2, '719.96': 2, '032.9': 2, '894.0': 2, '516.33': 2, '84.05': 2, '531.30': 2, '695.13': 2, '368.00': 2, '054.13': 2, '695.14': 2, '362.51': 2, '602.1': 2, '39.78': 2, '082.49': 2, 'E003.9': 2, '307.52': 2, '550.00': 2, '64.98': 2, '799.29': 2, '716.50': 2, '677': 2, 'V10.71': 2, 'V53.6': 2, '388.5': 2, '26.3': 2, '813.53': 2, '79.84': 2, '648.11': 2, 'E008.9': 2, 'E874.8': 2, '122.8': 2, '259.8': 2, '359.5': 2, '209.43': 2, 'E019.0': 2, '72.9': 2, '799.23': 2, '288.63': 2, '27.61': 2, '41.99': 2, '287.32': 2, '664.11': 2, '715.93': 2, 'V56.8': 2, 'V14.3': 2, '205.12': 2, '611.9': 2, '375.30': 2, 'V49.61': 2, '49.23': 2, '719.58': 2, '362.32': 2, '752.89': 2, '512.83': 2, '39.8': 2, '695.15': 2, '536.9': 2, '639.0': 2, '266.9': 2, 'E817.0': 2, '150.0': 2, '593.73': 2, '91.46': 2, '43.82': 2, '362.33': 2, '53.43': 2, '365.73': 2, '784.43': 2, '953.1': 2, '86.96': 2, '377.51': 2, '649.43': 2, '646.53': 2, '649.13': 2, '599.3': 2, 'V25.41': 2, 'E933.4': 2, '209.75': 2, '784.91': 2, '60.95': 2, '53.14': 2, '339.20': 2, '76.4': 2, 'E945.5': 2, '538': 2, '527.8': 2, '93.08': 2, '874.5': 2, '814.11': 2, '901.81': 2, '304.43': 2, '369.10': 2, '342.12': 2, '365.11': 2, '79.46': 2, '737.29': 2, '81.0': 2, '148.8': 2, '695.0': 2, '099.3': 2, '518.6': 2, '368.44': 2, '960.0': 2, '801.61': 2, '805.5': 2, '44.68': 2, '729.90': 2, '43.6': 2, '686.8': 2, '237.72': 2, '237.9': 2, '359.3': 2, '031.8': 2, '529.6': 2, '611.89': 2, '362.75': 2, '82.35': 2, 'E824.0': 2, '053.11': 2, '344.41': 2, '534.10': 2, '310.8': 2, '202.03': 2, '799.53': 2, '295.64': 2, '453.79': 2, '334.4': 2, '203.12': 2, '996.70': 2, '447.73': 2, '249.90': 2, '753.14': 2, '620.9': 2, '447.72': 2, '727.62': 2, '80.41': 2, '799.59': 2, 'V13.64': 2, '909.5': 2, '173.8': 2, '649.31': 2, 'V27.1': 2, '834.00': 2, '238.73': 2, '379.22': 2, '525.63': 2, '78.62': 2, '451.2': 2, '523.42': 2, '913.2': 2, '954.0': 2, '25.59': 2, '173.1': 2, 'E934.0': 2, '46.4': 2, '786.9': 2, '389.18': 2, '237.73': 2, '78.02': 2, 'E007.6': 2, '756.16': 2, '670.12': 2, '189.3': 2, '81.1': 2, '210.2': 2, '969.79': 2, '346.70': 2, '665.31': 2, '80.49': 2, '284.2': 2, '393': 2, 'V64.06': 2, '272.5': 2, '712.39': 2, '81.97': 2, '337.21': 2, '753.11': 2, '79.55': 2, '493.82': 2, '179': 2, '996.87': 2, '26.0': 2, '088.0': 1, '711.04': 1, '951.8': 1, '804.16': 1, '532.91': 1, '959.6': 1, '795.00': 1, '370.05': 1, '16.49': 1, '255.3': 1, '76.69': 1, '97.64': 1, '745.0': 1, '766.2': 1, '928.11': 1, '301.0': 1, '127.0': 1, '160.3': 1, '741.01': 1, '795.89': 1, '794.00': 1, '12.81': 1, 'E885.1': 1, '902.0': 1, '054.71': 1, '923.8': 1, '800.09': 1, '364.9': 1, '083.9': 1, '723.6': 1, '16.31': 1, '754.53': 1, 'V12.2': 1, '711.80': 1, '454.2': 1, '760.77': 1, 'V61.49': 1, '049.0': 1, '994.2': 1, '97.75': 1, 'V11.1': 1, '713.2': 1, '214.9': 1, '89.03': 1, '298.4': 1, '790.09': 1, '38.80': 1, '388.32': 1, '41.42': 1, '754.50': 1, 'V12.3': 1, '63.2': 1, '902.29': 1, '35.5': 1, '357.8': 1, '92.04': 1, '852.12': 1, '755.57': 1, '82.69': 1, '81.74': 1, '834.01': 1, '358.8': 1, '750.10': 1, '736.00': 1, '81.71': 1, 'E922.2': 1, '755.29': 1, '362.84': 1, '46.8': 1, '87.63': 1, '70.74': 1, '35.41': 1, '117.4': 1, '719.86': 1, '718.95': 1, '96.14': 1, '643.03': 1, '005.1': 1, 'E884.0': 1, '410.20': 1, '639.2': 1, '67.61': 1, '743.43': 1, '34.83': 1, '871.3': 1, 'E822.8': 1, '593.71': 1, '45.00': 1, '232.9': 1, '71.4': 1, '493.01': 1, '134.8': 1, 'E815.6': 1, '29.9': 1, '665.82': 1, '851.44': 1, 'E823.3': 1, '391.8': 1, '633.20': 1, '639.6': 1, 'V23.0': 1, '74.3': 1, '904.6': 1, 'V49.63': 1, '88.6': 1, '77.75': 1, '376.11': 1, '376.12': 1, '173.0': 1, '642.71': 1, '648.62': 1, 'V43.5': 1, '26.91': 1, '58.41': 1, '927.21': 1, '86.71': 1, '160.9': 1, 'E819.3': 1, '753.7': 1, '57.51': 1, '690.18': 1, '635.92': 1, '78.48': 1, '83.11': 1, 'E919.6': 1, '595.81': 1, '665.34': 1, '654.04': 1, '91.72': 1, '928.01': 1, 'E953.8': 1, '674.02': 1, '669.21': 1, '295.73': 1, '036.41': 1, '787.4': 1, '254.1': 1, '38.29': 1, '11.49': 1, '81.8': 1, '99.74': 1, '804.75': 1, '778.9': 1, '242.81': 1, '213.2': 1, '801.42': 1, '806.62': 1, '86.03': 1, '777.9': 1, '81.57': 1, '755.31': 1, '142.9': 1, '975.3': 1, '789.40': 1, '68.1': 1, '669.42': 1, '761.0': 1, '791.5': 1, '383.02': 1, '383.21': 1, '20.01': 1, '140.1': 1, 'E940.8': 1, '251.9': 1, '813.92': 1, '930.1': 1, '16.81': 1, '10.6': 1, '763.6': 1, '004.9': 1, '852.56': 1, 'V69.8': 1, '765.06': 1, '780.31': 1, '99.84': 1, 'V13.5': 1, '54.': 1, '146.3': 1, 'E814.0': 1, '49.93': 1, '49.03': 1, '097.0': 1, '696.3': 1, '254.0': 1, '801.09': 1, '755.66': 1, 'V54.23': 1, '730.29': 1, '77.82': 1, '012.15': 1, '018.80': 1, '131.02': 1, '780.55': 1, '361.05': 1, '14.34': 1, '202.90': 1, '446.20': 1, 'E941.0': 1, 'V50.3': 1, '697.9': 1, '744.3': 1, '736.70': 1, '94.63': 1, '521.9': 1, '46.62': 1, 'E821.9': 1, '738.10': 1, '666.04': 1, '372.75': 1, '82.22': 1, '902.34': 1, '125.1': 1, '94.2': 1, 'V19.2': 1, '634.51': 1, '48.25': 1, '815.04': 1, '812.52': 1, '018.96': 1, '97.53': 1, '87.79': 1, '69.49': 1, '745.11': 1, '806.22': 1, '375.01': 1, '374.89': 1, '959.14': 1, '802.32': 1, '887.1': 1, '114.0': 1, 'V60.8': 1, '923.01': 1, '16.63': 1, 'V42.84': 1, '307.6': 1, 'V06.2': 1, '81.15': 1, '307.49': 1, '726.19': 1, '305.83': 1, '635.02': 1, '281.8': 1, '78.0': 1, '377.02': 1, '68.0': 1, '750.0': 1, '750.26': 1, '25.92': 1, '26.21': 1, '390': 1, '93.94': 1, '788.69': 1, '814.12': 1, '814.07': 1, '798.1': 1, '478.11': 1, '45.': 1, '642.92': 1, '646.82': 1, '873.21': 1, '55.': 1, '791.2': 1, '768.6': 1, '726.73': 1, 'V76.51': 1, '952.15': 1, '33.92': 1, '803.24': 1, '200.05': 1, '146.4': 1, '765.08': 1, '31.62': 1, '212.2': 1, '742.8': 1, '620.3': 1, 'E876.1': 1, '275.8': 1, '389.08': 1, '16.52': 1, 'V02.4': 1, '216.4': 1, '25.02': 1, '23.11': 1, '56.84': 1, '656.13': 1, '140.0': 1, '755.60': 1, '761.1': 1, '286.52': 1, '754.33': 1, '87.09': 1, '806.31': 1, '170.2': 1, '976.7': 1, 'E958.1': 1, '804.23': 1, '828.1': 1, '671.53': 1, '76.64': 1, '91.63': 1, '50.': 1, '350.2': 1, '535.20': 1, '851.84': 1, '67.39': 1, '56.42': 1, '058.89': 1, '54.71': 1, 'E947.0': 1, '288.9': 1, '77.09': 1, '114.9': 1, '53.12': 1, '792.9': 1, '59.95': 1, '896.0': 1, '801.96': 1, '778.5': 1, '146.7': 1, '26.30': 1, '379.90': 1, 'V58.42': 1, '004.8': 1, '952.4': 1, '611.6': 1, '803.06': 1, '99.75': 1, '853.10': 1, '219.1': 1, '221.0': 1, '365.00': 1, 'E819.6': 1, '803.41': 1, '779.1': 1, '86.02': 1, '551.29': 1, '293.84': 1, '309.3': 1, '781.7': 1, '735.8': 1, '202.11': 1, 'E831.8': 1, '747.89': 1, '887.3': 1, '754.30': 1, '750.16': 1, '755.22': 1, '752.65': 1, '084.6': 1, 'V09.71': 1, '930.8': 1, '749.02': 1, '31.92': 1, '21.00': 1, '838.05': 1, '815.11': 1, '867.7': 1, '839.79': 1, '926.0': 1, '96.39': 1, '49.79': 1, '79.79': 1, '96.24': 1, '804.73': 1, '594.2': 1, '765.11': 1, '438.31': 1, '154.2': 1, '60.62': 1, '976.0': 1, '812.12': 1, 'E957.9': 1, 'V46.9': 1, '279.11': 1, '995.62': 1, '70.79': 1, '474.00': 1, '85.95': 1, '57.11': 1, '41.08': 1, '753.20': 1, '814.18': 1, '903.4': 1, 'E986': 1, '008.47': 1, '524.62': 1, '618.2': 1, '671.54': 1, '719.11': 1, '728.4': 1, '531.20': 1, '854.00': 1, '88.40': 1, '78.08': 1, '145.8': 1, '374.52': 1, '451.11': 1, '735.5': 1, '924.5': 1, '750.5': 1, '422.92': 1, '952.3': 1, '643.13': 1, '364.41': 1, '160.8': 1, '21.32': 1, '770.9': 1, '527.7': 1, 'E821.6': 1, '97.35': 1, '900.9': 1, '852.19': 1, '803.50': 1, '443.1': 1, '362.17': 1, '618.04': 1, '70.8': 1, '733.40': 1, '63.81': 1, '011.36': 1, '923.3': 1, '961.8': 1, '961.4': 1, 'V16.43': 1, 'V90.89': 1, '52.95': 1, '941.28': 1, '41.43': 1, '757.5': 1, '211.7': 1, '66.61': 1, '164.3': 1, '944.21': 1, '989.3': 1, '414.03': 1, '77.58': 1, '727.1': 1, '77.56': 1, '89.67': 1, '29.6': 1, '692.0': 1, '245.4': 1, '14.22': 1, '376.35': 1, '801.45': 1, '258.8': 1, 'E870.2': 1, '81.99': 1, 'V23.9': 1, '88.35': 1, '948.40': 1, '719.26': 1, '202.43': 1, '85.48': 1, '705.21': 1, '779.6': 1, '550.93': 1, '079.53': 1, '874.2': 1, '395.9': 1, 'E834.1': 1, '200.01': 1, '996.83': 1, '251.0': 1, '15.': 1, '832.02': 1, '192.0': 1, 'E908.1': 1, '447.5': 1, 'E801.2': 1, '800.82': 1, 'E840.5': 1, 'V46.8': 1, '794.6': 1, '804.12': 1, '21.86': 1, '130.8': 1, '716.16': 1, 'V54.26': 1, '380.15': 1, '272.7': 1, '277.8': 1, '89.50': 1, '96.41': 1, 'V32.00': 1, '719.17': 1, '404.13': 1, '576.0': 1, '649.44': 1, '211.0': 1, '618.8': 1, '590.9': 1, 'E812.7': 1, '523.4': 1, '070.9': 1, '091.81': 1, '905.2': 1, '99.16': 1, '386.9': 1, '648.33': 1, 'E905.3': 1, '854.03': 1, '349.1': 1, 'E823.8': 1, 'V18.69': 1, 'V64.43': 1, '880.19': 1, '525.12': 1, '765.10': 1, '59.11': 1, 'E967.4': 1, '99.36': 1, '65.51': 1, '801.51': 1, '35.42': 1, '772.1': 1, '184.8': 1, 'E967.7': 1, '886.1': 1, '816.03': 1, '82.86': 1, '774.31': 1, '906.3': 1, '018.95': 1, '018.05': 1, '99.24': 1, '710.5': 1, '861.02': 1, '641.33': 1, '69.93': 1, '41.98': 1, '327.25': 1, '52.83': 1, '763.81': 1, '776.8': 1, '43.91': 1, 'E861.3': 1, '907.5': 1, '68.31': 1, '54.29': 1, 'E949.9': 1, 'E901.1': 1, 'E858.7': 1, '836.3': 1, '81.44': 1, '44.19': 1, '85.45': 1, '646.61': 1, '656.71': 1, '279.9': 1, '202.12': 1, 'E955.9': 1, '070.53': 1, '047.0': 1, '98.26': 1, '793.3': 1, '779.85': 1, '77.35': 1, '617.8': 1, '718.15': 1, '879.1': 1, '584.7': 1, '758.3': 1, '29.0': 1, 'E946.3': 1, '81.82': 1, '738.8': 1, '804.66': 1, '777.3': 1, '37.63': 1, '521.01': 1, '846.0': 1, 'E962.0': 1, '376.9': 1, 'V71.6': 1, '594.9': 1, '718.43': 1, '727.9': 1, '906.1': 1, '748.61': 1, '635.52': 1, '69.29': 1, '904.3': 1, '904.8': 1, '926.12': 1, '927.8': 1, '79.82': 1, '062.2': 1, '813.81': 1, '31.49': 1, '665.11': 1, '365.23': 1, '885.1': 1, '842.09': 1, '111.8': 1, '520.0': 1, '68.51': 1, '41.01': 1, '919.1': 1, '914.9': 1, '40.53': 1, '674.14': 1, '654.44': 1, '669.44': 1, '84.08': 1, '86.21': 1, '70.51': 1, '901.42': 1, '374.9': 1, '552.02': 1, '53.31': 1, '81.16': 1, '374.82': 1, '21.83': 1, '382.00': 1, '79.18': 1, '760.1': 1, '836.52': 1, '323.51': 1, '368.12': 1, '582.0': 1, '583.0': 1, '46.96': 1, '672.04': 1, '863.55': 1, '878.7': 1, '878.5': 1, '71.22': 1, 'V70.3': 1, '841.1': 1, '804.41': 1, 'V40.3': 1, '807.10': 1, '362.13': 1, '642.04': 1, '851.31': 1, '143.1': 1, '551.20': 1, '712.16': 1, '608.0': 1, '60.71': 1, '62.41': 1, '508.8': 1, '75.4': 1, '804.32': 1, '95.25': 1, 'E967.8': 1, 'V12.50': 1, '943.32': 1, '942.34': 1, 'V62.82': 1, '795.09': 1, 'V61.11': 1, 'V33.1': 1, '746.00': 1, '78.38': 1, '447.3': 1, '749.12': 1, 'V28.9': 1, '896.1': 1, 'E920.0': 1, '236.0': 1, '274.89': 1, '89.18': 1, '952.02': 1, '627.8': 1, '090.2': 1, '755.20': 1, '971.0': 1, '733.01': 1, '26.11': 1, '521.08': 1, 'E821.1': 1, '65.52': 1, '851.90': 1, 'E985.0': 1, '379.42': 1, '327.13': 1, 'E814.2': 1, '242.30': 1, '81.40': 1, '974.4': 1, '71.23': 1, 'V40.0': 1, '81.84': 1, '77.43': 1, '76.97': 1, '039.2': 1, '756.11': 1, '300.12': 1, '078.0': 1, '902.51': 1, '42.10': 1, '86.84': 1, '008.61': 1, '714.32': 1, '622.1': 1, '673.11': 1, '668.11': 1, '871.0': 1, '871.6': 1, '200.03': 1, '616.8': 1, '944.20': 1, '82.89': 1, '812.02': 1, '49.02': 1, '232.5': 1, 'V64.0': 1, 'E835.3': 1, '799.2': 1, '749.23': 1, '368.11': 1, '385.82': 1, '20.59': 1, '800.85': 1, '210.1': 1, '711.55': 1, '131.01': 1, '754.40': 1, '852.10': 1, '873.50': 1, '836.61': 1, '846.9': 1, 'V17.2': 1, '098.0': 1, '755.11': 1, '011.93': 1, '851.75': 1, 'V70.8': 1, '300.09': 1, '719.70': 1, '070.0': 1, '999.6': 1, '230.2': 1, 'E885.0': 1, '774.30': 1, '190.5': 1, '98.13': 1, '740.2': 1, '693.8': 1, '53.80': 1, '749.10': 1, 'V49.89': 1, 'E919.3': 1, '361.01': 1, '600.9': 1, '85.53': 1, '730.02': 1, '77.02': 1, '726.72': 1, '97.32': 1, '633.11': 1, '282.1': 1, '730.89': 1, '852.15': 1, '12.73': 1, '14.9': 1, '823.42': 1, '15.2': 1, '283.10': 1, '804.30': 1, '804.10': 1, '953.9': 1, '88.68': 1, '364.03': 1, '738.6': 1, '77.01': 1, 'E924.9': 1, '37.92': 1, '297.2': 1, '715.30': 1, '323.4': 1, '747.32': 1, '282.8': 1, '730.97': 1, '89.52': 1, '43.49': 1, '88.58': 1, '51.86': 1, '719.90': 1, '376.03': 1, '904.50': 1, '836.60': 1, '715.15': 1, '305.43': 1, '665.24': 1, '085.9': 1, '47.91': 1, '673.33': 1, '96.54': 1, 'V64.05': 1, '774.1': 1, '52.19': 1, 'E933.8': 1, '718.56': 1, '847.2': 1, '56.89': 1, '371.43': 1, 'E870.9': 1, '717.84': 1, '837.1': 1, '86.51': 1, '854.04': 1, '957.8': 1, '533.11': 1, '210.4': 1, '27.72': 1, 'E800.2': 1, 'E922.5': 1, 'V66.0': 1, '76.46': 1, '878.6': 1, '145.3': 1, '756.13': 1, '35.35': 1, '202.20': 1, '756.9': 1, 'E939.5': 1, '230.6': 1, '815.14': 1, '054.43': 1, '98.12': 1, 'E963': 1, 'E964': 1, '833.02': 1, '851.09': 1, '27.53': 1, '801.74': 1, '853.14': 1, '801.64': 1, 'E923.8': 1, '82.51': 1, '800.32': 1, '743.59': 1, '18.21': 1, '29.59': 1, '799.01': 1, '95.03': 1, '70.62': 1, '70.75': 1, '290.10': 1, '851.73': 1, '091.2': 1, '698.4': 1, '312.89': 1, '85.25': 1, '45.49': 1, '524.06': 1, '755.69': 1, '204.90': 1, '205.90': 1, '757.2': 1, '594.0': 1, '985.1': 1, 'E866.3': 1, '360.40': 1, '733.20': 1, '54.97': 1, '45.26': 1, '874.11': 1, '774.4': 1, 'V11.0': 1, '292.82': 1, '35.70': 1, '56.62': 1, 'V10.50': 1, '901.2': 1, '78.11': 1, '747.11': 1, '607.1': 1, 'V44.9': 1, '763.4': 1, '44.96': 1, '459.10': 1, '80.79': 1, '313.81': 1, '658.21': 1, '660.21': 1, '654.11': 1, '833.05': 1, '78.54': 1, '718.28': 1, '776.0': 1, 'E858.9': 1, '99.26': 1, '942.03': 1, 'E924.0': 1, '730.19': 1, '307.42': 1, '780.54': 1, '701.3': 1, '023.9': 1, '200.42': 1, '57.31': 1, '735.9': 1, '070.52': 1, '22.5': 1, '21.22': 1, 'V50.49': 1, 'V84.09': 1, '46.22': 1, '813.91': 1, '94.4': 1, 'V62.4': 1, '786.00': 1, '560.30': 1, '312.34': 1, '764.01': 1, '759.7': 1, '021.8': 1, 'V02.0': 1, '81.85': 1, '674.12': 1, '013.04': 1, '011.64': 1, '84.59': 1, 'E824.2': 1, '820.9': 1, '806.5': 1, '867.9': 1, '231.2': 1, 'E957.2': 1, '380.39': 1, '760.70': 1, '752.62': 1, '484.3': 1, '033.8': 1, '861.10': 1, '598.1': 1, '833.11': 1, '944.25': 1, 'V15.86': 1, '365.12': 1, '76.01': 1, 'V02.52': 1, '519.00': 1, '806.23': 1, '879.9': 1, '230.8': 1, '429.81': 1, '625.5': 1, '360.12': 1, '39.21': 1, 'V18.11': 1, '870.3': 1, '46.64': 1, '335.11': 1, '733.96': 1, '718.45': 1, '755.64': 1, '531.01': 1, '989.0': 1, '44.02': 1, '642.61': 1, '755.10': 1, '404.00': 1, 'E806.2': 1, '85.22': 1, '901.40': 1, 'V18.19': 1, '887.5': 1, '955.8': 1, 'V46.14': 1, '296.53': 1, '93.29': 1, '60.21': 1, '42.91': 1, '666.22': 1, '65.79': 1, '793.80': 1, '49.74': 1, 'V10.20': 1, '533.41': 1, '897.7': 1, 'E804.2': 1, '617.2': 1, '633.80': 1, '752.3': 1, '535.21': 1, '302.9': 1, '200.22': 1, '330.8': 1, 'V72.19': 1, '690.12': 1, '200.71': 1, '31.72': 1, '32.9': 1, '722.30': 1, '51.49': 1, 'E863.7': 1, '49.11': 1, '61.42': 1, '642.34': 1, '352.9': 1, '361.06': 1, '417.9': 1, '775.3': 1, '296.51': 1, '749.22': 1, '743.53': 1, '013.54': 1, '12.1': 1, '296.54': 1, '98.27': 1, 'E910.9': 1, 'V61.10': 1, '897.3': 1, '362.07': 1, '831.14': 1, '296.81': 1, '41.39': 1, '839.21': 1, '754.31': 1, '879.0': 1, '716.87': 1, '649.33': 1, '642.93': 1, '384.01': 1, 'E965.9': 1, '972.2': 1, '598.00': 1, '41.91': 1, 'E838.4': 1, '97.87': 1, '14.41': 1, '42.62': 1, '86.82': 1, 'E834.3': 1, '75.1': 1, '745.12': 1, '723.7': 1, '57.79': 1, '665.61': 1, '655.71': 1, '656.61': 1, '659.21': 1, '75.99': 1, '45.52': 1, '531.11': 1, '323.62': 1, '306.2': 1, '712.13': 1, '341.1': 1, '375.00': 1, '754.82': 1, '959.3': 1, '438.85': 1, 'E816.3': 1, '012.05': 1, '16.51': 1, '942.09': 1, '200.11': 1, '315.31': 1, '60.1': 1, '603.1': 1, '176.8': 1, '35.98': 1, '732.5': 1, '62.2': 1, '63.1': 1, '97.52': 1, '56.91': 1, '99.45': 1, '195.2': 1, '757.1': 1, '014.05': 1, '275.9': 1, '27.55': 1, 'E838.1': 1, 'E945.3': 1, '53.42': 1, '69.3': 1, '833.00': 1, '839.07': 1, '202.05': 1, '38.37': 1, 'E985.4': 1, '93.51': 1, '646.81': 1, '724.6': 1, '912.2': 1, '279.41': 1, '82.19': 1, '999.41': 1, '344.5': 1, '732.0': 1, '200.18': 1, '057.8': 1, '819.1': 1, '362.05': 1, '803.72': 1, '70.61': 1, '56.61': 1, '255.12': 1, '209.51': 1, '17.41': 1, 'E980.1': 1, '803.76': 1, '803.66': 1, '132.9': 1, '790.95': 1, '793.81': 1, '046.71': 1, '711.08': 1, '389.17': 1, '55.21': 1, '737.33': 1, '634.12': 1, '249.91': 1, '299.90': 1, '049.1': 1, '058.81': 1, '295.12': 1, '619.2': 1, '642.94': 1, '890.2': 1, '969.71': 1, '973.3': 1, 'E858.4': 1, '567.1': 1, '83.0': 1, '298.0': 1, '840.6': 1, '18.11': 1, '71.6': 1, '715.33': 1, '911.6': 1, '945.04': 1, '616.9': 1, '008.43': 1, '654.14': 1, '665.54': 1, '83.62': 1, '82.43': 1, '35.1': 1, '270.2': 1, 'E999.1': 1, 'E855.4': 1, 'E927.4': 1, '338.22': 1, '947.3': 1, 'E002.0': 1, '84.82': 1, 'E874.0': 1, '666.14': 1, '209.74': 1, '380.21': 1, '649.04': 1, '524.4': 1, '943.20': 1, 'E873.5': 1, '825.32': 1, '795.10': 1, '716.66': 1, '344.42': 1, '405.11': 1, 'E816.9': 1, '386.19': 1, '733.5': 1, '695.12': 1, '97.15': 1, 'V54.10': 1, '474.9': 1, '151.5': 1, '52.98': 1, '003.9': 1, '642.63': 1, '728.79': 1, '999.71': 1, '309.1': 1, '65.62': 1, '195.1': 1, '959.19': 1, '146.1': 1, '76.09': 1, '402.10': 1, '816.10': 1, '235.8': 1, '282.44': 1, '853.16': 1, '323.72': 1, '281.3': 1, 'E003.2': 1, 'E805.8': 1, 'V51.0': 1, '718.48': 1, '76.45': 1, '873.23': 1, 'E029.1': 1, 'V61.09': 1, '696.5': 1, '922.9': 1, '478.71': 1, 'E006.9': 1, '599.5': 1, '851.83': 1, 'E006.0': 1, '800.13': 1, '801.13': 1, '973.5': 1, '750.29': 1, '951.7': 1, '330.0': 1, '551.1': 1, '796.71': 1, '42.69': 1, '999.84': 1, '852.16': 1, '680.9': 1, '880.29': 1, '355.71': 1, '171.7': 1, '014.02': 1, '60.99': 1, '295.22': 1, '995.85': 1, '375.9': 1, '516.34': 1, '41.93': 1, 'E933.5': 1, '42.65': 1, '33.71': 1, '68.69': 1, '347.01': 1, '368.59': 1, 'V61.29': 1, 'E920.5': 1, '698.2': 1, '045.90': 1, '131.09': 1, '89.34': 1, '852.04': 1, '293.81': 1, '851.21': 1, '840.0': 1, '840.7': 1, '491.0': 1, '46.63': 1, '88.19': 1, '551.21': 1, 'V60.1': 1, '189.8': 1, '333.84': 1, '817.1': 1, '45.9': 1, '569.87': 1, '709.00': 1, '55.22': 1, '97.11': 1, '801.44': 1, '200.33': 1, '977.3': 1, '200.78': 1, '755.30': 1, '212.0': 1, '389.21': 1, '19.9': 1, '149.8': 1, '173.22': 1, '20.61': 1, '523.5': 1, '752.2': 1, '752.19': 1, '14.73': 1, 'E913.2': 1, '448.1': 1, '801.46': 1, '834.11': 1, '64.41': 1, '550.01': 1, '300.89': 1, '070.49': 1, '972.5': 1, '627.4': 1, 'E861.9': 1, '902.10': 1, 'E013.8': 1, '621.2': 1, '60.4': 1, 'V40.31': 1, '293.82': 1, '291.89': 1, '253.3': 1, '77.1': 1, '851.62': 1, '296.25': 1, '277.31': 1, '64.97': 1, 'E879.5': 1, '013.30': 1, '79.42': 1, '833.09': 1, '233.31': 1, '70.33': 1, '648.52': 1, '453.74': 1, '288.51': 1, 'V09.1': 1, '426.54': 1, '923.21': 1, '305.31': 1, '69.7': 1, '173.72': 1, '629.81': 1, '26.99': 1, '526.5': 1, '079.83': 1, '745.9': 1, '806.10': 1, '007.1': 1, '665.74': 1, '917.3': 1, '750.8': 1, '380.23': 1, '516.4': 1, 'E824.9': 1, '269.3': 1, '282.64': 1, '379.55': 1, '863.51': 1, '952.07': 1, '642.74': 1, '78.43': 1, '143.0': 1, '346.93': 1, '194.5': 1, '116.0': 1, '306.0': 1, '618.00': 1, 'E910.0': 1, 'E831.4': 1, '200.60': 1, 'E818.9': 1, '47.4': 1, '813.14': 1, '835.03': 1, '948.50': 1, '943.25': 1, '945.19': 1, '48.51': 1, '788.65': 1, '295.02': 1, '032.89': 1, '953.5': 1, '912.5': 1, '066.41': 1, '066.42': 1, '321.2': 1, '736.05': 1, '36.17': 1, '625.4': 1, '569.60': 1, '378.20': 1, '296.14': 1, '88.04': 1, '42.83': 1, '736.30': 1, '68.25': 1, '939.3': 1, 'E913.8': 1, '726.32': 1, '410.40': 1, '209.36': 1, '526.2': 1, '863.83': 1, 'E007.1': 1, 'E016.1': 1, '214.4': 1, 'E813.8': 1, '996.80': 1, '396.9': 1, '722.70': 1, '345.2': 1, 'E818.8': 1, '14.79': 1, '80.54': 1, 'E874.2': 1, '31.95': 1, '388.71': 1, '527.9': 1, '18.12': 1, '85.71': 1, '88.3': 1, '378.82': 1, '674.01': 1, '788.7': 1, 'V54.22': 1, '398.99': 1, 'V15.06': 1, 'V10.22': 1, '333.79': 1, '943.23': 1, '011.23': 1, '21.5': 1, '21.3': 1, '744.01': 1, '795.52': 1, '800.34': 1, '89.5': 1, '357.1': 1, '414.4': 1, '238.74': 1, 'V03.89': 1, 'E825.8': 1, 'V87.45': 1, 'V06.5': 1, '852.45': 1, 'E006.1': 1, '959.5': 1, '360.02': 1, '789.37': 1, '115.05': 1, '799.55': 1, '371.82': 1, '361.00': 1, '581.0': 1, '534.01': 1, '593.1': 1, 'V48.9': 1, 'V87.09': 1, '054.0': 1, '801.70': 1, '719.50': 1, '296.04': 1, 'V13.52': 1, '923.09': 1, '269.2': 1, 'E006.2': 1, '148.9': 1, '209.25': 1, '209.71': 1, '789.47': 1, '712.37': 1, 'V62.85': 1, '362.41': 1, '760.76': 1, '337.01': 1, '789.32': 1, '719.18': 1, 'E977': 1, '146.9': 1, 'V65.5': 1, '27.69': 1, 'E941.9': 1, 'V13.81': 1, '805.3': 1, '388.60': 1, '202.52': 1, '44.64': 1, '295.24': 1, '825.20': 1, 'E921.1': 1, '917.0': 1, 'E008.0': 1, '727.63': 1, '316': 1, '008.04': 1, '87.34': 1, '404.11': 1, '477.2': 1, '058.29': 1, '053.14': 1, '943.35': 1, '96.58': 1, '803.62': 1, '746.82': 1, '366.46': 1, '257.1': 1, '364.70': 1, '003.1': 1, '315.2': 1, '711.98': 1, '721.91': 1, '77.2': 1, '380.00': 1, '330.1': 1, 'E876.7': 1, 'E003.0': 1, 'V53.01': 1, 'V25.01': 1, '659.53': 1, '362.9': 1, '941.08': 1, '529.1': 1, '301.13': 1, '944.01': 1, '87.65': 1, 'V61.03': 1, '362.53': 1, '709.4': 1, '389.05': 1, '307.47': 1, '145.5': 1, '21.89': 1, '242.21': 1, '005.9': 1, '362.04': 1, '327.37': 1, '382.4': 1, '451.81': 1, 'V16.52': 1, 'E804.1': 1, '810.12': 1, 'E946.6': 1, 'E932.4': 1, '582.4': 1, '339.12': 1, '660.01': 1, '652.51': 1, '668.22': 1, '288.2': 1, '24.7': 1, '92.32': 1, '801.23': 1, '583.4': 1, 'E900.1': 1, 'E006.4': 1, '174.5': 1, '201.48': 1, '29.32': 1, '127.3': 1, '200.37': 1, '49.31': 1, '807.18': 1, '823.31': 1, '820.30': 1, '838.01': 1, 'E824.8': 1, 'V48.6': 1, '93.21': 1, '308.1': 1, '410.10': 1, 'E982.1': 1, '942.32': 1, '202.06': 1, '711.91': 1, 'E925.0': 1, '48.76': 1, '69.98': 1, '995.69': 1, '916.4': 1, '80.96': 1, '84.48': 1, '695.81': 1, '551.00': 1, '424.91': 1, '245.8': 1, '60.12': 1, '275.09': 1, 'E932.9': 1, '732.3': 1, '937': 1, '23.73': 1, 'V67.2': 1, 'V67.1': 1, '363.72': 1, '215.0': 1, '995.65': 1, '209.69': 1, '845.12': 1, '209.57': 1, '054.40': 1, 'E007.3': 1, '337.09': 1, '906.0': 1, '647.93': 1, '646.73': 1, '030.9': 1, '788.62': 1, '209.03': 1, '648.83': 1, '642.43': 1, '389.06': 1, 'V81.2': 1, '45.83': 1, '997.00': 1, 'V07.39': 1, '52.99': 1, '190.6': 1, '589.0': 1, '780.53': 1, 'V15.53': 1, '084.9': 1, '736.20': 1, '27.23': 1, '718.65': 1, '80.35': 1, '681.01': 1, '941.00': 1, '295.75': 1, '373.12': 1, '371.89': 1, '76.63': 1, '802.30': 1, '327.24': 1, '872.69': 1, '295.44': 1, '380.03': 1, '221.8': 1, '65.11': 1, '322.1': 1, '015.05': 1, '728.82': 1, '685.0': 1, '756.15': 1, '526.0': 1, '558.41': 1, 'V43.63': 1, 'V88.11': 1, '727.06': 1, '80.33': 1, 'E869.4': 1, '945.24': 1, 'E866.9': 1, '759.83': 1, '626.1': 1, '965.69': 1, '726.64': 1, '851.50': 1, '173.42': 1, '173.32': 1, '173.31': 1, 'E926.2': 1, '800.02': 1, '495.7': 1, '905.6': 1, '46.95': 1, '794.7': 1, '532.11': 1, '718.97': 1, '641.03': 1, '372.14': 1, '81.49': 1, '80.47': 1, 'E905.1': 1, '906.5': 1, '961.0': 1, 'E857': 1, '718.91': 1, '910.2': 1, '516.1': 1, '969.01': 1, '733.45': 1, '32.09': 1, '623.2': 1, 'V53.7': 1, 'V65.42': 1, 'E003.1': 1, '724.09': 1, '961.7': 1, '961.9': 1, '641.01': 1, '75.8': 1, '176.5': 1, '115.19': 1, '58.23': 1, '372.89': 1, '531.71': 1, '715.92': 1, '97.04': 1, '296.31': 1, '305.92': 1, 'E814.5': 1, '911.4': 1, '378.86': 1, '373.9': 1, '718.42': 1, '808.59': 1, 'E805.2': 1, '784.99': 1, '814.04': 1, '959.4': 1, '079.98': 1, '98.05': 1, '852.23': 1, '200.48': 1, 'E958.3': 1, 'E975': 1, '93.35': 1, '379.93': 1, 'E929.5': 1, 'V03.7': 1, 'E001.1': 1, 'E013.9': 1, '327.20': 1, '209.17': 1, '83.86': 1, '93.03': 1, '52.21': 1, '372.74': 1, '14.59': 1, '726.79': 1, '952.19': 1, '377.33': 1, '011.86': 1, '59.99': 1, '851.94': 1, '365.44': 1, '288.59': 1, '625.70': 1, '55.12': 1, '564.6': 1, '625.1': 1, '713.8': 1, 'V09.90': 1, '388.42': 1, '091.9': 1, '50.94': 1, '008.41': 1, 'E827.8': 1, '200.24': 1, '712.96': 1, '304.72': 1, '718.21': 1, '327.19': 1, '661.31': 1, '665.41': 1, '75.51': 1, '33.49': 1, '372.10': 1, '204.80': 1, '282.43': 1, '726.65': 1, '711.95': 1, '719.85': 1, '789.61': 1, '508.2': 1, 'E890.8': 1, '852.32': 1, '807.11': 1, '81.29': 1, '800.80': 1, '376.52': 1, 'E883.9': 1, '704.01': 1, '375.20': 1, 'V61.04': 1, '719.30': 1, '77.98': 1, '918.9': 1, '77.95': 1, '194.3': 1, '352.4': 1, '868.00': 1, '994.9': 1, '225.3': 1, '487.8': 1, '32.22': 1, '529.3': 1, '646.71': 1, '642.52': 1, '657.01': 1, 'V91.03': 1, 'V49.66': 1, '027.9': 1, '995.86': 1, '825.34': 1, '719.04': 1, '919.6': 1, '747.20': 1, '170.3': 1, '277.09': 1, '277.02': 1, 'E029.9': 1, '996.51': 1, 'V78.9': 1, '365.52': 1, '12.64': 1, '88.4': 1, 'V53.02': 1, '352.3': 1, '249.50': 1, '833.13': 1, '833.03': 1, 'E016.2': 1, '77.12': 1, 'V04.5': 1, '62.99': 1, '63.09': 1, '50.93': 1, '869.1': 1, 'V90.10': 1, '729.99': 1, '200.47': 1, '800.11': 1, '79.83': 1, '692.79': 1, '207.22': 1, '711.20': 1, '215.3': 1, '968.3': 1, '378.50': 1, '642.14': 1, '38.35': 1, '33.73': 1, '51.42': 1, '203.82': 1, '927.01': 1, '110.6': 1, '23.0': 1, '38.69': 1, '607.85': 1, 'V44.51': 1, '209.63': 1, '733.93': 1, '387.9': 1, '930.9': 1, '39.89': 1, '457.2': 1, 'V84.89': 1, '532.61': 1, '624.02': 1, '910.4': 1, 'V15.85': 1, '941.26': 1, '943.21': 1, '719.44': 1, '282.41': 1, '534.30': 1, '706.9': 1, '17.7': 1, '86.19': 1, '649.34': 1, '648.84': 1, '372.9': 1, '443.82': 1, '718.35': 1, '379.52': 1, '368.01': 1, '014.85': 1, '29.31': 1, '870.4': 1, '82.33': 1, '82.79': 1, '438.14': 1, '48.73': 1, '694.0': 1, '339.02': 1, '853.04': 1, '209.30': 1, '402.11': 1, '525.79': 1, '800.60': 1, 'V90.39': 1, 'E967.1': 1, '282.0': 1, '695.51': 1, '634.21': 1, '380.14': 1, '018.90': 1, '344.31': 1, '718.40': 1, '46.14': 1, '556.5': 1, '296.21': 1, '590.01': 1, '370.24': 1, '386.2': 1, '915.2': 1, 'E007.0': 1, '183.2': 1, '928.21': 1, 'E939.9': 1, '908.3': 1, '38.01': 1, '838.13': 1, '878.3': 1, '62.61': 1, 'E856': 1, 'E817.8': 1, '524.89': 1, '851.03': 1, '363.15': 1, '963.5': 1, '315.4': 1, '315.39': 1, '833.01': 1, '997.41': 1, '058.21': 1, '39.77': 1, '872.64': 1, '56.72': 1, '995.82': 1, '728.83': 1, '346.91': 1, '607.81': 1, '091.52': 1, '825.39': 1, '618.82': 1, '729.91': 1, '801.19': 1, '214.0': 1, 'V61.07': 1, '183.8': 1, '173.41': 1, '298.2': 1, '999.4': 1, '564.4': 1, '17.71': 1})
# for code,count in another_counter.items():
#     if count != counts.get(code):
#         print(code)

In [56]:

codes_50 = set(codes_50)
for splt in ['train', 'dev', 'test']:
    hadm_ids = set()
    with open('%s/%s_50_hadm_ids.csv' % (conf.MIMIC_3_DIR, splt), 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
            
    labeled_notes_subset = labeled_notes_df[labeled_notes_df['HADM_ID'].isin(hadm_ids)]
    modified_labels = []

    for index, row in labeled_notes_subset.iterrows():
        codes = str(row['LABELS']).split(';')
        filtered_codes = [code for code in codes if code in codes_50]
        if len(filtered_codes) > 0:
            modified_labels.append(';'.join(filtered_codes))
    labeled_notes_subset['LABELS'] = modified_labels
#     labeled_notes_subset['LENGTH'] = labeled_notes_subset.apply(word_count, axis=1)
    labeled_notes_subset = labeled_notes_subset.sort_values(by=['LENGTH'])
    labeled_notes_subset.to_csv(os.path.join(conf.MIMIC_3_DIR, f'{splt}_{Y}.csv'),index=False)         

/home/xiexc/miniconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
labeled_notes_subset.head()

,HADM_ID,LABELS,SUBJECT_ID,TEXT,LENGTH
50281,193800,518.81;96.04;96.71;96.04;96.71,92003,"[{'topic': 'service', 'content': 'admission da...",227
51215,158927,401.9;V45.81;96.71,95088,"[{'topic': 'service', 'content': 'admission da...",346
51809,129034,038.9;995.92;427.31;250.00;96.71,96937,"[{'topic': 'service', 'content': 'admission da...",353
50414,124309,38.93;96.04;96.71,92458,"[{'topic': 'service', 'content': 'admission da...",374
52360,101071,401.9;414.01;96.71,98802,"[{'topic': 'service', 'content': 'admission da...",389


In [58]:
labeled_notes_df[labeled_notes_df['HADM_ID'] == 193800]

,HADM_ID,LABELS,SUBJECT_ID,TEXT,LENGTH
50281,193800,806.01;518.81;348.1;807.01;427.5;E826.1;288.50...,92003,"[{'topic': 'service', 'content': 'admission da...",227


In [59]:
## read code relationship
relations = {}
with open('{}/ICD9_parent_child_relations'.format('/'.join(conf.MIMIC_3_DIR.split('/')[:-1])),'r') as inf:
    for line in inf:
        items = line.split()
        parent = items[0]
        child = items[1]
        if child in relations:
            relations[child].append(parent)
        else:
            relations[child] = [parent]
print(relations)

{'00.0': ['00'], '00.1': ['00'], '00.2': ['00'], '00.3': ['00'], '00.4': ['00'], '00.5': ['00'], '00.6': ['00'], '00.7': ['00'], '00.8': ['00'], '00.9': ['00'], '00': ['00-99.99'], '01-05.99': ['00-99.99'], '06-07.99': ['00-99.99'], '08-16.99': ['00-99.99'], '18-20.99': ['00-99.99'], '21-29.99': ['00-99.99'], '30-34.99': ['00-99.99'], '35-39.99': ['00-99.99'], '40-41.99': ['00-99.99'], '42-54.99': ['00-99.99'], '55-59.99': ['00-99.99'], '60-64.99': ['00-99.99'], '65-71.99': ['00-99.99'], '72-75.99': ['00-99.99'], '76-84.99': ['00-99.99'], '85-86.99': ['00-99.99'], '87-99.99': ['00-99.99'], '00.01': ['00.0'], '00.02': ['00.0'], '00.03': ['00.0'], '00.09': ['00.0'], '00.10': ['00.1'], '00.11': ['00.1'], '00.12': ['00.1'], '00.13': ['00.1'], '00.14': ['00.1'], '00.15': ['00.1'], '00.16': ['00.1'], '00.17': ['00.1'], '00.18': ['00.1'], '00.19': ['00.1'], '00.21': ['00.2'], '00.22': ['00.2'], '00.23': ['00.2'], '00.24': ['00.2'], '00.25': ['00.2'], '00.28': ['00.2'], '00.29': ['00.2'], '00.

In [71]:
## 
from queue import Queue
def construct_relation_subtree(code_subset, relations):
    q = Queue()
    sub_relations = {}
    node_trace = {}
    for code in code_subset:
        q.put(code)
    while not q.empty():
        code = q.get()
        if code in relations:
            parents = relations[code]
            sub_relations[code] = parents
            for parent in parents:
                if parent not in node_trace:
                    node_trace[parent] = 1
                    q.put(parent)
    all_parents = list(node_trace.keys())
    return sub_relations, all_parents

code50_tree, code50_parents = construct_relation_subtree(codes_50, relations)
all_codes = [code[0] for code in codes_sort_by_frequency]
full_tree, full_parents = construct_relation_subtree(all_codes, relations)

with open('{}/TOP_50_relation.csv'.format(conf.MIMIC_3_DIR),'w') as inf:
    for child, parents in code50_tree.items():
        for parent in parents:
            inf.write(parent+'\t'+child+'\n')
        
        
with open('{}/FULL_relation.csv'.format(conf.MIMIC_3_DIR),'w') as inf:
    for child, parents in full_tree.items():
        for parent in parents:
            inf.write(parent+'\t'+child+'\n')
                         
        